In [ ]:
# Download data from DL and save a csv file with filename, identifier, scientificName, CommonName, and Comments columns

## Note that you might have to download in segments because of discover life black listing ip addresses
import os
import requests
from bs4 import BeautifulSoup
import csv

# Function to download an image
def download_image(url, folder, filename):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        filepath = os.path.join(folder, filename)
        with open(filepath, 'wb') as file:
            for chunk in response.iter_content(1024):
                file.write(chunk)
        return filepath
    return None

# Configurables
output_folder = "/Users/markfisher/Desktop/cr_moth_classification/1_dataset_prep/ciee_monte_verde_dataset/cr_images"
csv_filename = "fisher_image_data.csv"
photoAuthorCode = "I_MFS"
baseDownloadUrl = f"https://www.discoverlife.org/mp/20p?res=640&see={photoAuthorCode}/"


# Define the folder for saving images
os.makedirs(output_folder, exist_ok=True)

# Prepare CSV file
csv_headers = ["Filename", "Identifier", "scientificName", "CommonName", "Comments"]

# Open the CSV file once and manage it across all iterations
with open(csv_filename, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(csv_headers)  # Write headers only once

# Loop through the URLs
# for pageNum in range(0, 359):
for pageNum in range(118, 164): # Note that I had to do this in subsections to get all of the data
    url = f"{baseDownloadUrl}{pageNum:04}&flags=col9:"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Append rows to the CSV file
    with open(csv_filename, "a", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)

        # Parse the HTML to find image entries
        for td in soup.find_all("td", valign="top"):
            img_tag = td.find("img")
            if img_tag and "src" in img_tag.attrs:
                img_src = img_tag["src"]
                img_url = f"https://www.discoverlife.org{img_src}"
                filename = os.path.basename(img_src)

                # Download the image
                download_image(img_url, output_folder, filename)

                # Extract text elements (identifier, scientific name, common name, comments)
                text_elements = td.find_all(string=True)
                text_values = [text.strip() for text in text_elements if text.strip()]

                # Ensure exactly 4 columns (fill with None if missing)
                while len(text_values) < 4:
                    text_values.append(None)

                # Write the data row to CSV
                writer.writerow([filename] + text_values[:4])

    print(f"Processed page {pageNum:04}: Images downloaded and data appended to '{csv_filename}'.")

print(f"All pages processed. Images saved to '{output_folder}' and metadata saved to '{csv_filename}'.")


In [ ]:
# Combine all of the target csvs generated from the above download runs

import os
import pandas as pd

# Specify the directory containing the CSV files
input_directory = "/Users/markfisher/Desktop/cr_moth_classification/1_dataset_prep/ciee_monte_verde_dataset/scraped_image_data/"  # Change this to the path where your CSV files are stored
output_file = "combined_output.csv"  # Name of the output file

# Get a list of all CSV files in the directory
csv_files = [file for file in os.listdir(input_directory) if file.endswith('.csv')]

# Combine the CSV files
combined_df = pd.DataFrame()
for csv_file in csv_files:
    file_path = os.path.join(input_directory, csv_file)
    df = pd.read_csv(file_path)
    combined_df = pd.concat([combined_df, df], ignore_index=True)

# Save the combined data to a new CSV file
combined_df.to_csv(output_file, index=False)

print(f"Combined {len(csv_files)} files into {output_file}")


In [ ]:
# Get all unique sciNames from the combined csv file above

## This is only needed when creating the initial classification training data. Creates and de-dupes the scientificName column

import pandas as pd

# Specify the input CSV file
input_file = "combined_output.csv"  # Change this to your input CSV file
output_file = "unique_values_output.csv"  # Name of the output file

# Specify the column name to extract unique values from
target_column = "scientificName"  # Replace with the actual column name you want to target

try:
    # Read the input CSV file
    df = pd.read_csv(input_file)

    # Check if the target column exists
    if target_column not in df.columns:
        raise ValueError(f"Column '{target_column}' not found in the input file.")

    # Extract unique values from the target column
    unique_values = df[target_column].drop_duplicates().reset_index(drop=True)

    # Save the unique values to a new CSV file
    unique_values.to_csv(output_file, index=False, header=[target_column])

    print(f"Unique values from column '{target_column}' have been written to {output_file}")

except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
# Run the GBIF tool https://www.gbif.org/tools/species-lookup. Drop that unique_values_output.csv file in the drop in. Click the "Generate CSV" button in the bottom right as it appears. 

## E.g.

In [3]:
!pip install tabulate

In [4]:
import pandas as pd
from IPython.display import Markdown

df = pd.read_csv("/Users/markfisher/Desktop/cr_moth_classification/1_dataset_prep/ciee_monte_verde_dataset/normalized.csv")
markdown_table = df.to_markdown(index=False)
display(Markdown(markdown_table))

|   occurrenceId | verbatimScientificName                      | scientificName                                   |            key | matchType   |   confidence | status   | rank    | kingdom   | phylum        | class          | order           | family         | genus          | species                     | canonicalName               | authorship                   |       usageKey |   acceptedUsageKey |
|---------------:|:--------------------------------------------|:-------------------------------------------------|---------------:|:------------|-------------:|:---------|:--------|:----------|:--------------|:---------------|:----------------|:---------------|:---------------|:----------------------------|:----------------------------|:-----------------------------|---------------:|-------------------:|
|            nan | Lobocleta tenellata                         | Lobocleta tenellata (Möschler, 1886)             |    1.99026e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Lobocleta      | Lobocleta tenellata         | Lobocleta tenellata         | (Möschler, 1886)             |    1.99026e+06 |      nan           |
|            nan | moth Notodontidae                           | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Iridopsis lurida--oberthuri--herse          | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Clemensia leopardina                        | Clemensia leopardina Schaus, 1911                |    5.1149e+06  | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Clemensia      | Clemensia leopardina        | Clemensia leopardina        | Schaus, 1911                 |    5.1149e+06  |      nan           |
|            nan | Rhabdatomis laudamia                        | Rhabdatomis laudamia Druce, 1885                 |    1.81126e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Rhabdatomis    | Rhabdatomis laudamia        | Rhabdatomis laudamia        | Druce, 1885                  |    1.81126e+06 |      nan           |
|            nan | Eupithecia sp_group_san_luis                | Eupithecia Curtis, 1825                          |    1.98237e+06 | HIGHERRANK  |           92 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eupithecia     | nan                         | Eupithecia                  | Curtis, 1825                 |    1.98237e+06 |      nan           |
|            nan | Crambidia myrlosea                          | Crambidia myrlosea Dyar, 1917                    |    1.80554e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Crambidia      | Crambidia myrlosea          | Crambidia myrlosea          | Dyar, 1917                   |    1.80554e+06 |      nan           |
|            nan | Leptostales crossii                         | Leptostales crossii (Hulst, 1900)                |    9.61909e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Leptostales    | Leptostales crossii         | Leptostales crossii         | (Hulst, 1900)                |    9.61909e+06 |      nan           |
|            nan | moth                                        | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Meganola sp_san_luis_b                      | Meganola Dyar, 1898                              |    4.40549e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Nolidae        | Meganola       | nan                         | Meganola                    | Dyar, 1898                   |    4.40549e+06 |      nan           |
|            nan | moth Crambidae                              | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Nephodia auxesia                            | Nephodia auxesia Druce, 1892                     |    1.98148e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nephodia       | Nephodia auxesia            | Nephodia auxesia            | Druce, 1892                  |    1.98148e+06 |      nan           |
|            nan | moth Hemiceras                              | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Nemoria defectiva                           | Nemoria defectiva (Prout, 1932)                  |    1.98126e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nemoria        | Nemoria defectiva           | Nemoria defectiva           | (Prout, 1932)                |    1.98126e+06 |      nan           |
|            nan | Nematocampa arenosa                         | Nematocampa arenosa Butler, 1881                 |    1.99101e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nematocampa    | Nematocampa arenosa         | Nematocampa arenosa         | Butler, 1881                 |    1.99101e+06 |      nan           |
|            nan | moth Lascoria                               | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | moth Tortricidae                            | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Patalene aenetusaria                        | Patalene aenetusaria (Walker, 1860)              |    9.24326e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Patalene       | Patalene aenetusaria        | Patalene aenetusaria        | (Walker, 1860)               |    9.24326e+06 |      nan           |
|            nan | Clemensia cincinnata                        | Clemensia cincinnata Schaus, 1911                |    5.11488e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Clemensia      | Clemensia cincinnata        | Clemensia cincinnata        | Schaus, 1911                 |    5.11488e+06 |      nan           |
|            nan | Brachurapteryx tesserata                    | Brachurapteryx tesserata (Guenée)                |    1.95582e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Brachurapteryx | Brachurapteryx tesserata    | Brachurapteryx tesserata    | (Guenée)                     |    1.95582e+06 |      nan           |
|            nan | Crambidia cephalica                         | Crambidia cephalica (Grote & Robinson, 1870)     |    1.80554e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Crambidia      | Crambidia cephalica         | Crambidia cephalica         | (Grote & Robinson, 1870)     |    1.80554e+06 |      nan           |
|            nan | moth Geometridae                            | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | moth Noctuidae                              | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Glena mopsaria                              | Glena mopsaria (Schaus, 1913)                    |    1.96568e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Glena          | Glena mopsaria              | Glena mopsaria              | (Schaus, 1913)               |    1.96568e+06 |      nan           |
|            nan | Iridopsis validaria                         | Iridopsis validaria Guenée, 1858                 |    5.14859e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Iridopsis      | Iridopsis validaria         | Iridopsis validaria         | Guenée, 1858                 |    5.14859e+06 |      nan           |
|            nan | Micrathetis dasarada                        | Micrathetis dasarada Druce, 1898                 |    1.76469e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Micrathetis    | Micrathetis dasarada        | Micrathetis dasarada        | Druce, 1898                  |    1.76469e+06 |      nan           |
|            nan | Macaria sp_san_luis_a                       | Arthropoda                                       |   54           | HIGHERRANK  |           92 | ACCEPTED | PHYLUM  | Animalia  | Arthropoda    | nan            | nan             | nan            | nan            | nan                         | Arthropoda                  | nan                          |   54           |      nan           |
|            nan | Idaea sp_group_san_luis                     | Idaea Treitschke, 1825                           |    1.97391e+06 | HIGHERRANK  |           92 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Idaea          | nan                         | Idaea                       | Treitschke, 1825             |    1.97391e+06 |      nan           |
|            nan | Cell phone                                  | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | light                                       | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | moth Megalopygidae - Limacodidae            | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | moth plume                                  | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Lophosis labeculata                         | Lophosis labeculata (Hulst, 1887)                |    9.23703e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Lophosis       | Lophosis labeculata         | Lophosis labeculata         | (Hulst, 1887)                |    9.23703e+06 |      nan           |
|            nan | Trygodes amphion                            | Trygodes amphion Schaus, 1912                    |    1.97636e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Trygodes       | Trygodes amphion            | Trygodes amphion            | Schaus, 1912                 |    1.97636e+06 |      nan           |
|            nan | moth Pyralidae                              | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | moth emerald                                | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Afrida ydatodes                             | Afrida ydatodes Dyar, 1913                       |    8.92451e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Nolidae        | Afrida         | Afrida ydatodes             | Afrida ydatodes             | Dyar, 1913                   |    8.92451e+06 |      nan           |
|            nan | Meganola bifiliferata                       | Meganola bifiliferata (Walker, 1863)             |    8.12684e+06 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Nolidae        | Nola           | Nola bifiliferata           | Meganola bifiliferata       | (Walker, 1863)               |    8.12684e+06 |        1.17082e+07 |
|            nan | Clemensia brunneomedia                      | Clemensia brunneomedia Schaus, 1905              |    5.11486e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Clemensia      | Clemensia brunneomedia      | Clemensia brunneomedia      | Schaus, 1905                 |    5.11486e+06 |      nan           |
|            nan | Eucereon xanthoperas                        | Eucereon xanthoperas Hampson, 1898               |    1.81031e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | Eucereon xanthoperas        | Eucereon xanthoperas        | Hampson, 1898                |    1.81031e+06 |      nan           |
|            nan | Eucereon rosinum                            | Eucereon rosina Walker, 1854                     |    1.81028e+06 | FUZZY       |           95 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | Eucereon rosina             | Eucereon rosina             | Walker, 1854                 |    1.81028e+06 |      nan           |
|            nan | Pheia elegans                               | Pheia elegans Druce, 1884                        |    1.81151e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Pheia          | Pheia elegans               | Pheia elegans               | Druce, 1884                  |    1.81151e+06 |      nan           |
|            nan | Idaea subfervens                            | Idaea subfervens (Prout, 1920)                   |    5.14703e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Idaea          | Idaea subfervens            | Idaea subfervens            | (Prout, 1920)                |    5.14703e+06 |      nan           |
|            nan | Hymenia perspectalis                        | Hymenia perspectalis (Hübner, 1796)              |    1.88637e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Hymenia        | Hymenia perspectalis        | Hymenia perspectalis        | (Hübner, 1796)               |    1.88637e+06 |      nan           |
|            nan | insect                                      | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Gn sp_san_luis_1013                         | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Metanema bonadea                            | Metanema bonadea Druce, 1892                     |    5.14962e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Metanema       | Metanema bonadea            | Metanema bonadea            | Druce, 1892                  |    5.14962e+06 |      nan           |
|            nan | moth Palthis                                | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Nematocampa completa                        | Nematocampa completa Warren, 1904                |    1.99102e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nematocampa    | Nematocampa completa        | Nematocampa completa        | Warren, 1904                 |    1.99102e+06 |      nan           |
|            nan | Elaphria subobliqua                         | Elaphria subobliqua Walker, 1858                 |    1.78041e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Elaphria       | Elaphria subobliqua         | Elaphria subobliqua         | Walker, 1858                 |    1.78041e+06 |      nan           |
|            nan | Nemoria vermiculata                         | Nemoria vermiculata (Dognin, 1914)               |    1.98117e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nemoria        | Nemoria vermiculata         | Nemoria vermiculata         | (Dognin, 1914)               |    1.98117e+06 |      nan           |
|            nan | Epimecis matronaria--nasica                 | Epimecis Hübner                                  |    1.98558e+06 | HIGHERRANK  |           92 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Epimecis       | nan                         | Epimecis                    | Hübner                       |    1.98558e+06 |      nan           |
|            nan | Macaria approximaria--gambarina--ostia      | Arthropoda                                       |   54           | HIGHERRANK  |           92 | ACCEPTED | PHYLUM  | Animalia  | Arthropoda    | nan            | nan             | nan            | nan            | nan                         | Arthropoda                  | nan                          |   54           |      nan           |
|            nan | Synnomos sp_san_luis_a                      | Synnomos Guenée                                  |    1.96363e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Synnomos       | nan                         | Synnomos                    | Guenée                       |    1.96363e+06 |      nan           |
|            nan | moth Arctiidae                              | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Euphyia sp_group_san_luis                   | Euphyia Hübner                                   |    1.96512e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Euphyia        | nan                         | Euphyia                     | Hübner                       |    1.96512e+06 |      nan           |
|            nan | Bryolymnia roma                             | Bryolymnia roma Druce, 1894                      |    1.76415e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Bryolymnia     | Bryolymnia roma             | Bryolymnia roma             | Druce, 1894                  |    1.76415e+06 |      nan           |
|            nan | Hampsonodes mastoides                       | Hampsonodes mastoides Hampson, 1910              |    1.79133e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Hampsonodes    | Hampsonodes mastoides       | Hampsonodes mastoides       | Hampson, 1910                |    1.79133e+06 |      nan           |
|            nan | Microxydia ruficomma                        | Microxydia ruficomma Prout, 1910                 |    1.95425e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Microxydia     | Microxydia ruficomma        | Microxydia ruficomma        | Prout, 1910                  |    1.95425e+06 |      nan           |
|            nan | Hypercompe nemophila                        | Hypercompe nemophila Herrich-Schäffer, 1853      |    1.81098e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hypercompe     | Hypercompe nemophila        | Hypercompe nemophila        | Herrich-Schäffer, 1853       |    1.81098e+06 |      nan           |
|            nan | Synchlora astraeoides                       | Synchlora astraeoides (Warren, 1901)             |    1.96209e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Synchlora      | Synchlora astraeoides       | Synchlora astraeoides       | (Warren, 1901)               |    1.96209e+06 |      nan           |
|            nan | moth Mocis                                  | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Oospila venezuelata                         | Oospila venezuelata (Walker, 1861)               |    1.97366e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Oospila        | Oospila venezuelata         | Oospila venezuelata         | (Walker, 1861)               |    1.97366e+06 |      nan           |
|            nan | Idalus perlineosa                           | Idalus perlineosa Rothschild, 1917               |    1.81468e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Idalus         | Idalus perlineosa           | Idalus perlineosa           | Rothschild, 1917             |    1.81468e+06 |      nan           |
|            nan | Dycladia emerita                            | Dycladia emerita                                 |    9.11168e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Dycladia       | Dycladia emerita            | Dycladia emerita            | nan                          |    9.11168e+06 |      nan           |
|            nan | moth new species                            | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Melese monima                               | Melese monima Schaus, 1910                       |    1.81758e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Melese         | Melese monima               | Melese monima               | Schaus, 1910                 |    1.81758e+06 |      nan           |
|            nan | Eubaphe unicolor                            | Eubaphe unicolor (Robinson, 1869)                |    1.98595e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eubaphe        | Eubaphe unicolor            | Eubaphe unicolor            | (Robinson, 1869)             |    1.98595e+06 |      nan           |
|            nan | Melese laodamia                             | Melese laodamia Druce, 1884                      |    1.81759e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Melese         | Melese laodamia             | Melese laodamia             | Druce, 1884                  |    1.81759e+06 |      nan           |
|            nan | Nephodia betala                             | Nephodia betala (Druce, 1892)                    |    1.98166e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nephodia       | Nephodia betala             | Nephodia betala             | (Druce, 1892)                |    1.98166e+06 |      nan           |
|            nan | moth Sphingidae                             | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Elaphria hyposcota                          | Elaphria hyposcota Hampson, 1909                 |    1.78031e+06 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Elaphria       | Elaphria hypophaea          | Elaphria hyposcota          | Hampson, 1909                |    1.78031e+06 |        1.78035e+06 |
|            nan | Thysanopyga carfinia                        | Thysanopyga carfinia (Druce, 1893)               |    1.95838e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Thysanopyga    | Thysanopyga carfinia        | Thysanopyga carfinia        | (Druce, 1893)                |    1.95838e+06 |      nan           |
|            nan | Scena potentia                              | Scena potentia Druce, 1894                       |    1.80452e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Scena          | Scena potentia              | Scena potentia              | Druce, 1894                  |    1.80452e+06 |      nan           |
|            nan | Aclytia heber                               | Aclytia heber Cramer, 1782                       |    5.11505e+06 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Aclytia        | Aclytia heber               | Aclytia heber               | Cramer, 1782                 |    5.11505e+06 |      nan           |
|            nan | Feltia subterranea                          | Feltia subterranea Fabricius, 1794               |    7.40081e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Feltia         | Feltia subterranea          | Feltia subterranea          | Fabricius, 1794              |    7.40081e+06 |      nan           |
|            nan | Gonodes liquida                             | Gonodes liquida Möschler, 1886                   |    1.77493e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Gonodes        | Gonodes liquida             | Gonodes liquida             | Möschler, 1886               |    1.77493e+06 |      nan           |
|            nan | Eois sp_san_luis_s                          | Eois Hübner, 1818                                |    1.98961e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eois           | nan                         | Eois                        | Hübner, 1818                 |    1.98961e+06 |      nan           |
|            nan | Neotherina imperilla                        | Neotherina imperilla (Dognin, 1911)              |    1.99135e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Neotherina     | Neotherina imperilla        | Neotherina imperilla        | (Dognin, 1911)               |    1.99135e+06 |      nan           |
|            nan | Gn sp_san_luis_1011                         | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Lomographa argentata                        | Lomographa argentata (Schaus, 1911)              |    1.98608e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Lomographa     | Lomographa argentata        | Lomographa argentata        | (Schaus, 1911)               |    1.98608e+06 |      nan           |
|            nan | Gn sp_san_luis_1005                         | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Simopteryx torquataria                      | Simopteryx torquataria (Walker, 1860)            |    1.9769e+06  | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Simopteryx     | Simopteryx torquataria      | Simopteryx torquataria      | (Walker, 1860)               |    1.9769e+06  |      nan           |
|            nan | moth Saturniidae                            | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Synchlora gerularia                         | Synchlora gerularia (Hübner)                     |    1.96212e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Synchlora      | Synchlora gerularia         | Synchlora gerularia         | (Hübner)                     |    1.96212e+06 |      nan           |
|            nan | Synchlora dependens                         | Synchlora dependens (Warren, 1904)               |    1.96208e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Synchlora      | Synchlora dependens         | Synchlora dependens         | (Warren, 1904)               |    1.96208e+06 |      nan           |
|            nan | Eois sp_san_luis_b                          | Eois Hübner, 1818                                |    1.98961e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eois           | nan                         | Eois                        | Hübner, 1818                 |    1.98961e+06 |      nan           |
|            nan | Clemensia sp_san_luis_a                     | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Simena luctifera                            | Simena luctifera Walker, 1856                    |    1.96589e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Simena         | Simena luctifera            | Simena luctifera            | Walker, 1856                 |    1.96589e+06 |      nan           |
|            nan | Epimecis semicompleta                       | Epimecis semicompleta (Warren, 1905)             |    5.14924e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Epimecis       | Epimecis semicompleta       | Epimecis semicompleta       | (Warren, 1905)               |    5.14924e+06 |      nan           |
|            nan | Antiblemma pira                             | Antiblemma pira                                  |    1.79467e+06 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Antiblemma     | Antiblemma pira             | Antiblemma pira             | nan                          |    1.79467e+06 |      nan           |
|            nan | Gn sp_san_luis_1003                         | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Anisoperas atropunctaria                    | Anisoperas atropunctaria (Walker, 1862)          |    1.98675e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Anisoperas     | Anisoperas atropunctaria    | Anisoperas atropunctaria    | (Walker, 1862)               |    1.98675e+06 |      nan           |
|            nan | Dyspteris tenuivitta                        | Dyspteris tenuivitta Dognin, 1908                |    1.96135e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Dyspteris      | Dyspteris tenuivitta        | Dyspteris tenuivitta        | Dognin, 1908                 |    1.96135e+06 |      nan           |
|            nan | Macaria abydata--nundinata                  | Arthropoda                                       |   54           | HIGHERRANK  |           92 | ACCEPTED | PHYLUM  | Animalia  | Arthropoda    | nan            | nan             | nan            | nan            | nan                         | Arthropoda                  | nan                          |   54           |      nan           |
|            nan | Anycles adusta                              | Anycles adusta Felder, 1869                      |    1.80728e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Anycles        | Anycles adusta              | Anycles adusta              | Felder, 1869                 |    1.80728e+06 |      nan           |
|            nan | Napata lelex                                | Napata lelex                                     |    1.02778e+07 | EXACT       |           97 | DOUBTFUL | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Uranophora     | Napata lelex                | Napata lelex                | nan                          |    1.02778e+07 |      nan           |
|            nan | Opisthoxia miletia                          | Opisthoxia miletia (Druce, 1892)                 |    1.96026e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Opisthoxia     | Opisthoxia miletia          | Opisthoxia miletia          | (Druce, 1892)                |    1.96026e+06 |      nan           |
|            nan | moth Pyraidae                               | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | moth merald                                 | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Ametris nitocris                            | Ametris nitocris (Cramer)                        |    1.95964e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Ametris        | Ametris nitocris            | Ametris nitocris            | (Cramer)                     |    1.95964e+06 |      nan           |
|            nan | Hagnagora sp_san_luis                       | Hagnagora Druce, 1885                            |    1.97824e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Hagnagora      | nan                         | Hagnagora                   | Druce, 1885                  |    1.97824e+06 |      nan           |
|            nan | Periclina sp_san_luis_a                     | Periclina Guenée                                 |    1.96854e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Periclina      | nan                         | Periclina                   | Guenée                       |    1.96854e+06 |      nan           |
|            nan | Nematocampa c                               | Nematocampa Guenée                               |    1.991e+06   | EXACT       |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nematocampa    | nan                         | Nematocampa                 | Guenée                       |    1.991e+06   |      nan           |
|            nan | moth Herbita                                | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Macrocneme iole                             | Macrocneme iole Druce, 1884                      |    1.80519e+06 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Macrocneme     | Macrocneme chrysitis        | Macrocneme iole             | Druce, 1884                  |    1.80519e+06 |        1.8052e+06  |
|            nan | Isogona continua--natatrix                  | Isogona Guenée, 1852                             |    1.79093e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Isogona        | nan                         | Isogona                     | Guenée, 1852                 |    1.79093e+06 |      nan           |
|            nan | Lichnoptera illudens                        | Lichnoptera illudens Walker, 1856                |    1.8001e+06  | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Lichnoptera    | Lichnoptera illudens        | Lichnoptera illudens        | Walker, 1856                 |    1.8001e+06  |      nan           |
|            nan | moth Noctuidae delta                        | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Nepheloleuca illiturata                     | Nepheloleuca illiturata (Guenée, 1858)           |    1.98958e+06 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nepheloleuca   | Nepheloleuca politia        | Nepheloleuca illiturata     | (Guenée, 1858)               |    1.98958e+06 |        1.14625e+07 |
|            nan | Nemoria pescadora                           | Nemoria pescadora (Beutelspacher-Baigts, 1984)   |    1.98131e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nemoria        | Nemoria pescadora           | Nemoria pescadora           | (Beutelspacher-Baigts, 1984) |    1.98131e+06 |      nan           |
|            nan | Scopula sp_group_san_luis                   | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Semaeopus viridiplaga                       | Semaeopus viridiplaga (Walker, 1861)             |    1.95614e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Semaeopus      | Semaeopus viridiplaga       | Semaeopus viridiplaga       | (Walker, 1861)               |    1.95614e+06 |      nan           |
|            nan | Nola sp_san_luis_a                          | Nola Leach, 1815                                 |    1.80176e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Nolidae        | Nola           | nan                         | Nola                        | Leach, 1815                  |    1.80176e+06 |      nan           |
|            nan | Oxydia trychiata                            | Oxydia trychiata Guenée                          |    1.97675e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Oxydia         | Oxydia trychiata            | Oxydia trychiata            | Guenée                       |    1.97675e+06 |      nan           |
|            nan | moth Melese                                 | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Eucereon tigrata                            | Eucereon tigrata Herrich-Schäffer, 1855          |    1.81017e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | Eucereon tigrata            | Eucereon tigrata            | Herrich-Schäffer, 1855       |    1.81017e+06 |      nan           |
|            nan | Lophocampa citrina                          | Lophocampa citrina Sepp, 1852                    |    1.81776e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Lophocampa     | Lophocampa citrina          | Lophocampa citrina          | Sepp, 1852                   |    1.81776e+06 |      nan           |
|            nan | moth Hydata                                 | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Amastus aconia                              | Amastus aconia Herrich-Schäffer, 1853            |    5.11735e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Amastus        | Amastus aconia              | Amastus aconia              | Herrich-Schäffer, 1853       |    5.11735e+06 |      nan           |
|            nan | Antiblemma concinnula                       | Antiblemma concinnula Walker, 1865               |    1.79464e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Antiblemma     | Antiblemma concinnula       | Antiblemma concinnula       | Walker, 1865                 |    1.79464e+06 |      nan           |
|            nan | Spragueia lepus                             | Spragueia Grote, 1875                            |    1.75876e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Spragueia      | nan                         | Spragueia                   | Grote, 1875                  |    1.75876e+06 |      nan           |
|            nan | Hyponerita tipolis                          | Hyponerita tipolis Druce, 1896                   |    1.80862e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hyponerita     | Hyponerita tipolis          | Hyponerita tipolis          | Druce, 1896                  |    1.80862e+06 |      nan           |
|            nan | moth todo                                   | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Episcepsis thetis                           | Episcepsis thetis                                |    1.21865e+07 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Episcepsis     | Episcepsis thetis           | Episcepsis thetis           | nan                          |    1.21865e+07 |      nan           |
|            nan | Hypercompe icasia                           | Hypercompe icasia Cramer, 1777                   |    1.81112e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hypercompe     | Hypercompe icasia           | Hypercompe icasia           | Cramer, 1777                 |    1.81112e+06 |      nan           |
|            nan | Hampsonodes melagona                        | Hampsonodes melagona Hampson, 1910               |    1.79132e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Hampsonodes    | Hampsonodes melagona        | Hampsonodes melagona        | Hampson, 1910                |    1.79132e+06 |      nan           |
|            nan | moth Pyalidae                               | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Ramphia albizona                            | Ramphia albizona Latreille, 1817                 |    1.7675e+06  | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Ramphia        | Ramphia albizona            | Ramphia albizona            | Latreille, 1817              |    1.7675e+06  |      nan           |
|            nan | Nemoria eugeniae                            | Nemoria eugeniae Pitkin, 1993                    |    1.98121e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nemoria        | Nemoria eugeniae            | Nemoria eugeniae            | Pitkin, 1993                 |    1.98121e+06 |      nan           |
|            nan | Procalypta victorina                        | Procalypta victorina Druce, 1884                 |    1.81249e+06 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Procalypta     | Procalypta subcyanea        | Procalypta victorina        | Druce, 1884                  |    1.81249e+06 |        1.81249e+06 |
|            nan | Itomia opistographa--lignaris               | Itomia Hübner, 1823                              |    4.68824e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Itomia         | nan                         | Itomia                      | Hübner, 1823                 |    4.68824e+06 |      nan           |
|            nan | Thysanoprymna superba                       | Thysanoprymna superba Schaus, 1889               |    1.81529e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Thysanoprymna  | Thysanoprymna superba       | Thysanoprymna superba       | Schaus, 1889                 |    1.81529e+06 |      nan           |
|            nan | Macaria tenuiscripta--trigonata             | Arthropoda                                       |   54           | HIGHERRANK  |           92 | ACCEPTED | PHYLUM  | Animalia  | Arthropoda    | nan            | nan             | nan            | nan            | nan                         | Arthropoda                  | nan                          |   54           |      nan           |
|            nan | Dichorda obliquata                          | Dichorda obliquata Warren, 1904                  |    1.98025e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Dichorda       | Dichorda obliquata          | Dichorda obliquata          | Warren, 1904                 |    1.98025e+06 |      nan           |
|            nan | Xenosoma flavicedes                         | Xenosoma flavisedes Dognin, 1891                 |    5.11637e+06 | FUZZY       |           94 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Xenosoma       | Xenosoma flavisedes         | Xenosoma flavisedes         | Dognin, 1891                 |    5.11637e+06 |      nan           |
|            nan | Herbita amicaria                            | Herbita amicaria (Schaus, 1912)                  |    1.97641e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Herbita        | Herbita amicaria            | Herbita amicaria            | (Schaus, 1912)               |    1.97641e+06 |      nan           |
|            nan | Opharus bimaculata                          | Opharus bimaculata Dewitz, 1877                  |    5.11765e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Opharus        | Opharus bimaculata          | Opharus bimaculata          | Dewitz, 1877                 |    5.11765e+06 |      nan           |
|            nan | Hyalurga sora                               | Hyalurga sora Boisduval, 1870                    |    1.8042e+06  | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hyalurga       | Hyalurga sora               | Hyalurga sora               | Boisduval, 1870              |    1.8042e+06  |      nan           |
|            nan | moth Gelechiidae Dichomeris                 | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Melanolophia sp_san_luis                    | Melanolophia Hulst, 1896                         |    1.9627e+06  | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Melanolophia   | nan                         | Melanolophia                | Hulst, 1896                  |    1.9627e+06  |      nan           |
|            nan | Cliniodes opalalis                          | Cliniodes opalalis Guenée, 1854                  |    1.89128e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Cliniodes      | Cliniodes opalalis          | Cliniodes opalalis          | Guenée, 1854                 |    1.89128e+06 |      nan           |
|            nan | Eois sp_san_luis_f                          | Eois Hübner, 1818                                |    1.98961e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eois           | nan                         | Eois                        | Hübner, 1818                 |    1.98961e+06 |      nan           |
|            nan | moth misc macro                             | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Hygrochroma nondina                         | Hygrochroma nondina Druce, 1892                  |    5.14433e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Hygrochroma    | Hygrochroma nondina         | Hygrochroma nondina         | Druce, 1892                  |    5.14433e+06 |      nan           |
|            nan | Leucanopsis longa                           | Leucanopsis longa Grote, 1880                    |    1.81002e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Leucanopsis    | Leucanopsis longa           | Leucanopsis longa           | Grote, 1880                  |    1.81002e+06 |      nan           |
|            nan | Crinodes schausi                            | Crinodes schausi Rothschild, 1917                |    1.82419e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Notodontidae   | Crinodes       | Crinodes schausi            | Crinodes schausi            | Rothschild, 1917             |    1.82419e+06 |      nan           |
|            nan | Oospila athena                              | Oospila athena (Druce, 1892)                     |    1.97364e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Oospila        | Oospila athena              | Oospila athena              | (Druce, 1892)                |    1.97364e+06 |      nan           |
|            nan | Symphlebia herbosa                          | Symphlebia herbosa Schaus, 1910                  |    1.80528e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Symphlebia     | Symphlebia herbosa          | Symphlebia herbosa          | Schaus, 1910                 |    1.80528e+06 |      nan           |
|            nan | Megaloptera                                 | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Hymenomima umbelularia                      | Hymenomima umbelularia (Hübner)                  |    1.97586e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Hymenomima     | Hymenomima umbelularia      | Hymenomima umbelularia      | (Hübner)                     |    1.97586e+06 |      nan           |
|            nan | Phrudocentra opaca                          | Phrudocentra opaca (Butler, 1881)                |    1.98502e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Phrudocentra   | Phrudocentra opaca          | Phrudocentra opaca          | (Butler, 1881)               |    1.98502e+06 |      nan           |
|            nan | moth Xylophanes                             | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Amaxia apyga                                | Amaxia apyga Hampson, 1901                       |    1.81404e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Amaxia         | Amaxia apyga                | Amaxia apyga                | Hampson, 1901                |    1.81404e+06 |      nan           |
|            nan | Leucotmemis nexa                            | Leucotmemis nexa Herrich-Schäffer, 1854          |    1.81709e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Leucotmemis    | Leucotmemis nexa            | Leucotmemis nexa            | Herrich-Schäffer, 1854       |    1.81709e+06 |      nan           |
|            nan | Hemiceras vecina                            | Hemiceras vecina Schaus, 1901                    |    5.11871e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Notodontidae   | Hemiceras      | Hemiceras vecina            | Hemiceras vecina            | Schaus, 1901                 |    5.11871e+06 |      nan           |
|            nan | Eucereon aroa                               | Eucereon aroa Schaus, 1894                       |    1.81022e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | Eucereon aroa               | Eucereon aroa               | Schaus, 1894                 |    1.81022e+06 |      nan           |
|            nan | Phalaenophana pyramusalis                   | Phalaenophana pyramusalis (Walker, 1859)         |    9.34294e+06 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Phaeolita      | Phaeolita pyramusalis       | Phalaenophana pyramusalis   | (Walker, 1859)               |    9.34294e+06 |        1.79083e+06 |
|            nan | Natada fusca                                | Natada fusca (Druce, 1887)                       |    1.75366e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Limacodidae    | Natada         | Natada fusca                | Natada fusca                | (Druce, 1887)                |    1.75366e+06 |      nan           |
|            nan | Dysschema mariamne                          | Dysschema mariamne Geyer, 1838                   |    5.11742e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Dysschema      | Dysschema mariamne          | Dysschema mariamne          | Geyer, 1838                  |    5.11742e+06 |      nan           |
|            nan | Herminodes sp_san_luis_a                    | Herminodes Guenée, 1852                          |    4.6962e+06  | HIGHERRANK  |           94 | SYNONYM  | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Scolecocampa   | nan                         | Herminodes                  | Guenée, 1852                 |    4.6962e+06  |        1.76518e+06 |
|            nan | Paragonia cruraria                          | Paragonia cruraria (Herrich-Schäffer)            |    5.14436e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Paragonia      | Paragonia cruraria          | Paragonia cruraria          | (Herrich-Schäffer)           |    5.14436e+06 |      nan           |
|            nan | moth Macrodes                               | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Afrida sp_san_luis_a                        | Afrida Möschler, 1886                            |    1.80216e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Nolidae        | Afrida         | nan                         | Afrida                      | Möschler, 1886               |    1.80216e+06 |      nan           |
|            nan | Microphysetica hermeasalis                  | Microphysetica hermeasalis                       |    1.04394e+07 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Microphysetica | Microphysetica hermeasalis  | Microphysetica hermeasalis  | nan                          |    1.04394e+07 |      nan           |
|            nan | Talara mona                                 | Talara mona Dyar, 1914                           |    1.81477e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Talara         | Talara mona                 | Talara mona                 | Dyar, 1914                   |    1.81477e+06 |      nan           |
|            nan | Xenosoma nigromarginatum                    | Xenosoma nigromarginatum Druce, 1886             |    5.11637e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Xenosoma       | Xenosoma nigromarginatum    | Xenosoma nigromarginatum    | Druce, 1886                  |    5.11637e+06 |      nan           |
|            nan | Zale peruncta                               | Zale peruncta Guenée, 1852                       |    5.113e+06   | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Zale           | Zale peruncta               | Zale peruncta               | Guenée, 1852                 |    5.113e+06   |      nan           |
|            nan | Gonodonta paraequalis                       | Gonodonta paraequalis Todd, 1959                 |    1.77004e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Gonodonta      | Gonodonta paraequalis       | Gonodonta paraequalis       | Todd, 1959                   |    1.77004e+06 |      nan           |
|            nan | Perigea leucanoides                         | Perigea leucanioides Hampson, 1908               |    1.79074e+06 | FUZZY       |           95 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Perigea        | Perigea leucanioides        | Perigea leucanioides        | Hampson, 1908                |    1.79074e+06 |      nan           |
|            nan | Dysschema lycaste                           | Dysschema lycaste Klug, 1836                     |    5.11751e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Dysschema      | Dysschema lycaste           | Dysschema lycaste           | Klug, 1836                   |    5.11751e+06 |      nan           |
|            nan | Hypena lollia                               | Hypena lollia Druce, 1890                        |    8.25686e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hypena         | Hypena lollia               | Hypena lollia               | Druce, 1890                  |    8.25686e+06 |      nan           |
|            nan | Eacles ormondei                             | Eacles ormondei Schaus, 1889                     |    1.86511e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Saturniidae    | Eacles         | Eacles ormondei             | Eacles ormondei             | Schaus, 1889                 |    1.86511e+06 |      nan           |
|            nan | moth misc micro                             | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Gn_sp_san_luis_1014                         | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Antiblemma sp_san_luis_a                    | Antiblemma Hübner, 1823                          |    1.79463e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Antiblemma     | nan                         | Antiblemma                  | Hübner, 1823                 |    1.79463e+06 |      nan           |
|            nan | Eucereon relegata                           | Eucereon relegatum Schaus, 1911                  |    1.81015e+06 | FUZZY       |           96 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | Eucereon relegatum          | Eucereon relegatum          | Schaus, 1911                 |    1.81015e+06 |      nan           |
|            nan | Gn sp_san_luis_1004                         | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Hemiceras modesta                           | Hemiceras modesta Butler, 1879                   |    5.11859e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Notodontidae   | Hemiceras      | Hemiceras modesta           | Hemiceras modesta           | Butler, 1879                 |    5.11859e+06 |      nan           |
|            nan | moth Oecophoridae                           | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Cosmosoma advena                            | Cosmosoma advena Druce, 1884                     |    5.11628e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Cosmosoma      | Cosmosoma advena            | Cosmosoma advena            | Druce, 1884                  |    5.11628e+06 |      nan           |
|            nan | Phyllodonta latrata                         | Phyllodonta latrata (Guenée)                     |    1.96396e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Phyllodonta    | Phyllodonta latrata         | Phyllodonta latrata         | (Guenée)                     |    1.96396e+06 |      nan           |
|            nan | Anisoperas lurida                           | Anisoperas Warren, 1895                          |    1.98675e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Anisoperas     | nan                         | Anisoperas                  | Warren, 1895                 |    1.98675e+06 |      nan           |
|            nan | Ammalo helops                               | Ammalo helops Cramer, 1775                       |    1.81418e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Ammalo         | Ammalo helops               | Ammalo helops               | Cramer, 1775                 |    1.81418e+06 |      nan           |
|            nan | Gn_sp_san_luis_1028                         | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Herminodes atrosignata                      | Herminodes atrosignata (Walker, 1858)            |    8.02821e+06 | EXACT       |           97 | DOUBTFUL | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Scolecocampa   | Herminodes atrosignata      | Herminodes atrosignata      | (Walker, 1858)               |    8.02821e+06 |      nan           |
|            nan | Eusarca minucia                             | Eusarca minucia (Druce, 1892)                    |    1.04662e+07 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eusarca        | Eusarca minucia             | Eusarca minucia             | (Druce, 1892)                |    1.04662e+07 |      nan           |
|            nan | Orthofidonia sp_san_luis_a                  | Orthofidonia Packard, 1876                       |    1.9805e+06  | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Orthofidonia   | nan                         | Orthofidonia                | Packard, 1876                |    1.9805e+06  |      nan           |
|            nan | Gn sp_san_luis_1028                         | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Pelochyta affinis                           | Pelochyta affinis Rothschild, 1909               |    1.80678e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Pelochyta      | Pelochyta affinis           | Pelochyta affinis           | Rothschild, 1909             |    1.80678e+06 |      nan           |
|            nan | Nepheloleuca politia                        | Nepheloleuca politia (Cramer, 1777)              |    1.98958e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nepheloleuca   | Nepheloleuca politia        | Nepheloleuca politia        | (Cramer, 1777)               |    1.98958e+06 |      nan           |
|            nan | Diacme mopsalis                             | Diacme mopsalis                                  |    1.15064e+07 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Diacme         | Diacme mopsalis             | Diacme mopsalis             | nan                          |    1.15064e+07 |      nan           |
|            nan | moth Oecophoridae Ethmia                    | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Epitausa dilina                             | Epitausa dilina Herrich-Schäffer, 1858           |    1.77224e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Epitausa       | Epitausa dilina             | Epitausa dilina             | Herrich-Schäffer, 1858       |    1.77224e+06 |      nan           |
|            nan | Hygrochroma olivinaria                      | Hygrochroma olivinaria Herrich-Schäffer, 1855    |    5.14433e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Hygrochroma    | Hygrochroma olivinaria      | Hygrochroma olivinaria      | Herrich-Schäffer, 1855       |    5.14433e+06 |      nan           |
|            nan | Pyrinia selecta                             | Pyrinia selecta Schaus, 1913                     |    1.98899e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Pyrinia        | Pyrinia selecta             | Pyrinia selecta             | Schaus, 1913                 |    1.98899e+06 |      nan           |
|            nan | wasp                                        | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Opharus rudis                               | Opharus rudis Schaus, 1911                       |    5.11762e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Opharus        | Opharus rudis               | Opharus rudis               | Schaus, 1911                 |    5.11762e+06 |      nan           |
|            nan | moth Tineidae Nemapogon                     | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | moth Blastobasidae                          | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Lepidokirbyia vittipes                      | Lepidokirbyia vittipes Walker, 1855              |    1.81838e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Lepidokirbyia  | Lepidokirbyia vittipes      | Lepidokirbyia vittipes      | Walker, 1855                 |    1.81838e+06 |      nan           |
|            nan | = I_PBA10487                                | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Gardinia anopla                             | Gardinia anopla Hering, 1925                     |    5.11788e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Gardinia       | Gardinia anopla             | Gardinia anopla             | Hering, 1925                 |    5.11788e+06 |      nan           |
|            nan | Epicrisias eschara                          | Epicrisias eschara Dyar, 1912                    |    1.80553e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Epicrisias     | Epicrisias eschara          | Epicrisias eschara          | Dyar, 1912                   |    1.80553e+06 |      nan           |
|            nan | Anomis gentilis                             | Anomis gentilis Schaus, 1912                     |    1.78509e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Anomis         | Anomis gentilis             | Anomis gentilis             | Schaus, 1912                 |    1.78509e+06 |      nan           |
|            nan | moth Depressariidae Ethmia                  | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Diphthera festiva                           | Diphthera festiva Fabricius, 1775                |    5.10895e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Diphthera      | Diphthera festiva           | Diphthera festiva           | Fabricius, 1775              |    5.10895e+06 |      nan           |
|            nan | Tosale oviplagalis                          | Tosale oviplagalis Walker, 1866                  |    1.87251e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Pyralidae      | Tosale         | Tosale oviplagalis          | Tosale oviplagalis          | Walker, 1866                 |    1.87251e+06 |      nan           |
|            nan | Eucereon aurantiaca                         | Eucereon aurantiaca Draudt, 1915                 |    1.81031e+06 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | Eucereon pseudarchias       | Eucereon aurantiaca         | Draudt, 1915                 |    1.81031e+06 |        1.81032e+06 |
|            nan | Elysius conspersus                          | Elysius conspersus Walker, 1855                  |    1.81131e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Elysius        | Elysius conspersus          | Elysius conspersus          | Walker, 1855                 |    1.81131e+06 |      nan           |
|            nan | Bagisara laverna                            | Bagisara laverna Druce, 1889                     |    9.56777e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Bagisara       | Bagisara laverna            | Bagisara laverna            | Druce, 1889                  |    9.56777e+06 |      nan           |
|            nan | Gn sp_san_luis_1042                         | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Semiothisa divergentata                     | Semiothisa divergentata (Snellen, 1874)          |    1.95881e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Semiothisa     | Semiothisa divergentata     | Semiothisa divergentata     | (Snellen, 1874)              |    1.95881e+06 |      nan           |
|            nan | moth Darceta                                | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Symphlebia pyrgion                          | Symphlebia pyrgion Druce, 1897                   |    1.80528e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Symphlebia     | Symphlebia pyrgion          | Symphlebia pyrgion          | Druce, 1897                  |    1.80528e+06 |      nan           |
|            nan | Hemiceras nigricosta                        | Hemiceras nigricosta Schaus, 1901                |    5.11872e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Notodontidae   | Hemiceras      | Hemiceras nigricosta        | Hemiceras nigricosta        | Schaus, 1901                 |    5.11872e+06 |      nan           |
|            nan | Amaxia lepida                               | Amaxia lepida Schaus, 1912                       |    1.81405e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Amaxia         | Amaxia lepida               | Amaxia lepida               | Schaus, 1912                 |    1.81405e+06 |      nan           |
|            nan | Gn sp_san_luis_2008                         | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Chetone angulosa                            | Chetone angulosa Walker, 1854                    |    1.80648e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Chetone        | Chetone angulosa            | Chetone angulosa            | Walker, 1854                 |    1.80648e+06 |      nan           |
|            nan | mothe Geometridae                           | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Prochoerodes pilosa                         | Prochoerodes pilosa Warren, 1897                 |    1.96384e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Prochoerodes   | Prochoerodes pilosa         | Prochoerodes pilosa         | Warren, 1897                 |    1.96384e+06 |      nan           |
|            nan | Nemoria astraea                             | Nemoria astraea (Druce, 1892)                    |    1.98125e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nemoria        | Nemoria astraea             | Nemoria astraea             | (Druce, 1892)                |    1.98125e+06 |      nan           |
|            nan | Hypercompe laeta                            | Hypercompe laeta Walker, 1855                    |    1.81111e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hypercompe     | Hypercompe laeta            | Hypercompe laeta            | Walker, 1855                 |    1.81111e+06 |      nan           |
|            nan | Leuciris fimbriaria                         | Leuciris fimbriaria (Stoll, 1781)                |    1.97846e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Leuciris       | Leuciris fimbriaria         | Leuciris fimbriaria         | (Stoll, 1781)                |    1.97846e+06 |      nan           |
|            nan | moth Crambidae Scoparia                     | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Hypena andraca                              | Hypena andraca Druce, 1890                       |    1.03547e+07 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hypena         | Hypena andraca              | Hypena andraca              | Druce, 1890                  |    1.03547e+07 |      nan           |
|            nan | Rivula leucosticta                          | Rivula leucosticta Swinhoe, 1895                 |    1.76986e+06 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Rivula         | Rivula leucosticta          | Rivula leucosticta          | Swinhoe, 1895                |    1.76986e+06 |      nan           |
|            nan | Macaria delia                               | Macaria delia (Schaus, 1912)                     |    1.06167e+07 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Macaria        | Macaria delia               | Macaria delia               | (Schaus, 1912)               |    1.06167e+07 |      nan           |
|            nan | Himeromima aulis                            | Himeromima aulis (Druce, 1892)                   |    1.98548e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Himeromima     | Himeromima aulis            | Himeromima aulis            | (Druce, 1892)                |    1.98548e+06 |      nan           |
|            nan | Sphacelodes vulneraria                      | Sphacelodes vulneraria (Hübner, 1823)            |    1.99059e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Sphacelodes    | Sphacelodes vulneraria      | Sphacelodes vulneraria      | (Hübner, 1823)               |    1.99059e+06 |      nan           |
|            nan | Leucula meganira                            | Leucula meganira Druce, 1892                     |    1.99065e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Leucula        | Leucula meganira            | Leucula meganira            | Druce, 1892                  |    1.99065e+06 |      nan           |
|            nan | Gn sp_san_luis_2012                         | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Eumarozia beckeri                           | Eumarozia beckeri Clarke, 1973                   |    1.74346e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Tortricidae    | Eumarozia      | Eumarozia beckeri           | Eumarozia beckeri           | Clarke, 1973                 |    1.74346e+06 |      nan           |
|            nan | Glyphodes extorris                          | Glyphodes extorris Dognin, 1905                  |    5.12758e+06 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Glyphodes      | Glyphodes confinis          | Glyphodes extorris          | Dognin, 1905                 |    5.12758e+06 |        5.12756e+06 |
|            nan | Nemoria aturia                              | Nemoria aturia (Druce, 1892)                     |    1.98124e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nemoria        | Nemoria aturia              | Nemoria aturia              | (Druce, 1892)                |    1.98124e+06 |      nan           |
|            nan | Sicya aurunca--medangula                    | Animalia                                         |    1           | HIGHERRANK  |           92 | ACCEPTED | KINGDOM | Animalia  | nan           | nan            | nan             | nan            | nan            | nan                         | Animalia                    | nan                          |    1           |      nan           |
|            nan | Hypena mactatalis                           | Hypena mactatalis Walker, 1859                   |    9.31945e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hypena         | Hypena mactatalis           | Hypena mactatalis           | Walker, 1859                 |    9.31945e+06 |      nan           |
|            nan | Gn sp_san_luis_1014                         | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Hypena parialis                             | Hypena perialis Schaus, 1904                     |    7.74073e+06 | FUZZY       |           93 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hypena         | Hypena perialis             | Hypena perialis             | Schaus, 1904                 |    7.74073e+06 |      nan           |
|            nan | Eois sp_san_luis_e                          | Eois Hübner, 1818                                |    1.98961e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eois           | nan                         | Eois                        | Hübner, 1818                 |    1.98961e+06 |      nan           |
|            nan | Nemoria sp_san_luis_a                       | Nemoria Hübner, 1818                             |    1.98114e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nemoria        | nan                         | Nemoria                     | Hübner, 1818                 |    1.98114e+06 |      nan           |
|            nan | Eusarca nemora                              | Eusarca nemora (Druce, 1892)                     |    1.02887e+07 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eusarca        | Eusarca nemora              | Eusarca nemora              | (Druce, 1892)                |    1.02887e+07 |      nan           |
|            nan | Prochoerodes striata                        | Prochoerodes striata (Stoll)                     |    9.94744e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Prochoerodes   | Prochoerodes striata        | Prochoerodes striata        | (Stoll)                      |    9.94744e+06 |      nan           |
|            nan | Idalus crinis                               | Idalus crinis Druce, 1884                        |    1.81471e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Idalus         | Idalus crinis               | Idalus crinis               | Druce, 1884                  |    1.81471e+06 |      nan           |
|            nan | Brabirodes cerevia                          | Brabirodes cerevia Druce, 1893                   |    1.96865e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Brabirodes     | Brabirodes cerevia          | Brabirodes cerevia          | Druce, 1893                  |    1.96865e+06 |      nan           |
|            nan | Opisthoxia molpadia                         | Opisthoxia molpadia (Druce, 1892)                |    1.96032e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Opisthoxia     | Opisthoxia molpadia         | Opisthoxia molpadia         | (Druce, 1892)                |    1.96032e+06 |      nan           |
|            nan | Eucereon centrale                           | Eucereon centrale Draudt, 1915                   |    1.81031e+06 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | Eucereon formosum           | Eucereon centrale           | Draudt, 1915                 |    1.81031e+06 |        1.81032e+06 |
|            nan | Hypena gaudialis                            | Hypena gaudialis Schaus, 1912                    |    1.0444e+07  | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hypena         | Hypena gaudialis            | Hypena gaudialis            | Schaus, 1912                 |    1.0444e+07  |      nan           |
|            nan | Hemiceras pernubila                         | Hemiceras pernubila Dyar, 1908                   |    5.11878e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Notodontidae   | Hemiceras      | Hemiceras pernubila         | Hemiceras pernubila         | Dyar, 1908                   |    5.11878e+06 |      nan           |
|            nan | moth Anomis                                 | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Prolimacodes montanus--triangulifera        | Prolimacodes Schaus, 1896                        |    1.75282e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Limacodidae    | Prolimacodes   | nan                         | Prolimacodes                | Schaus, 1896                 |    1.75282e+06 |      nan           |
|            nan | Ptychamalia sp_san_luis_a                   | Ptychamalia Prout, 1932                          |    1.96353e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Ptychamalia    | nan                         | Ptychamalia                 | Prout, 1932                  |    1.96353e+06 |      nan           |
|            nan | Pantherodes conglomerata                    | Pantherodes conglomerata (Warren, 1894)          |    1.97896e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Pantherodes    | Pantherodes conglomerata    | Pantherodes conglomerata    | (Warren, 1894)               |    1.97896e+06 |      nan           |
|            nan | Tosale aucta                                | Tosale aucta Hampson, 1897                       |    1.87252e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Pyralidae      | Tosale         | Tosale aucta                | Tosale aucta                | Hampson, 1897                |    1.87252e+06 |      nan           |
|            nan | Gonodonta holosericea                       | Gonodonta holosericea Guenée, 1852               |    1.77003e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Gonodonta      | Gonodonta holosericea       | Gonodonta holosericea       | Guenée, 1852                 |    1.77003e+06 |      nan           |
|            nan | Halysidota orientalis                       | Halysidota orientalis Rothschild, 1909           |    1.81433e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Halysidota     | Halysidota orientalis       | Halysidota orientalis       | Rothschild, 1909             |    1.81433e+06 |      nan           |
|            nan | Synnomos firmamentaria                      | Synnomos firmamentaria Guenée, 1857              |    1.96363e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Synnomos       | Synnomos firmamentaria      | Synnomos firmamentaria      | Guenée, 1857                 |    1.96363e+06 |      nan           |
|            nan | Amaxia osmophora--pardalis                  | Amaxia Walker, 1855                              |    1.81401e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Amaxia         | nan                         | Amaxia                      | Walker, 1855                 |    1.81401e+06 |      nan           |
|            nan | Trygodes merta                              | Trygodes merta Druce, 1892                       |    1.97636e+06 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Trygodes       | Trygodes solaniferata       | Trygodes merta              | Druce, 1892                  |    1.97636e+06 |        1.97637e+06 |
|            nan | Letis herilia                               | Letis herilia Stoll, 1780                        |    1.78834e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Letis          | Letis herilia               | Letis herilia               | Stoll, 1780                  |    1.78834e+06 |      nan           |
|            nan | Eucereon tripunctatum                       | Eucereon tripunctatum Druce, 1884                |    1.81036e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | Eucereon tripunctatum       | Eucereon tripunctatum       | Druce, 1884                  |    1.81036e+06 |      nan           |
|            nan | Gn sp_san_luis_8019                         | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Cosmosoma centralis                         | Cosmosoma centralis Walker, 1854                 |    5.11615e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Cosmosoma      | Cosmosoma centralis         | Cosmosoma centralis         | Walker, 1854                 |    5.11615e+06 |      nan           |
|            nan | Artace cribrarius                           | Artace cribrarius (Ljungh, 1825)                 |    1.7329e+06  | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Lasiocampidae  | Artace         | Artace cribrarius           | Artace cribrarius           | (Ljungh, 1825)               |    1.7329e+06  |      nan           |
|            nan | moth Phyllodonta                            | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Antiblemma harmodia                         | Antiblemma harmodia Schaus, 1901                 |    1.79464e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Antiblemma     | Antiblemma harmodia         | Antiblemma harmodia         | Schaus, 1901                 |    1.79464e+06 |      nan           |
|            nan | Epitausa atriplaga                          | Epitausa atriplaga Walker, 1858                  |    1.77222e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Epitausa       | Epitausa atriplaga          | Epitausa atriplaga          | Walker, 1858                 |    1.77222e+06 |      nan           |
|            nan | Glyphodes rubrocinctalis                    | Glyphodes rubrocinctalis (Guenée, 1854)          |    9.49951e+06 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Trithyris      | Prenesta rubrocinctalis     | Glyphodes rubrocinctalis    | (Guenée, 1854)               |    9.49951e+06 |        1.08613e+07 |
|            nan | Gizma undilinealis                          | Gizama undilinealis Schaus, 1916                 |    1.76553e+06 | FUZZY       |           85 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Gizama         | Gizama undilinealis         | Gizama undilinealis         | Schaus, 1916                 |    1.76553e+06 |      nan           |
|            nan | Sericoptera mahometaria                     | Sericoptera mahometaria (Herrich-Schäffer, 1853) |    1.96609e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Sericoptera    | Sericoptera mahometaria     | Sericoptera mahometaria     | (Herrich-Schäffer, 1853)     |    1.96609e+06 |      nan           |
|            nan | Hemiceras punctata                          | Hemiceras punctata Dognin, 1889                  |    5.11866e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Notodontidae   | Hemiceras      | Hemiceras punctata          | Hemiceras punctata          | Dognin, 1889                 |    5.11866e+06 |      nan           |
|            nan | Phrudocentra trimaculata                    | Phrudocentra trimaculata (Warren, 1901)          |    1.98501e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Phrudocentra   | Phrudocentra trimaculata    | Phrudocentra trimaculata    | (Warren, 1901)               |    1.98501e+06 |      nan           |
|            nan | Altinote ozomene                            | Altinote ozomene                                 |    1.04871e+07 | EXACT       |           97 | DOUBTFUL | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Nymphalidae    | Acraea         | Altinote ozomene            | Altinote ozomene            | nan                          |    1.04871e+07 |      nan           |
|            nan | Phyle cartago                               | Phyle cartago Rindge, 1990                       |    1.96427e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Phyle          | Phyle cartago               | Phyle cartago               | Rindge, 1990                 |    1.96427e+06 |      nan           |
|            nan | Eusarca crameraria--green                   | Eusarca Hübner                                   |    7.51469e+06 | HIGHERRANK  |           92 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eusarca        | nan                         | Eusarca                     | Hübner                       |    7.51469e+06 |      nan           |
|            nan | Melese amastris                             | Melese amastris Druce, 1884                      |    1.81756e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Melese         | Melese amastris             | Melese amastris             | Druce, 1884                  |    1.81756e+06 |      nan           |
|            nan | Gn_sp_san_luis_1011                         | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Phyllodonta indeterminata                   | Phyllodonta indeterminata Schaus, 1901           |    1.96398e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Phyllodonta    | Phyllodonta indeterminata   | Phyllodonta indeterminata   | Schaus, 1901                 |    1.96398e+06 |      nan           |
|            nan | = I_PBA13087                                | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Scaptius vinasia                            | Scaptius vinasia Schaus, 1910                    |    1.80589e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Scaptius       | Scaptius vinasia            | Scaptius vinasia            | Schaus, 1910                 |    1.80589e+06 |      nan           |
|            nan | Microxydia sp_san_luis_a                    | Microxydia Warren, 1895                          |    1.95424e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Microxydia     | nan                         | Microxydia                  | Warren, 1895                 |    1.95424e+06 |      nan           |
|            nan | Crinodes besckei                            | Crinodes besckei Hübner, 1824                    |    1.82418e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Notodontidae   | Crinodes       | Crinodes besckei            | Crinodes besckei            | Hübner, 1824                 |    1.82418e+06 |      nan           |
|            nan | Argyria lacteella                           | Argyria lacteella Fabricius, 1794                |    5.12672e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Argyria        | Argyria lacteella           | Argyria lacteella           | Fabricius, 1794              |    5.12672e+06 |      nan           |
|            nan | Eucereon sp_san_luis_c                      | Eucereon Hübner, 1819                            |    1.81014e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | nan                         | Eucereon                    | Hübner, 1819                 |    1.81014e+06 |      nan           |
|            nan | Racasta spatiaria                           | Racasta spatiaria (Guenée)                       |    1.98883e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Racasta        | Racasta spatiaria           | Racasta spatiaria           | (Guenée)                     |    1.98883e+06 |      nan           |
|            nan | Gn sp_san_luis_2028                         | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Cosmosoma pudica                            | Cosmosoma pudica Druce, 1894                     |    5.11625e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Cosmosoma      | Cosmosoma pudica            | Cosmosoma pudica            | Druce, 1894                  |    5.11625e+06 |      nan           |
|            nan | Macaria regulata                            | Macaria regulata (Fabricius, 1775)               |    1.01271e+07 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Macaria        | Macaria regulata            | Macaria regulata            | (Fabricius, 1775)            |    1.01271e+07 |      nan           |
|            nan | Phrudocentra janeira                        | Phrudocentra janeira (Schaus, 1897)              |    1.98502e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Phrudocentra   | Phrudocentra janeira        | Phrudocentra janeira        | (Schaus, 1897)               |    1.98502e+06 |      nan           |
|            nan | moth Pyralidae Acrobasis                    | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Podalia contigua                            | Podalia contigua (Walker, 1866)                  |    5.10519e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Megalopygidae  | Podalia        | Podalia contigua            | Podalia contigua            | (Walker, 1866)               |    5.10519e+06 |      nan           |
|            nan | Therinia transversaria                      | Therinia transversaria Druce, 1887               |    1.01036e+07 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Saturniidae    | Therinia       | Therinia transversaria      | Therinia transversaria      | Druce, 1887                  |    1.01036e+07 |      nan           |
|            nan | Oxydia translinquens                        | Oxydia translinquens (Walker, 1860)              |    1.97665e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Oxydia         | Oxydia translinquens        | Oxydia translinquens        | (Walker, 1860)               |    1.97665e+06 |      nan           |
|            nan | Spragueia apicalis                          | Spragueia apicalis (Herrich-Schäffer, 1868)      |    7.91115e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Spragueia      | Spragueia apicalis          | Spragueia apicalis          | (Herrich-Schäffer, 1868)     |    7.91115e+06 |      nan           |
|            nan | Hypercompe albescens                        | Hypercompe albescens Hampson, 1901               |    1.8111e+06  | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hypercompe     | Hypercompe albescens        | Hypercompe albescens        | Hampson, 1901                |    1.8111e+06  |      nan           |
|            nan | Rhescyntis hippodamia                       | Rhescyntis hippodamia Cramer, 1777               |    1.86584e+06 | EXACT       |           98 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Saturniidae    | Rhescyntis     | Rhescyntis hippodamia       | Rhescyntis hippodamia       | Cramer, 1777                 |    1.86584e+06 |      nan           |
|            nan | = I_PBA14372                                | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | moth species group2                         | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Sericoptera reductata                       | Sericoptera reducta Warren, 1909                 |    1.96608e+06 | FUZZY       |           92 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Sericoptera    | Sericoptera area            | Sericoptera reducta         | Warren, 1909                 |    1.96608e+06 |        1.96607e+06 |
|            nan | Argyrotome prospectata                      | Argyrotome prospectata (Snellen, 1874)           |    1.96735e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Argyrotome     | Argyrotome prospectata      | Argyrotome prospectata      | (Snellen, 1874)              |    1.96735e+06 |      nan           |
|            nan | Crinodes ritsemae                           | Crinodes ritsemae Druce, 1898                    |    1.82419e+06 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Notodontidae   | Crinodes       | Crinodes besckei            | Crinodes ritsemae           | Druce, 1898                  |    1.82419e+06 |        1.82418e+06 |
|            nan | Polla hemeraria                             | Polla hemeraria Dyar, 1910                       |    5.14404e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Polla          | Polla hemeraria             | Polla hemeraria             | Dyar, 1910                   |    5.14404e+06 |      nan           |
|            nan | Cosmosoma stibostictum                      | Cosmosoma stibosticta Butler, 1876               |    5.11621e+06 | FUZZY       |           95 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Cosmosoma      | Cosmosoma stibosticta       | Cosmosoma stibosticta       | Butler, 1876                 |    5.11621e+06 |      nan           |
|            nan | Thysanoprymna haemorrhoides                 | Thysanoprymna haemorrhoides Schaus, 1905         |    1.8153e+06  | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Thysanoprymna  | Thysanoprymna haemorrhoides | Thysanoprymna haemorrhoides | Schaus, 1905                 |    1.8153e+06  |      nan           |
|            nan | Macrodes cynara                             | Macrodes cynara Cramer, 1775                     |    1.77772e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Macrodes       | Macrodes cynara             | Macrodes cynara             | Cramer, 1775                 |    1.77772e+06 |      nan           |
|            nan | Hypena livia                                | Hypena livia Druce, 1890                         |    1.15407e+07 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hypena         | Hypena livia                | Hypena livia                | Druce, 1890                  |    1.15407e+07 |      nan           |
|            nan | Ormetica maura--triangularis                | Ormetica Clemens, 1861                           |    1.80362e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Ormetica       | nan                         | Ormetica                    | Clemens, 1861                |    1.80362e+06 |      nan           |
|            nan | Dysschema magdala                           | Dysschema magdala Boisduval, 1870                |    5.11755e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Dysschema      | Dysschema magdala           | Dysschema magdala           | Boisduval, 1870              |    5.11755e+06 |      nan           |
|            nan | Ormetica taeniata--temperata                | Ormetica Clemens, 1861                           |    1.80362e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Ormetica       | nan                         | Ormetica                    | Clemens, 1861                |    1.80362e+06 |      nan           |
|            nan | Trauaxa lua                                 | Trauaxa lua Druce, 1890                          |    1.78853e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Trauaxa        | Trauaxa lua                 | Trauaxa lua                 | Druce, 1890                  |    1.78853e+06 |      nan           |
|            nan | moth Meganola                               | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Syntomeida melanthus                        | Syntomeida melanthus Cramer                      |    1.80913e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Syntomeida     | Syntomeida melanthus        | Syntomeida melanthus        | Cramer                       |    1.80913e+06 |      nan           |
|            nan | Phalaenophana pyramusalis Dark-banded Owlet | Phalaenophana pyramusalis (Walker, 1859)         |    9.34294e+06 | EXACT       |           96 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Phaeolita      | Phaeolita pyramusalis       | Phalaenophana pyramusalis   | (Walker, 1859)               |    9.34294e+06 |        1.79083e+06 |
|            nan | Lepidodes gallopavo                         | Lepidodes gallopavo Druce                        |    5.11089e+06 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Lepidodes      | Lepidodes limbulata         | Lepidodes gallopavo         | Druce                        |    5.11089e+06 |        5.11089e+06 |
|            nan | Pantographa expansalis                      | Pantographa Lederer, 1863                        |    1.88468e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Pantographa    | nan                         | Pantographa                 | Lederer, 1863                |    1.88468e+06 |      nan           |
|            nan | Sematura luna                               | Sematura lunus (Linnaeus, 1758)                  |    9.52713e+06 | FUZZY       |           96 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sematuridae    | Nothus         | Nothus lunus                | Sematura lunus              | (Linnaeus, 1758)             |    9.52713e+06 |        5.14328e+06 |
|            nan | Schrankia macula                            | Schrankia macula Druce, 1891                     |    9.52238e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Schrankia      | Schrankia macula            | Schrankia macula            | Druce, 1891                  |    9.52238e+06 |      nan           |
|            nan | Polypoetes villia                           | Polypoetes villia Druce, 1897                    |    1.82472e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Notodontidae   | Polypoetes     | Polypoetes villia           | Polypoetes villia           | Druce, 1897                  |    1.82472e+06 |      nan           |
|            nan | Urania fulgens                              | Urania fulgens Walker, 1854                      |    5.14323e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Uraniidae      | Urania         | Urania fulgens              | Urania fulgens              | Walker, 1854                 |    5.14323e+06 |      nan           |
|            nan | Opisthoxia uncinata                         | Opisthoxia uncinata (Schaus, 1912)               |    1.96025e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Opisthoxia     | Opisthoxia uncinata         | Opisthoxia uncinata         | (Schaus, 1912)               |    1.96025e+06 |      nan           |
|            nan | Castrica phalaenoides                       | Castrica phalaenoides Drury, 1773                |    1.80903e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Castrica       | Castrica phalaenoides       | Castrica phalaenoides       | Drury, 1773                  |    1.80903e+06 |      nan           |
|            nan | Hyalorista sp_san_luis_a                    | Hyalorista Warren, 1892                          |    1.89103e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Hyalorista     | nan                         | Hyalorista                  | Warren, 1892                 |    1.89103e+06 |      nan           |
|            nan | Pseudophaloe troetschi                      | Pseudophaloe troetschi Druce, 1884               |    1.8092e+06  | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Pseudophaloe   | Pseudophaloe troetschi      | Pseudophaloe troetschi      | Druce, 1884                  |    1.8092e+06  |      nan           |
|            nan | Hypena hemonalis                            | Hypena hemonalis Walker, 1859                    |    9.43658e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hypena         | Hypena hemonalis            | Hypena hemonalis            | Walker, 1859                 |    9.43658e+06 |      nan           |
|            nan | Melese asana                                | Melese asana Druce, 1884                         |    1.8176e+06  | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Melese         | Melese asana                | Melese asana                | Druce, 1884                  |    1.8176e+06  |      nan           |
|            nan | Eucereon aeolum                             | Eucereon aeolum Hampson, 1898                    |    1.81035e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | Eucereon aeolum             | Eucereon aeolum             | Hampson, 1898                |    1.81035e+06 |      nan           |
|            nan | Nematocampa straminea                       | Nematocampa straminea (Warren, 1900)             |    1.99102e+06 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nematocampa    | Nematocampa straminea       | Nematocampa straminea       | (Warren, 1900)               |    1.99102e+06 |      nan           |
|            nan | Isochromodes granula                        | Isochromodes granula (Dognin, 1896)              |    1.97566e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Isochromodes   | Isochromodes granula        | Isochromodes granula        | (Dognin, 1896)               |    1.97566e+06 |      nan           |
|            nan | Eucereon sp_san_luis_a                      | Eucereon Hübner, 1819                            |    1.81014e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | nan                         | Eucereon                    | Hübner, 1819                 |    1.81014e+06 |      nan           |
|            nan | moth micro                                  | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Morpho peleides                             | Morpho peleides Kollar, 1850                     |    5.13343e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Nymphalidae    | Morpho         | Morpho peleides             | Morpho peleides             | Kollar, 1850                 |    5.13343e+06 |      nan           |
|            nan | = I_PBA17329                                | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Tornos erectarius                           | Tornos erectarius Grossbeck, 1909                |    1.97539e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Tornos         | Tornos erectarius           | Tornos erectarius           | Grossbeck, 1909              |    1.97539e+06 |      nan           |
|            nan | Bagisara patula                             | Bagisara patula Druce, 1898                      |    1.76759e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Bagisara       | Bagisara patula             | Bagisara patula             | Druce, 1898                  |    1.76759e+06 |      nan           |
|            nan | Dysschema leucophaea                        | Dysschema leucophaea Walker, 1854                |    5.11746e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Dysschema      | Dysschema leucophaea        | Dysschema leucophaea        | Walker, 1854                 |    5.11746e+06 |      nan           |
|            nan | Cyphoedma transvolutata                     | Cyphoedma transvolutata (Walker, 1860)           |    1.98688e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Cyphoedma      | Cyphoedma transvolutata     | Cyphoedma transvolutata     | (Walker, 1860)               |    1.98688e+06 |      nan           |
|            nan | truebug                                     | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Viviennea ardesiaca                         | Viviennea ardesiaca Rothschild, 1909             |    1.808e+06   | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Viviennea      | Viviennea ardesiaca         | Viviennea ardesiaca         | Rothschild, 1909             |    1.808e+06   |      nan           |
|            nan | Spilomela pantheralis                       | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Perigea hippia--micrippia                   | Perigea Guenée, 1852                             |    1.79073e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Perigea        | nan                         | Perigea                     | Guenée, 1852                 |    1.79073e+06 |      nan           |
|            nan | Gonodonta pyrgo                             | Gonodonta pyrgo Cramer                           |    1.77004e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Gonodonta      | Gonodonta pyrgo             | Gonodonta pyrgo             | Cramer                       |    1.77004e+06 |      nan           |
|            nan | Cathydata batina                            | Cathydata batina Druce, 1892                     |    1.9647e+06  | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Cathydata      | Cathydata batina            | Cathydata batina            | Druce, 1892                  |    1.9647e+06  |      nan           |
|            nan | moth Clemensia                              | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Lophocampa modesta                          | Lophocampa modesta Kirby, 1892                   |    1.81777e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Lophocampa     | Lophocampa modesta          | Lophocampa modesta          | Kirby, 1892                  |    1.81777e+06 |      nan           |
|            nan | moth Eulepidotis                            | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Bertholdia albipuncta                       | Bertholdia albipuncta Schaus, 1896               |    5.11549e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Bertholdia     | Bertholdia albipuncta       | Bertholdia albipuncta       | Schaus, 1896                 |    5.11549e+06 |      nan           |
|            nan | Isanthrene monticola                        | Isanthrene monticola Schaus, 1911                |    1.8112e+06  | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Isanthrene     | Isanthrene monticola        | Isanthrene monticola        | Schaus, 1911                 |    1.8112e+06  |      nan           |
|            nan | Carales arizonensis--astur                  | Carales Walker, 1855                             |    1.80929e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Carales        | nan                         | Carales                     | Walker, 1855                 |    1.80929e+06 |      nan           |
|            nan | Eucereon patrona                            | Eucereon patrona Schaus, 1896                    |    1.81027e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | Eucereon patrona            | Eucereon patrona            | Schaus, 1896                 |    1.81027e+06 |      nan           |
|            nan | Opisthoxia phrynearia                       | Opisthoxia phrynearia (Schaus, 1912)             |    1.96028e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Opisthoxia     | Opisthoxia phrynearia       | Opisthoxia phrynearia       | (Schaus, 1912)               |    1.96028e+06 |      nan           |
|            nan | Eusarca melenda                             | Eusarca melenda (Druce, 1892)                    |    1.0279e+07  | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eusarca        | Eusarca melenda             | Eusarca melenda             | (Druce, 1892)                |    1.0279e+07  |      nan           |
|            nan | Semiothisa valmonaria                       | Semiothisa valmonaria Schaus, 1901               |    1.95894e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Semiothisa     | Semiothisa valmonaria       | Semiothisa valmonaria       | Schaus, 1901                 |    1.95894e+06 |      nan           |
|            nan | Herminodes subclara                         | Herminodes Guenée, 1852                          |    4.6962e+06  | HIGHERRANK  |           94 | SYNONYM  | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Scolecocampa   | nan                         | Herminodes                  | Guenée, 1852                 |    4.6962e+06  |        1.76518e+06 |
|            nan | Ascalapha odorata                           | Ascalapha odorata (Linnaeus, 1758)               |    1.7758e+06  | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Ascalapha      | Ascalapha odorata           | Ascalapha odorata           | (Linnaeus, 1758)             |    1.7758e+06  |      nan           |
|            nan | = I_PBA20160                                | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Renia discoloralis                          | Renia discoloralis Guenée, 1854                  |    5.11206e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Renia          | Renia discoloralis          | Renia discoloralis          | Guenée, 1854                 |    5.11206e+06 |      nan           |
|            nan | Spodoptera dolichos                         | Spodoptera dolichos Fabricius, 1794              |    5.10988e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Spodoptera     | Spodoptera dolichos         | Spodoptera dolichos         | Fabricius, 1794              |    5.10988e+06 |      nan           |
|            nan | Hemeroblemma opigena                        | Hemeroblemma opigena Drury, 1773                 |    1.78622e+06 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hemeroblemma   | Hemeroblemma opigena        | Hemeroblemma opigena        | Drury, 1773                  |    1.78622e+06 |      nan           |
|            nan | Argyrotome alba                             | Argyrotome alba (Druce, 1892)                    |    1.96735e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Argyrotome     | Argyrotome alba             | Argyrotome alba             | (Druce, 1892)                |    1.96735e+06 |      nan           |
|            nan | Eois isographata                            | Eois isographata Walker                          |    1.98978e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eois           | Eois isographata            | Eois isographata            | Walker                       |    1.98978e+06 |      nan           |
|            nan | Cosmosoma impudica                          | Cosmosoma impudica Schaus, 1911                  |    5.11615e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Cosmosoma      | Cosmosoma impudica          | Cosmosoma impudica          | Schaus, 1911                 |    5.11615e+06 |      nan           |
|            nan | Hypercompe caudata                          | Hypercompe caudata Walker, 1855                  |    1.81102e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hypercompe     | Hypercompe caudata          | Hypercompe caudata          | Walker, 1855                 |    1.81102e+06 |      nan           |
|            nan | Symphlebia ipsea                            | Symphlebia ipsea Druce, 1884                     |    1.80526e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Symphlebia     | Symphlebia ipsea            | Symphlebia ipsea            | Druce, 1884                  |    1.80526e+06 |      nan           |
|            nan | Neonerita dorsipuncta                       | Neonerita dorsipuncta Hampson, 1901              |    1.80548e+06 | EXACT       |           98 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Neonerita      | Neonerita dorsipuncta       | Neonerita dorsipuncta       | Hampson, 1901                |    1.80548e+06 |      nan           |
|            nan | Hypocrita reedia                            | Hypocrita reedia Schaus, 1910                    |    1.8054e+06  | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hypocrita      | Hypocrita reedia            | Hypocrita reedia            | Schaus, 1910                 |    1.8054e+06  |      nan           |
|            nan | Pseudasellodes fenestraria                  | Pseudasellodes fenestraria (Guenée)              |    1.95705e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Pseudasellodes | Pseudasellodes fenestraria  | Pseudasellodes fenestraria  | (Guenée)                     |    1.95705e+06 |      nan           |
|            nan | Renia vinasalis                             | Renia vinasalis Schaus, 1912                     |    5.11209e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Renia          | Renia vinasalis             | Renia vinasalis             | Schaus, 1912                 |    5.11209e+06 |      nan           |
|            nan | Celiptera levina                            | Celiptera levina Stoll, 1781                     |    1.79283e+06 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Celiptera      | Celiptera levina            | Celiptera levina            | Stoll, 1781                  |    1.79283e+06 |      nan           |
|            nan | Phrygionis polita                           | Phrygionis polita (Cramer)                       |    1.96831e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Phrygionis     | Phrygionis polita           | Phrygionis polita           | (Cramer)                     |    1.96831e+06 |      nan           |
|            nan | Bernathonomus piperita                      | Bernathonomus piperita Herrich-Schäffer, 1855    |    1.81753e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Bernathonomus  | Bernathonomus piperita      | Bernathonomus piperita      | Herrich-Schäffer, 1855       |    1.81753e+06 |      nan           |
|            nan | Herminodes partina                          | Herminodes Guenée, 1852                          |    4.6962e+06  | HIGHERRANK  |           94 | SYNONYM  | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Scolecocampa   | nan                         | Herminodes                  | Guenée, 1852                 |    4.6962e+06  |        1.76518e+06 |
|            nan | Bagisara repanda                            | Bagisara repanda Fabricius, 1793                 |    1.76757e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Bagisara       | Bagisara repanda            | Bagisara repanda            | Fabricius, 1793              |    1.76757e+06 |      nan           |
|            nan | Mesothen temperata                          | Mesothen temperata Schaus, 1911                  |    1.81254e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Mesothen       | Mesothen temperata          | Mesothen temperata          | Schaus, 1911                 |    1.81254e+06 |      nan           |
|            nan | Macaria pernicata                           | Macaria pernicata Guenée                         |    1.03252e+07 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Macaria        | Macaria pernicata           | Macaria pernicata           | Guenée                       |    1.03252e+07 |      nan           |
|            nan | Perasia garnoti                             | Perasia garnoti Guenée, 1852                     |    1.78746e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Perasia        | Perasia garnoti             | Perasia garnoti             | Guenée, 1852                 |    1.78746e+06 |      nan           |
|            nan | Sennia auritalis                            | Sennia auritalis                                 |    1.0252e+07  | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Pyralidae      | Sennia         | Sennia auritalis            | Sennia auritalis            | nan                          |    1.0252e+07  |      nan           |
|            nan | Notarctia proxima                           | Notarctia proxima Guérin-Méneville, 1844         |    1.81541e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Notarctia      | Notarctia proxima           | Notarctia proxima           | Guérin-Méneville, 1844       |    1.81541e+06 |      nan           |
|            nan | Ormetica sp_san_luis_a                      | Ormetica Clemens, 1861                           |    1.80362e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Ormetica       | nan                         | Ormetica                    | Clemens, 1861                |    1.80362e+06 |      nan           |
|            nan | spider                                      | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Massala hieroglyphica                       | Massala hieroglyphica Walker, 1867               |    1.79661e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Massala        | Massala hieroglyphica       | Massala hieroglyphica       | Walker, 1867                 |    1.79661e+06 |      nan           |
|            nan | Nesara casada                               | Nesara casada (Schaus, 1911)                     |    1.73304e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Lasiocampidae  | Nesara         | Nesara casada               | Nesara casada               | (Schaus, 1911)               |    1.73304e+06 |      nan           |
|            nan | Eucereon decora                             | Eucereon Hübner, 1819                            |    1.81014e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | nan                         | Eucereon                    | Hübner, 1819                 |    1.81014e+06 |      nan           |
|            nan | Elaphria deltoides                          | Elaphria deltoides Möschler, 1880                |    1.78044e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Elaphria       | Elaphria deltoides          | Elaphria deltoides          | Möschler, 1880               |    1.78044e+06 |      nan           |
|            nan | Micranthops alceste                         | Micranthops alceste Druce, 1890                  |    1.79368e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Micranthops    | Micranthops alceste         | Micranthops alceste         | Druce, 1890                  |    1.79368e+06 |      nan           |
|            nan | Lesmone formularis                          | Lesmone formularis Geyer, 1837                   |    8.33169e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Lesmone        | Lesmone formularis          | Lesmone formularis          | Geyer, 1837                  |    8.33169e+06 |      nan           |
|            nan | Stenoma luctifica                           | Stenoma luctifica Zeller, 1877                   |    1.83673e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Depressariidae | Stenoma        | Stenoma luctifica           | Stenoma luctifica           | Zeller, 1877                 |    1.83673e+06 |      nan           |
|            nan | Mimosema consociata                         | Mimosema consociata (Schaus, 1911)               |    1.96022e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Mimosema       | Mimosema consociata         | Mimosema consociata         | (Schaus, 1911)               |    1.96022e+06 |      nan           |
|            nan | Scopula compensata                          | Scopula compensata (Walker, 1861)                |    5.14547e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Scopula        | Scopula compensata          | Scopula compensata          | (Walker, 1861)               |    5.14547e+06 |      nan           |
|            nan | moth Eucereon                               | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Oospila albicoma                            | Oospila albicoma Felder & Rogenhofer, 1875       |    1.97369e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Oospila        | Oospila albicoma            | Oospila albicoma            | Felder & Rogenhofer, 1875    |    1.97369e+06 |      nan           |
|            nan | Glenopteris oculifera                       | Glenopteris oculifera Hübner, 1821               |    5.11025e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Glenopteris    | Glenopteris oculifera       | Glenopteris oculifera       | Hübner, 1821                 |    5.11025e+06 |      nan           |
|            nan | todo                                        | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Gerra sevorsa                               | Gerra sevorsa Grote, 1882                        |    1.76114e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Gerra          | Gerra sevorsa               | Gerra sevorsa               | Grote, 1882                  |    1.76114e+06 |      nan           |
|            nan | Xylophanes ceratomioides                    | Xylophanes ceratomioides Grote & Robinson, 1866  |    1.86209e+06 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sphingidae     | Xylophanes     | Xylophanes ceratomioides    | Xylophanes ceratomioides    | Grote & Robinson, 1866       |    1.86209e+06 |      nan           |
|            nan | Cosmosoma impar                             | Cosmosoma impar Walker, 1854                     |    5.1162e+06  | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Cosmosoma      | Cosmosoma impar             | Cosmosoma impar             | Walker, 1854                 |    5.1162e+06  |      nan           |
|            nan | Perasia helvina                             | Perasia helvina Guenée, 1852                     |    1.78745e+06 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Perasia        | Perasia helvina             | Perasia helvina             | Guenée, 1852                 |    1.78745e+06 |      nan           |
|            nan | Gonodonta sicheas                           | Gonodonta sicheas Cramer                         |    1.77007e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Gonodonta      | Gonodonta sicheas           | Gonodonta sicheas           | Cramer                       |    1.77007e+06 |      nan           |
|            nan | Eois sp_san_luis_k                          | Eois Hübner, 1818                                |    1.98961e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eois           | nan                         | Eois                        | Hübner, 1818                 |    1.98961e+06 |      nan           |
|            nan | moth Spingidae                              | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Quentalia surynorta                         | Quentalia surynorta Schaus, 1900                 |    1.86822e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Bombycidae     | Quentalia      | Quentalia surynorta         | Quentalia surynorta         | Schaus, 1900                 |    1.86822e+06 |      nan           |
|            nan | Euglyphis libnites                          | Euglyphis libnites (Druce, 1897)                 |    5.1022e+06  | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Lasiocampidae  | Euglyphis      | Euglyphis libnites          | Euglyphis libnites          | (Druce, 1897)                |    5.1022e+06  |      nan           |
|            nan | Tricentrogyna vinacea                       | Tricentrogyna vinacea (Butler, 1878)             |    1.96324e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Tricentrogyna  | Tricentrogyna vinacea       | Tricentrogyna vinacea       | (Butler, 1878)               |    1.96324e+06 |      nan           |
|            nan | Nyceryx ericea                              | Nyceryx ericea Druce, 1888                       |    1.86314e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sphingidae     | Nyceryx        | Nyceryx ericea              | Nyceryx ericea              | Druce, 1888                  |    1.86314e+06 |      nan           |
|            nan | untitled                                    | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Xylophanes anubus                           | Xylophanes anubus Cramer, 1777                   |    1.86208e+06 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sphingidae     | Xylophanes     | Xylophanes anubus           | Xylophanes anubus           | Cramer, 1777                 |    1.86208e+06 |      nan           |
|            nan | Meganola sp_san_luis_a                      | Meganola Dyar, 1898                              |    4.40549e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Nolidae        | Meganola       | nan                         | Meganola                    | Dyar, 1898                   |    4.40549e+06 |      nan           |
|            nan | Spodoptera androgea                         | Spodoptera androgea Stoll                        |    5.10992e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Spodoptera     | Spodoptera androgea         | Spodoptera androgea         | Stoll                        |    5.10992e+06 |      nan           |
|            nan | Eois xanthoperata                           | Eois xanthoperata (Warren, 1897)                 |    1.98983e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eois           | Eois xanthoperata           | Eois xanthoperata           | (Warren, 1897)               |    1.98983e+06 |      nan           |
|            nan | moth Eois sp san luis b                     | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Xylophanes turbata                          | Xylophanes turbata Edwards, 1887                 |    1.86222e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sphingidae     | Xylophanes     | Xylophanes turbata          | Xylophanes turbata          | Edwards, 1887                |    1.86222e+06 |      nan           |
|            nan | Phrygionis platinata                        | Phrygionis platinata (Guenée)                    |    1.96832e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Phrygionis     | Phrygionis platinata        | Phrygionis platinata        | (Guenée)                     |    1.96832e+06 |      nan           |
|            nan | moth Notodontida                            | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | spider webspinner                           | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | ant                                         | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Bromeliaceae                                | Bromeliaceae                                     | 3740           | EXACT       |           96 | ACCEPTED | FAMILY  | Plantae   | Tracheophyta  | Liliopsida     | Poales          | Bromeliaceae   | nan            | nan                         | Bromeliaceae                | nan                          | 3740           |      nan           |
|            nan | banana                                      | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | butterfly                                   | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | praying mantis                              | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Epimecis subroraria                         | Epimecis subroraria (Walker, 1860)               |    5.14926e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Epimecis       | Epimecis subroraria         | Epimecis subroraria         | (Walker, 1860)               |    5.14926e+06 |      nan           |
|            nan | fly                                         | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Anticarsia gemmatalis                       | Anticarsia gemmatalis Hübner, 1818               |    1.77794e+06 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Anticarsia     | Anticarsia gemmatalis       | Anticarsia gemmatalis       | Hübner, 1818                 |    1.77794e+06 |      nan           |
|            nan | Gonodonta sinaldus                          | Gonodonta sinaldus Guenée, 1852                  |    1.77004e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Gonodonta      | Gonodonta sinaldus          | Gonodonta sinaldus          | Guenée, 1852                 |    1.77004e+06 |      nan           |
|            nan | Gn sp_san_luis_0001                         | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | beetle                                      | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Cosmosoma cingulatum                        | Cosmosoma cingulatum Butler, 1876                |    5.1163e+06  | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Cosmosoma      | Cosmosoma teuthras          | Cosmosoma cingulatum        | Butler, 1876                 |    5.1163e+06  |        5.11616e+06 |
|            nan | Ammalo paranomon                            | Ammalo paranomon Dyar, 1912                      |    1.81418e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Ammalo         | Ammalo paranomon            | Ammalo paranomon            | Dyar, 1912                   |    1.81418e+06 |      nan           |
|            nan | Phrygionis privignaria                      | Phrygionis privignaria (Guenée)                  |    1.96832e+06 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Phrygionis     | Phrygionis privignaria      | Phrygionis privignaria      | (Guenée)                     |    1.96832e+06 |      nan           |
|            nan | eggs                                        | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Melinodes detersaria                        | Melinodes detersaria Herrich-Schäffer            |    9.97954e+06 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Melinoides     | Melinoides detersaria       | Melinodes detersaria        | Herrich-Schäffer             |    9.97954e+06 |        1.98548e+06 |
|            nan | lacewing                                    | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Gn sp_san_luis_1006                         | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Hyperthaema orbicularis                     | Hyperthaema orbicularis Maassen, 1890            |    1.81784e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hyperthaema    | Hyperthaema orbicularis     | Hyperthaema orbicularis     | Maassen, 1890                |    1.81784e+06 |      nan           |
|            nan | Condica mimica                              | Condica mimica Hampson, 1908                     |    7.61706e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Condica        | Condica mimica              | Condica mimica              | Hampson, 1908                |    7.61706e+06 |      nan           |
|            nan | Acrosemia vulpecularia                      | Acrosemia vulpecularia Herrich-Schäffer, 1855    |    1.96009e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Acrosemia      | Acrosemia vulpecularia      | Acrosemia vulpecularia      | Herrich-Schäffer, 1855       |    1.96009e+06 |      nan           |
|            nan | Eulepidotis guttata--ouocco                 | Eulepidotis Hübner, 1823                         |    1.76315e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eulepidotis    | nan                         | Eulepidotis                 | Hübner, 1823                 |    1.76315e+06 |      nan           |
|            nan | Hemiceras rufescens                         | Hemiceras rufescens Walker, 1965                 |    5.11876e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Notodontidae   | Hemiceras      | Hemiceras rufescens         | Hemiceras rufescens         | Walker, 1965                 |    5.11876e+06 |      nan           |
|            nan | moth Satuniidae                             | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Mesocia dumilla                             | Mesoscia dumilla Dyar, 1913                      |    1.75174e+06 | FUZZY       |           85 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Megalopygidae  | Mesoscia       | Mesoscia dumilla            | Mesoscia dumilla            | Dyar, 1913                   |    1.75174e+06 |      nan           |
|            nan | Coronidia orithea                           | Coronidia orithea Cramer, 1780                   |    1.95415e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sematuridae    | Coronidia      | Coronidia orithea           | Coronidia orithea           | Cramer, 1780                 |    1.95415e+06 |      nan           |
|            nan | moth Sphingidaemo                           | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Episimus transferranus                      | Episimus transferrana (Walker, 1863)             |    1.7414e+06  | FUZZY       |           96 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Tortricidae    | Episimus       | Episimus transferrana       | Episimus transferrana       | (Walker, 1863)               |    1.7414e+06  |      nan           |
|            nan | Orsa erythrospila                           | Orsa erythrospila Walker, 1865                   |    5.11395e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Orsa           | Orsa erythrospila           | Orsa erythrospila           | Walker, 1865                 |    5.11395e+06 |      nan           |
|            nan | Rothschildia orizaba                        | Rothschildia orizaba Westwood, 1854              |    5.80624e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Saturniidae    | Rothschildia   | Rothschildia orizaba        | Rothschildia orizaba        | Westwood, 1854               |    5.80624e+06 |      nan           |
|            nan | Mursa imitatrix                             | Mursa Walker, 1859                               |    1.78981e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Mursa          | nan                         | Mursa                       | Walker, 1859                 |    1.78981e+06 |      nan           |
|            nan | caterpillar                                 | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Calymniodes acamas--conchylis--pyrostorta   | Calymniodes Dognin, 1907                         |    1.79276e+06 | HIGHERRANK  |           92 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Calymniodes    | nan                         | Calymniodes                 | Dognin, 1907                 |    1.79276e+06 |      nan           |
|            nan | moth Geo                                    | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | archaeognatha                               | Archaeognatha                                    | 1187           | EXACT       |           95 | ACCEPTED | ORDER   | Animalia  | Arthropoda    | Insecta        | Archaeognatha   | nan            | nan            | nan                         | Archaeognatha               | nan                          | 1187           |      nan           |
|            nan | Glyphodes sibillalis                        | Glyphodes sibillalis Walker, 1859                |    5.12763e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Glyphodes      | Glyphodes sibillalis        | Glyphodes sibillalis        | Walker, 1859                 |    5.12763e+06 |      nan           |
|            nan | megaloptera                                 | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | katydid                                     | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Anomis rubida                               | Anomis rubida Schaus, 1911                       |    1.78493e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Anomis         | Anomis rubida               | Anomis rubida               | Schaus, 1911                 |    1.78493e+06 |      nan           |
|            nan | Bleptina caradrinalis                       | Bleptina caradrinalis Guenée, 1854               |    1.79354e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Bleptina       | Bleptina caradrinalis       | Bleptina caradrinalis       | Guenée, 1854                 |    1.79354e+06 |      nan           |
|            nan | Cosmosoma festivum                          | Cosmosoma festivum Walker, 1854                  |    5.11627e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Cosmosoma      | Cosmosoma festivum          | Cosmosoma festivum          | Walker, 1854                 |    5.11627e+06 |      nan           |
|            nan | Diaphania nitidalis                         | Diaphania nitidalis (Stoll, 1781)                |    7.59837e+06 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Cryptographis  | Cryptographis nitidalis     | Diaphania nitidalis         | (Stoll, 1781)                |    7.59837e+06 |        1.88612e+06 |
|            nan | Xylophanes libya                            | Xylophanes libya Druce, 1878                     |    1.86219e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sphingidae     | Xylophanes     | Xylophanes libya            | Xylophanes libya            | Druce, 1878                  |    1.86219e+06 |      nan           |
|            nan | Morphomima fulvivacta                       | Morphomima fulvivacta                            |    1.19107e+07 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Uraniidae      | Morphomima     | Morphomima fulvivacta       | Morphomima fulvivacta       | nan                          |    1.19107e+07 |      nan           |
|            nan | Aellopos titan                              | Aellopos titan Cramer, 1777                      |    5.12445e+06 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sphingidae     | Aellopos       | Aellopos titan              | Aellopos titan              | Cramer, 1777                 |    5.12445e+06 |      nan           |
|            nan | Light                                       | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Cliniodes nomadalis                         | Cliniodes nomadalis Dyar, 1912                   |    1.89129e+06 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Cliniodes      | Cliniodes cyllarusalis      | Cliniodes nomadalis         | Dyar, 1912                   |    1.89129e+06 |        1.89128e+06 |
|            nan | pupa chrysalis                              | Pupa chrysalis Férussac, 1832                    |    1.13318e+07 | EXACT       |           97 | SYNONYM  | SPECIES | Animalia  | Mollusca      | Gastropoda     | Stylommatophora | Cerionidae     | Cerion         | Cerion mumia                | Pupa chrysalis              | Férussac, 1832               |    1.13318e+07 |        1.13773e+07 |
|            nan | Lophocampa annulosa                         | Lophocampa annulosa Walker, 1855                 |    1.81773e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Lophocampa     | Lophocampa annulosa         | Lophocampa annulosa         | Walker, 1855                 |    1.81773e+06 |      nan           |
|            nan | Isanthrene felderi                          | Isanthrene felderi Druce, 1883                   |    1.8112e+06  | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Isanthrene     | Isanthrene felderi          | Isanthrene felderi          | Druce, 1883                  |    1.8112e+06  |      nan           |
|            nan | Cimicodes albicosta                         | Cimicodes albicosta Dognin, 1914                 |    1.95713e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Cimicodes      | Cimicodes albicosta         | Cimicodes albicosta         | Dognin, 1914                 |    1.95713e+06 |      nan           |
|            nan | Lampruna rosea                              | Lampruna rosea Schaus, 1894                      |    1.8166e+06  | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Lampruna       | Lampruna rosea              | Lampruna rosea              | Schaus, 1894                 |    1.8166e+06  |      nan           |
|            nan | Xylophanes cthulhu maybe                    | Xylophanes cthulhu Haxaire & Vaglia, 2008        |    7.73176e+06 | HIGHERRANK  |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sphingidae     | Xylophanes     | Xylophanes cthulhu          | Xylophanes cthulhu          | Haxaire & Vaglia, 2008       |    7.73176e+06 |      nan           |
|            nan | frog                                        | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Enyo lugubris                               | Enyo lugubris Linnaeus, 1771                     |    5.12425e+06 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sphingidae     | Enyo           | Enyo lugubris               | Enyo lugubris               | Linnaeus, 1771               |    5.12425e+06 |      nan           |
|            nan | bird                                        | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | crab                                        | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | earwig                                      | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | stick insect                                | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | fungus                                      | Fungus Adanson ex O.Kuntze, 1898                 |    2.52246e+06 | EXACT       |           93 | SYNONYM  | GENUS   | Fungi     | Basidiomycota | Agaricomycetes | Agaricales      | Agaricaceae    | Agaricus       | nan                         | Fungus                      | Adanson ex O.Kuntze, 1898    |    2.52246e+06 |        2.51865e+06 |
|            nan | Gn sp_san_luis_2014                         | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | snail                                       | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Nyceryx stuarti                             | Nyceryx stuarti Rothschild, 1894                 |    1.86315e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sphingidae     | Nyceryx        | Nyceryx stuarti             | Nyceryx stuarti             | Rothschild, 1894             |    1.86315e+06 |      nan           |
|            nan | moth Crambidae delta                        | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | ground                                      | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Sosxetra grata                              | Sosxetra grata Walker, 1862                      |    5.11039e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Sosxetra       | Sosxetra grata              | Sosxetra grata              | Walker, 1862                 |    5.11039e+06 |      nan           |
|            nan | Eacles imperialis                           | Eacles imperialis Drury, 1773                    |    1.86511e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Saturniidae    | Eacles         | Eacles imperialis           | Eacles imperialis           | Drury, 1773                  |    1.86511e+06 |      nan           |
|            nan | chrysalis pupa                              | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | moth Spodoptera                             | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Herminocala sabata                          | Herminocala Hampson, 1913                        |    9.4016e+06  | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Herminocala    | nan                         | Herminocala                 | Hampson, 1913                |    9.4016e+06  |      nan           |
|            nan | mantislfy mantidfly                         | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Rhodochlora rothschildi                     | Rhodochlora rothschildi Warren, 1901             |    1.98563e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Rhodochlora    | Rhodochlora rothschildi     | Rhodochlora rothschildi     | Warren, 1901                 |    1.98563e+06 |      nan           |
|            nan | Prolimacodes badia                          | Prolimacodes badia (Hübner, 1835)                |    1.75282e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Limacodidae    | Prolimacodes   | Prolimacodes badia          | Prolimacodes badia          | (Hübner, 1835)               |    1.75282e+06 |      nan           |
|            nan | cicada                                      | Cicada Linnaeus, 1758                            |    1.68254e+06 | EXACT       |           93 | DOUBTFUL | GENUS   | Animalia  | Arthropoda    | Insecta        | Hemiptera       | Cicadidae      | Cicada         | nan                         | Cicada                      | Linnaeus, 1758               |    1.68254e+06 |      nan           |
|            nan | Hemiceras singuloides                       | Hemiceras singuloides Dyar, 1908                 |    5.11865e+06 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Notodontidae   | Hemiceras      | Hemiceras violascens        | Hemiceras singuloides       | Dyar, 1908                   |    5.11865e+06 |        5.11866e+06 |
|            nan | Macrosoma cascaria                          | Macrosoma cascaria Schaus, 1901                  |    5.15004e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Hedylidae      | Macrosoma      | Macrosoma cascaria          | Macrosoma cascaria          | Schaus, 1901                 |    5.15004e+06 |      nan           |
|            nan | Eulepte concordalis                         | Eulepte concordalis Hübner, 1826                 |    1.88518e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Eulepte        | Eulepte concordalis         | Eulepte concordalis         | Hübner, 1826                 |    1.88518e+06 |      nan           |
|            nan | Lineodes sp_san_luis_a                      | Lineodes Guenée, 1854                            |    1.8924e+06  | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Lineodes       | nan                         | Lineodes                    | Guenée, 1854                 |    1.8924e+06  |      nan           |
|            nan | Synclera jarbusalis                         | Synclera jarbusalis Walker, 1859                 |    1.88478e+06 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Synclera       | Synclera traducalis         | Synclera jarbusalis         | Walker, 1859                 |    1.88478e+06 |        1.88477e+06 |
|            nan | stink bug                                   | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Argyrosticta aurifundens                    | Argyrosticta aurifundens Walker, 1858            |    1.76898e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Argyrosticta   | Argyrosticta aurifundens    | Argyrosticta aurifundens    | Walker, 1858                 |    1.76898e+06 |      nan           |
|            nan | Ischnurges eudamidasalis                    | Ischnurges Lederer, 1863                         |    1.88758e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Ischnurges     | nan                         | Ischnurges                  | Lederer, 1863                |    1.88758e+06 |      nan           |
|            nan | Tortyra slossonia                           | Tortyra slossonia Fernald, 1900                  |    1.86044e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Choreutidae    | Tortyra        | Tortyra slossonia           | Tortyra slossonia           | Fernald, 1900                |    1.86044e+06 |      nan           |
|            nan | Thoscora acca                               | Thoscora acca (Schaus, 1892)                     |    1.75152e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Megalopygidae  | Thoscora       | Thoscora acca               | Thoscora acca               | (Schaus, 1892)               |    1.75152e+06 |      nan           |
|            nan | Epitausa flagrans                           | Epitausa flagrans Walker, 1869                   |    1.7722e+06  | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Epitausa       | Epitausa flagrans           | Epitausa flagrans           | Walker, 1869                 |    1.7722e+06  |      nan           |
|            nan | Syngamia florella                           | Syngamia florella Cramer, 1781                   |    1.88595e+06 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Syngamia       | Syngamia florella           | Syngamia florella           | Cramer, 1781                 |    1.88595e+06 |      nan           |
|            nan | Syngamilyta apicolor                        | Syngamilyta apicolor                             |    1.00608e+07 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Syngamilyta    | Syngamilyta apicolor        | Syngamilyta apicolor        | nan                          |    1.00608e+07 |      nan           |
|            nan | Sarsina purpurascens                        | Sarsina purpurascens Walker, 1855                |    1.81931e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Sarsina        | Sarsina purpurascens        | Sarsina purpurascens        | Walker, 1855                 |    1.81931e+06 |      nan           |
|            nan | Pseudogerespa usipetes                      | Pseudogerespa usipetes Druce, 1898               |    1.79424e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Pseudogerespa  | Pseudogerespa usipetes      | Pseudogerespa usipetes      | Druce, 1898                  |    1.79424e+06 |      nan           |
|            nan | Euglyphis gutturalis                        | Euglyphis gutturalis                             |    1.00197e+07 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Lasiocampidae  | Euglyphis      | Euglyphis gutturalis        | Euglyphis gutturalis        | nan                          |    1.00197e+07 |      nan           |
|            nan | Hemiceras metastigma                        | Hemiceras metastigma Walker, 1857                |    5.11861e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Notodontidae   | Hemiceras      | Hemiceras metastigma        | Hemiceras metastigma        | Walker, 1857                 |    5.11861e+06 |      nan           |
|            nan | Eucereon xanthura                           | Eucereon Hübner, 1819                            |    1.81014e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | nan                         | Eucereon                    | Hübner, 1819                 |    1.81014e+06 |      nan           |
|            nan | Euglyphis larunda--obliterata               | Euglyphis Hübner, 1820                           |    7.89919e+06 | HIGHERRANK  |           92 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Lasiocampidae  | Euglyphis      | nan                         | Euglyphis                   | Hübner, 1820                 |    7.89919e+06 |      nan           |
|            nan | Empelathra amplificans                      | Empelathra amplificans Walker, 1858              |    1.76868e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Empelathra     | Empelathra amplificans      | Empelathra amplificans      | Walker, 1858                 |    1.76868e+06 |      nan           |
|            nan | Eulepidotis juncida                         | Eulepidotis juncida Guenée, 1852                 |    1.76316e+06 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eulepidotis    | Eulepidotis juncida         | Eulepidotis juncida         | Guenée, 1852                 |    1.76316e+06 |      nan           |
|            nan | Sarsina festiva                             | Sarsina festiva Schaus, 1912                     |    1.8193e+06  | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Sarsina        | Sarsina festiva             | Sarsina festiva             | Schaus, 1912                 |    1.8193e+06  |      nan           |
|            nan | Rothschildia erycina                        | Rothschildia erycina Shaw                        |    5.1246e+06  | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Saturniidae    | Rothschildia   | Rothschildia erycina        | Rothschildia erycina        | Shaw                         |    5.1246e+06  |      nan           |
|            nan | Cyanopepla scintillans                      | Cyanopepla scintillans Butler, 1872              |    1.80593e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Cyanopepla     | Cyanopepla scintillans      | Cyanopepla scintillans      | Butler, 1872                 |    1.80593e+06 |      nan           |
|            nan | Eucereon flavicaput                         | Eucereon Hübner, 1819                            |    1.81014e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | nan                         | Eucereon                    | Hübner, 1819                 |    1.81014e+06 |      nan           |
|            nan | Antiblemma lothos                           | Antiblemma lothos Cramer, 1777                   |    1.79474e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Antiblemma     | Antiblemma lothos           | Antiblemma lothos           | Cramer, 1777                 |    1.79474e+06 |      nan           |
|            nan | Bagisara tristicta                          | Bagisara tristicta Hampson, 1898                 |    9.51134e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Bagisara       | Bagisara tristicta          | Bagisara tristicta          | Hampson, 1898                |    9.51134e+06 |      nan           |
|            nan | Cerconota impresella                        | Cerconota impressella Walker, 1864               |    1.83607e+06 | FUZZY       |           96 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Depressariidae | Cerconota      | Cerconota impressella       | Cerconota impressella       | Walker, 1864                 |    1.83607e+06 |      nan           |
|            nan | Bertholdia specularis                       | Bertholdia specularis Herrich-Schäffer, 1853     |    5.1155e+06  | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Bertholdia     | Bertholdia specularis       | Bertholdia specularis       | Herrich-Schäffer, 1853       |    5.1155e+06  |      nan           |
|            nan | Argyrotome perspecta                        | Argyrotome Warren, 1894                          |    1.96734e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Argyrotome     | nan                         | Argyrotome                  | Warren, 1894                 |    1.96734e+06 |      nan           |
|            nan | caddisfly                                   | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Parasa wellesca                             | Parasa wellesca Dyar, 1900                       |    5.10545e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Limacodidae    | Parasa         | Parasa wellesca             | Parasa wellesca             | Dyar, 1900                   |    5.10545e+06 |      nan           |
|            nan | bee                                         | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Macrotes commatica                          | Macrotes commatica Prout, 1916                   |    1.95998e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Macrotes       | Macrotes commatica          | Macrotes commatica          | Prout, 1916                  |    1.95998e+06 |      nan           |
|            nan | Xylophanes titana                           | Xylophanes titana Druce, 1878                    |    1.86209e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sphingidae     | Xylophanes     | Xylophanes titana           | Xylophanes titana           | Druce, 1878                  |    1.86209e+06 |      nan           |
|            nan | Amastus episcotosia                         | Amastus episcotosia Dognin, 1901                 |    5.11726e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Amastus        | Amastus episcotosia         | Amastus episcotosia         | Dognin, 1901                 |    5.11726e+06 |      nan           |
|            nan | = I_MFS4922                                 | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | hymenoptera sawfly maybe                    | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Nebulosa erymas                             | Nebulosa erymas                                  |    1.04225e+07 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Notodontidae   | Nebulosa       | Nebulosa erymas             | Nebulosa erymas             | nan                          |    1.04225e+07 |      nan           |
|            nan | Pupa chrysalis                              | Pupa chrysalis Férussac, 1832                    |    1.13318e+07 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Mollusca      | Gastropoda     | Stylommatophora | Cerionidae     | Cerion         | Cerion mumia                | Pupa chrysalis              | Férussac, 1832               |    1.13318e+07 |        1.13773e+07 |
|            nan | = I_MFS5180                                 | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | fig wasp                                    | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | worm planarian                              | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | = I_MFS5233                                 | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | bee fly maybe                               | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | orthoptera cricket                          | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | beetle maybe                                | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | slug                                        | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | caddisfly trichoptera                       | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | = I_MFS5928                                 | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | caddisfly trichoptera maybe                 | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | truebug moth being eaten                    | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | = I_MFS6471                                 | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Udea rubigalis                              | Udea rubigalis Guenée, 1854                      |    1.88712e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Udea           | Udea rubigalis              | Udea rubigalis              | Guenée, 1854                 |    1.88712e+06 |      nan           |
|            nan | stick insect phasmatodea                    | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | beetle Cerambycidae                         | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | neuroptera Mantispidae mantidfly mantisfly  | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | butterfly Satyridae                         | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | moth Gonodonta                              | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Bradina aaronalis                           | Bradina aaronalis Schaus, 1924                   |    1.88583e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Bradina        | Bradina aaronalis           | Bradina aaronalis           | Schaus, 1924                 |    1.88583e+06 |      nan           |
|            nan | beetle Elateridae                           | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | stick insect Phasmatodea                    | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Orthoptera                                  | Orthoptera                                       | 1458           | EXACT       |           94 | ACCEPTED | ORDER   | Animalia  | Arthropoda    | Insecta        | Orthoptera      | nan            | nan            | nan                         | Orthoptera                  | nan                          | 1458           |      nan           |
|            nan | eggs lacewing maybe                         | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | planarian maybe                             | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | beetle mating                               | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | flower                                      | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | chrysalis cacoon                            | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | moth maybe                                  | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | fly mating                                  | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | = I_MFS7057                                 | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Antaeotricha stigmatias                     | Antaeotricha Zeller, 1854                        |    1.83834e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Depressariidae | Antaeotricha   | nan                         | Antaeotricha                | Zeller, 1854                 |    1.83834e+06 |      nan           |
|            nan | pupa cocoon                                 | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | orthoperta grasshopper                      | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Xylophanes cthulhu                          | Xylophanes cthulhu Haxaire & Vaglia, 2008        |    7.73176e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sphingidae     | Xylophanes     | Xylophanes cthulhu          | Xylophanes cthulhu          | Haxaire & Vaglia, 2008       |    7.73176e+06 |      nan           |
|            nan | beetle weevil                               | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Diaphania praxialis                         | Diaphania Hübner, 1818                           |    3.25813e+06 | HIGHERRANK  |           92 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Diaphania      | nan                         | Diaphania                   | Hübner, 1818                 |    3.25813e+06 |      nan           |
|            nan | moth temp                                   | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | cockroach                                   | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | earwig dermaptera                           | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Carathis byblis--septentrionalis            | Carathis Grote, 1865                             |    1.806e+06   | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Carathis       | nan                         | Carathis                    | Grote, 1865                  |    1.806e+06   |      nan           |
|            nan | butterfly skipper                           | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Parasa viridogrisea                         | Parasa viridogrisea (Dyar, 1898)                 |    5.10552e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Limacodidae    | Parasa         | Parasa viridogrisea         | Parasa viridogrisea         | (Dyar, 1898)                 |    5.10552e+06 |      nan           |
|            nan | Xylophanes porcus                           | Xylophanes porcus Hübner                         |    1.86209e+06 | EXACT       |           98 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sphingidae     | Xylophanes     | Xylophanes porcus           | Xylophanes porcus           | Hübner                       |    1.86209e+06 |      nan           |
|            nan | Rhodochlora brunneipalpis                   | Rhodochlora brunneipalpis Warren, 1894           |    1.98565e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Rhodochlora    | Rhodochlora brunneipalpis   | Rhodochlora brunneipalpis   | Warren, 1894                 |    1.98565e+06 |      nan           |
|            nan | Idalus borealis                             | Idalus borealis                                  |    1.8147e+06  | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Idalus         | Idalus borealis             | Idalus borealis             | nan                          |    1.8147e+06  |      nan           |
|            nan | Gonodonta pulverea                          | Gonodonta pulverea Schaus, 1911                  |    1.77006e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Gonodonta      | Gonodonta pulverea          | Gonodonta pulverea          | Schaus, 1911                 |    1.77006e+06 |      nan           |
|            nan | Patalene falcularia                         | Patalene falcularia (Sepp, 1854)                 |    1.97866e+06 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Patalene       | Patalene falcularia         | Patalene falcularia         | (Sepp, 1854)                 |    1.97866e+06 |      nan           |
|            nan | Fulgoridae                                  | Fulgoridae                                       | 8585           | EXACT       |           94 | ACCEPTED | FAMILY  | Animalia  | Arthropoda    | Insecta        | Hemiptera       | Fulgoridae     | nan            | nan                         | Fulgoridae                  | nan                          | 8585           |      nan           |
|            nan | todoHymenomima camerata                     | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Eois sp_san_luis_c                          | Eois Hübner, 1818                                |    1.98961e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eois           | nan                         | Eois                        | Hübner, 1818                 |    1.98961e+06 |      nan           |
|            nan | Eois brunneicosta                           | Eois brunneicosta (Dognin, 1916)                 |    1.98985e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eois           | Eois brunneicosta           | Eois brunneicosta           | (Dognin, 1916)               |    1.98985e+06 |      nan           |
|            nan | Eois sp_san_luis_j                          | Eois Hübner, 1818                                |    1.98961e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eois           | nan                         | Eois                        | Hübner, 1818                 |    1.98961e+06 |      nan           |
|            nan | Letis scops                                 | Letis scops Guenée, 1852                         |    1.78838e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Letis          | Letis scops                 | Letis scops                 | Guenée, 1852                 |    1.78838e+06 |      nan           |
|            nan | Perigonia ilus                              | Perigonia ilus Boisduval, 1870                   |    1.86157e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sphingidae     | Perigonia      | Perigonia ilus              | Perigonia ilus              | Boisduval, 1870              |    1.86157e+06 |      nan           |
|            nan | Automeris belti                             | Automeris belti Druce, 1886                      |    1.866e+06   | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Saturniidae    | Automeris      | Automeris belti             | Automeris belti             | Druce, 1886                  |    1.866e+06   |      nan           |
|            nan | Centipede                                   | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | = I_MFS11690                                | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Acharia hyperoche                           | Acharia hyperoche                                |    9.94107e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Limacodidae    | Acharia        | Acharia hyperoche           | Acharia hyperoche           | nan                          |    9.94107e+06 |      nan           |
|            nan | Notodontidae                                | Notodontidae                                     | 7016           | EXACT       |           94 | ACCEPTED | FAMILY  | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Notodontidae   | nan            | nan                         | Notodontidae                | nan                          | 7016           |      nan           |
|            nan | Nemoria astraeoides                         | Nemoria Hübner, 1818                             |    1.98114e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nemoria        | nan                         | Nemoria                     | Hübner, 1818                 |    1.98114e+06 |      nan           |
|            nan | Dalcerides mesoa                            | Dalcerides mesoa                                 |    1.05205e+07 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Dalceridae     | Dalcerides     | Dalcerides mesoa            | Dalcerides mesoa            | nan                          |    1.05205e+07 |      nan           |
|            nan | = I_PBA27442                                | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | moth Hermiceras                             | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Bryolymnia bicon                            | Bryolymnia bicon Druce, 1889                     |    1.76414e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Bryolymnia     | Bryolymnia bicon            | Bryolymnia bicon            | Druce, 1889                  |    1.76414e+06 |      nan           |
|            nan | Herminocala sp_san_luis_a                   | Herminocala Hampson, 1913                        |    9.4016e+06  | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Herminocala    | nan                         | Herminocala                 | Hampson, 1913                |    9.4016e+06  |      nan           |
|            nan | Gn sp_san_luis_0011                         | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | moth Letis                                  | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | moth Pyralidaev                             | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | moth Crambidia                              | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | moth Macaria                                | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | moth Tosale                                 | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Pseudodirphia menander                      | Pseudodirphia menander Druce, 1886               |    1.02485e+07 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Saturniidae    | Pseudodirphia  | Pseudodirphia menander      | Pseudodirphia menander      | Druce, 1886                  |    1.02485e+07 |      nan           |
|            nan | Dalcerides gugelmanni                       | Dalcerides gugelmanni                            |    1.05014e+07 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Dalceridae     | Dalcerides     | Dalcerides gugelmanni       | Dalcerides gugelmanni       | nan                          |    1.05014e+07 |      nan           |
|            nan | Eois sp_san_luis_i                          | Eois Hübner, 1818                                |    1.98961e+06 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eois           | nan                         | Eois                        | Hübner, 1818                 |    1.98961e+06 |      nan           |
|            nan | 394                                         | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Ctenucha sp_san_luis_a                      | Ctenucha Kirby, 1837                             |    1.8148e+06  | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Ctenucha       | nan                         | Ctenucha                    | Kirby, 1837                  |    1.8148e+06  |      nan           |
|            nan | Macrosoma rubedinaria                       | Macrosoma rubedinaria Walker, 1862               |    5.15003e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Hedylidae      | Macrosoma      | Macrosoma rubedinaria       | Macrosoma rubedinaria       | Walker, 1862                 |    5.15003e+06 |      nan           |
|            nan | moth Goemetridae                            | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Crambidae                                   | Crambidae                                        | 8841           | EXACT       |           94 | ACCEPTED | FAMILY  | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | nan            | nan                         | Crambidae                   | nan                          | 8841           |      nan           |
|            nan | moth Pyraldiae                              | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Tolype caieta                               | Tolype caieta Druce, 1897                        |    5.10238e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Lasiocampidae  | Tolype         | Tolype caieta               | Tolype caieta               | Druce, 1897                  |    5.10238e+06 |      nan           |
|            nan | Gn sp_san_luis_1008                         | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Saurita fumosa                              | Saurita fumosa Schaus, 1912                      |    1.8095e+06  | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Saurita        | Saurita fumosa              | Saurita fumosa              | Schaus, 1912                 |    1.8095e+06  |      nan           |
|            nan | moth Geomtridae                             | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Josiomorpha penetrata                       | Josiomorpha penetrata Walker, 1865               |    1.81842e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Josiomorpha    | Josiomorpha penetrata       | Josiomorpha penetrata       | Walker, 1865                 |    1.81842e+06 |      nan           |
|            nan | Eulepidotis hemura                          | Eulepidotis hemura                               |    1.02566e+07 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eulepidotis    | Eulepidotis hemura          | Eulepidotis hemura          | nan                          |    1.02566e+07 |      nan           |
|            nan | Glena mosparia                              | Glena mopsaria (Schaus, 1913)                    |    1.96568e+06 | FUZZY       |           95 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Glena          | Glena mopsaria              | Glena mopsaria              | (Schaus, 1913)               |    1.96568e+06 |      nan           |
|            nan | Seirocastnia tribuna                        | Seirocastnia tribuna Hübner, 1825                |    1.7904e+06  | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Seirocastnia   | Seirocastnia tribuna        | Seirocastnia tribuna        | Hübner, 1825                 |    1.7904e+06  |      nan           |
|            nan | Eusarca flexiilis                           | Eusarca flexilis (Schaus, 1912)                  |    1.17211e+07 | FUZZY       |           96 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eusarca        | Eusarca flexilis            | Eusarca flexilis            | (Schaus, 1912)               |    1.17211e+07 |      nan           |
|            nan | moth Notontidae                             | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Macaria carpo                               | Macaria carpo (Druce, 1893)                      |    9.33856e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Macaria        | Macaria carpo               | Macaria carpo               | (Druce, 1893)                |    9.33856e+06 |      nan           |
|            nan | Eusarca crameraria--brown                   | Eusarca Hübner                                   |    7.51469e+06 | HIGHERRANK  |           92 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eusarca        | nan                         | Eusarca                     | Hübner                       |    7.51469e+06 |      nan           |
|            nan | Elysius discoplaga                          | Elysius discoplaga Walker, 1856                  |    1.1667e+07  | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Elysius        | Elysius discoplaga          | Elysius discoplaga          | Walker, 1856                 |    1.1667e+07  |      nan           |
|            nan | Morpheis strigifer                          | Morpheis Hübner, 1820                            |    1.73068e+06 | HIGHERRANK  |           92 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Cossidae       | Morpheis       | nan                         | Morpheis                    | Hübner, 1820                 |    1.73068e+06 |      nan           |
|            nan | Ptychoglene xylophila                       | Ptychoglene xylophila Druce, 1885                |    1.80313e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Ptychoglene    | Ptychoglene xylophila       | Ptychoglene xylophila       | Druce, 1885                  |    1.80313e+06 |      nan           |
|            nan | = I_PBA34184                                | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Pachydota iodea                             | Pachydota iodea Herrich-Schäffer, 1855           |    1.806e+06   | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Pachydota      | Pachydota iodea             | Pachydota iodea             | Herrich-Schäffer, 1855       |    1.806e+06   |      nan           |
|            nan | moth Lymantriidae                           | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Mapeta xanthomelas                          | Mapeta xanthomelas Walker, 1863                  |    1.87031e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Pyralidae      | Mapeta         | Mapeta xanthomelas          | Mapeta xanthomelas          | Walker, 1863                 |    1.87031e+06 |      nan           |
|            nan | Oxydia sociata                              | Oxydia sociata (Warren, 1895)                    |    1.97666e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Oxydia         | Oxydia sociata              | Oxydia sociata              | (Warren, 1895)               |    1.97666e+06 |      nan           |
|            nan | Glena moparia                               | Glena mopsaria (Schaus, 1913)                    |    1.96568e+06 | FUZZY       |           95 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Glena          | Glena mopsaria              | Glena mopsaria              | (Schaus, 1913)               |    1.96568e+06 |      nan           |
|            nan | Trygodes musivaria                          | Trygodes musivaria (Herrich-Schäffer, 1855)      |    1.97637e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Trygodes       | Trygodes musivaria          | Trygodes musivaria          | (Herrich-Schäffer, 1855)     |    1.97637e+06 |      nan           |
|            nan | moth Eois                                   | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | = I_PBA35358                                | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | = I_PBA35425                                | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Geometridae                                 | Geometridae                                      | 6950           | EXACT       |           94 | ACCEPTED | FAMILY  | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | nan            | nan                         | Geometridae                 | nan                          | 6950           |      nan           |
|            nan | = I_PBA35476                                | nan                                              |  nan           | NONE        |          100 | nan      | nan     | nan       | nan           | nan            | nan             | nan            | nan            | nan                         | nan                         | nan                          |  nan           |      nan           |
|            nan | Phassus n-signatus                          | Phassus n-signatus Weymer, 1907                  |    5.11932e+06 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Hepialidae     | Phassus        | Phassus n-signatus          | Phassus n-signatus          | Weymer, 1907                 |    5.11932e+06 |      nan           |

In [5]:
# Then, I removed any rows where matchType was not EXACT or HIGHERRANK.
# Then, I realized that the results of my unique_values_output.csv file had a lot of "moth foo" entries. 
# I needed to remove the "moth " prefix, from the entries, and run all of the above again.
# These manual actions resulted in the following:

In [6]:
import pandas as pd
from IPython.display import Markdown

df = pd.read_csv("/Users/markfisher/Desktop/cr_moth_classification/1_dataset_prep/ciee_monte_verde_dataset/unique_values_output_gbif_unmatched_excluded_moth_removed.csv")
markdown_table = df.to_markdown(index=False)
display(Markdown(markdown_table))

|   occurrenceId | verbatimScientificName                      | scientificName                                   |      key | matchType   |   confidence | status   | rank    | kingdom   | phylum        | class          | order           | family         | genus          | species                     | canonicalName               | authorship                   |   usageKey |   acceptedUsageKey |
|---------------:|:--------------------------------------------|:-------------------------------------------------|---------:|:------------|-------------:|:---------|:--------|:----------|:--------------|:---------------|:----------------|:---------------|:---------------|:----------------------------|:----------------------------|:-----------------------------|-----------:|-------------------:|
|            nan | Lobocleta tenellata                         | Lobocleta tenellata (Möschler, 1886)             |  1990261 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Lobocleta      | Lobocleta tenellata         | Lobocleta tenellata         | (Möschler, 1886)             |    1990261 |      nan           |
|            nan | Notodontidae                                | Notodontidae                                     |     7016 | EXACT       |           94 | ACCEPTED | FAMILY  | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Notodontidae   | nan            | nan                         | Notodontidae                | nan                          |       7016 |      nan           |
|            nan | Clemensia leopardina                        | Clemensia leopardina Schaus, 1911                |  5114902 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Clemensia      | Clemensia leopardina        | Clemensia leopardina        | Schaus, 1911                 |    5114902 |      nan           |
|            nan | Rhabdatomis laudamia                        | Rhabdatomis laudamia Druce, 1885                 |  1811258 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Rhabdatomis    | Rhabdatomis laudamia        | Rhabdatomis laudamia        | Druce, 1885                  |    1811258 |      nan           |
|            nan | Eupithecia sp_group_san_luis                | Eupithecia Curtis, 1825                          |  1982367 | HIGHERRANK  |           92 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eupithecia     | nan                         | Eupithecia                  | Curtis, 1825                 |    1982367 |      nan           |
|            nan | Crambidia myrlosea                          | Crambidia myrlosea Dyar, 1917                    |  1805542 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Crambidia      | Crambidia myrlosea          | Crambidia myrlosea          | Dyar, 1917                   |    1805542 |      nan           |
|            nan | Leptostales crossii                         | Leptostales crossii (Hulst, 1900)                |  9619088 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Leptostales    | Leptostales crossii         | Leptostales crossii         | (Hulst, 1900)                |    9619088 |      nan           |
|            nan | Meganola sp_san_luis_b                      | Meganola Dyar, 1898                              |  4405493 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Nolidae        | Meganola       | nan                         | Meganola                    | Dyar, 1898                   |    4405493 |      nan           |
|            nan | Crambidae                                   | Crambidae                                        |     8841 | EXACT       |           94 | ACCEPTED | FAMILY  | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | nan            | nan                         | Crambidae                   | nan                          |       8841 |      nan           |
|            nan | Nephodia auxesia                            | Nephodia auxesia Druce, 1892                     |  1981483 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nephodia       | Nephodia auxesia            | Nephodia auxesia            | Druce, 1892                  |    1981483 |      nan           |
|            nan | Hemiceras                                   | Hemiceras Guenée, 1852                           |  1823790 | EXACT       |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Notodontidae   | Hemiceras      | nan                         | Hemiceras                   | Guenée, 1852                 |    1823790 |      nan           |
|            nan | Nemoria defectiva                           | Nemoria defectiva (Prout, 1932)                  |  1981259 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nemoria        | Nemoria defectiva           | Nemoria defectiva           | (Prout, 1932)                |    1981259 |      nan           |
|            nan | Nematocampa arenosa                         | Nematocampa arenosa Butler, 1881                 |  1991014 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nematocampa    | Nematocampa arenosa         | Nematocampa arenosa         | Butler, 1881                 |    1991014 |      nan           |
|            nan | Lascoria                                    | Lascoria Walker, 1859                            |  1761829 | EXACT       |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Lascoria       | nan                         | Lascoria                    | Walker, 1859                 |    1761829 |      nan           |
|            nan | Tortricidae                                 | Tortricidae                                      |     5343 | EXACT       |           94 | ACCEPTED | FAMILY  | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Tortricidae    | nan            | nan                         | Tortricidae                 | nan                          |       5343 |      nan           |
|            nan | Patalene aenetusaria                        | Patalene aenetusaria (Walker, 1860)              |  9243264 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Patalene       | Patalene aenetusaria        | Patalene aenetusaria        | (Walker, 1860)               |    9243264 |      nan           |
|            nan | Clemensia cincinnata                        | Clemensia cincinnata Schaus, 1911                |  5114878 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Clemensia      | Clemensia cincinnata        | Clemensia cincinnata        | Schaus, 1911                 |    5114878 |      nan           |
|            nan | Brachurapteryx tesserata                    | Brachurapteryx tesserata (Guenée)                |  1955815 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Brachurapteryx | Brachurapteryx tesserata    | Brachurapteryx tesserata    | (Guenée)                     |    1955815 |      nan           |
|            nan | Crambidia cephalica                         | Crambidia cephalica (Grote & Robinson, 1870)     |  1805537 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Crambidia      | Crambidia cephalica         | Crambidia cephalica         | (Grote & Robinson, 1870)     |    1805537 |      nan           |
|            nan | Geometridae                                 | Geometridae                                      |     6950 | EXACT       |           94 | ACCEPTED | FAMILY  | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | nan            | nan                         | Geometridae                 | nan                          |       6950 |      nan           |
|            nan | Noctuidae                                   | Noctuidae                                        |     7015 | EXACT       |           94 | ACCEPTED | FAMILY  | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | nan            | nan                         | Noctuidae                   | nan                          |       7015 |      nan           |
|            nan | Glena mopsaria                              | Glena mopsaria (Schaus, 1913)                    |  1965683 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Glena          | Glena mopsaria              | Glena mopsaria              | (Schaus, 1913)               |    1965683 |      nan           |
|            nan | Iridopsis validaria                         | Iridopsis validaria Guenée, 1858                 |  5148593 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Iridopsis      | Iridopsis validaria         | Iridopsis validaria         | Guenée, 1858                 |    5148593 |      nan           |
|            nan | Micrathetis dasarada                        | Micrathetis dasarada Druce, 1898                 |  1764686 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Micrathetis    | Micrathetis dasarada        | Micrathetis dasarada        | Druce, 1898                  |    1764686 |      nan           |
|            nan | Macaria sp_san_luis_a                       | Arthropoda                                       |       54 | HIGHERRANK  |           92 | ACCEPTED | PHYLUM  | Animalia  | Arthropoda    | nan            | nan             | nan            | nan            | nan                         | Arthropoda                  | nan                          |         54 |      nan           |
|            nan | Idaea sp_group_san_luis                     | Idaea Treitschke, 1825                           |  1973909 | HIGHERRANK  |           92 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Idaea          | nan                         | Idaea                       | Treitschke, 1825             |    1973909 |      nan           |
|            nan | Lophosis labeculata                         | Lophosis labeculata (Hulst, 1887)                |  9237031 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Lophosis       | Lophosis labeculata         | Lophosis labeculata         | (Hulst, 1887)                |    9237031 |      nan           |
|            nan | Trygodes amphion                            | Trygodes amphion Schaus, 1912                    |  1976364 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Trygodes       | Trygodes amphion            | Trygodes amphion            | Schaus, 1912                 |    1976364 |      nan           |
|            nan | Pyralidae                                   | Pyralidae                                        |     5336 | EXACT       |           94 | ACCEPTED | FAMILY  | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Pyralidae      | nan            | nan                         | Pyralidae                   | nan                          |       5336 |      nan           |
|            nan | Afrida ydatodes                             | Afrida ydatodes Dyar, 1913                       |  8924506 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Nolidae        | Afrida         | Afrida ydatodes             | Afrida ydatodes             | Dyar, 1913                   |    8924506 |      nan           |
|            nan | Meganola bifiliferata                       | Meganola bifiliferata (Walker, 1863)             |  8126835 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Nolidae        | Nola           | Nola bifiliferata           | Meganola bifiliferata       | (Walker, 1863)               |    8126835 |        1.17082e+07 |
|            nan | Clemensia brunneomedia                      | Clemensia brunneomedia Schaus, 1905              |  5114865 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Clemensia      | Clemensia brunneomedia      | Clemensia brunneomedia      | Schaus, 1905                 |    5114865 |      nan           |
|            nan | Eucereon xanthoperas                        | Eucereon xanthoperas Hampson, 1898               |  1810308 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | Eucereon xanthoperas        | Eucereon xanthoperas        | Hampson, 1898                |    1810308 |      nan           |
|            nan | Eucereon rosinum                            | Eucereon rosina Walker, 1854                     |  1810278 | FUZZY       |           95 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | Eucereon rosina             | Eucereon rosina             | Walker, 1854                 |    1810278 |      nan           |
|            nan | Pheia elegans                               | Pheia elegans Druce, 1884                        |  1811506 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Pheia          | Pheia elegans               | Pheia elegans               | Druce, 1884                  |    1811506 |      nan           |
|            nan | Idaea subfervens                            | Idaea subfervens (Prout, 1920)                   |  5147031 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Idaea          | Idaea subfervens            | Idaea subfervens            | (Prout, 1920)                |    5147031 |      nan           |
|            nan | Hymenia perspectalis                        | Hymenia perspectalis (Hübner, 1796)              |  1886374 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Hymenia        | Hymenia perspectalis        | Hymenia perspectalis        | (Hübner, 1796)               |    1886374 |      nan           |
|            nan | Metanema bonadea                            | Metanema bonadea Druce, 1892                     |  5149624 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Metanema       | Metanema bonadea            | Metanema bonadea            | Druce, 1892                  |    5149624 |      nan           |
|            nan | Palthis                                     | Palthis Hübner, 1825                             |  1781924 | EXACT       |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Palthis        | nan                         | Palthis                     | Hübner, 1825                 |    1781924 |      nan           |
|            nan | Nematocampa completa                        | Nematocampa completa Warren, 1904                |  1991016 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nematocampa    | Nematocampa completa        | Nematocampa completa        | Warren, 1904                 |    1991016 |      nan           |
|            nan | Elaphria subobliqua                         | Elaphria subobliqua Walker, 1858                 |  1780407 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Elaphria       | Elaphria subobliqua         | Elaphria subobliqua         | Walker, 1858                 |    1780407 |      nan           |
|            nan | Nemoria vermiculata                         | Nemoria vermiculata (Dognin, 1914)               |  1981168 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nemoria        | Nemoria vermiculata         | Nemoria vermiculata         | (Dognin, 1914)               |    1981168 |      nan           |
|            nan | Epimecis matronaria--nasica                 | Epimecis Hübner                                  |  1985578 | HIGHERRANK  |           92 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Epimecis       | nan                         | Epimecis                    | Hübner                       |    1985578 |      nan           |
|            nan | Macaria approximaria--gambarina--ostia      | Arthropoda                                       |       54 | HIGHERRANK  |           92 | ACCEPTED | PHYLUM  | Animalia  | Arthropoda    | nan            | nan             | nan            | nan            | nan                         | Arthropoda                  | nan                          |         54 |      nan           |
|            nan | Synnomos sp_san_luis_a                      | Synnomos Guenée                                  |  1963630 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Synnomos       | nan                         | Synnomos                    | Guenée                       |    1963630 |      nan           |
|            nan | Arctiidae                                   | Arctiidae                                        |     4548 | EXACT       |           94 | ACCEPTED | FAMILY  | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Arctiidae      | nan            | nan                         | Arctiidae                   | nan                          |       4548 |      nan           |
|            nan | Euphyia sp_group_san_luis                   | Euphyia Hübner                                   |  1965123 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Euphyia        | nan                         | Euphyia                     | Hübner                       |    1965123 |      nan           |
|            nan | Bryolymnia roma                             | Bryolymnia roma Druce, 1894                      |  1764152 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Bryolymnia     | Bryolymnia roma             | Bryolymnia roma             | Druce, 1894                  |    1764152 |      nan           |
|            nan | Hampsonodes mastoides                       | Hampsonodes mastoides Hampson, 1910              |  1791331 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Hampsonodes    | Hampsonodes mastoides       | Hampsonodes mastoides       | Hampson, 1910                |    1791331 |      nan           |
|            nan | Microxydia ruficomma                        | Microxydia ruficomma Prout, 1910                 |  1954247 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Microxydia     | Microxydia ruficomma        | Microxydia ruficomma        | Prout, 1910                  |    1954247 |      nan           |
|            nan | Hypercompe nemophila                        | Hypercompe nemophila Herrich-Schäffer, 1853      |  1810980 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hypercompe     | Hypercompe nemophila        | Hypercompe nemophila        | Herrich-Schäffer, 1853       |    1810980 |      nan           |
|            nan | Synchlora astraeoides                       | Synchlora astraeoides (Warren, 1901)             |  1962086 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Synchlora      | Synchlora astraeoides       | Synchlora astraeoides       | (Warren, 1901)               |    1962086 |      nan           |
|            nan | Mocis                                       | Mocis Hübner                                     |  9175687 | EXACT       |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Mocis          | nan                         | Mocis                       | Hübner                       |    9175687 |      nan           |
|            nan | Oospila venezuelata                         | Oospila venezuelata (Walker, 1861)               |  1973665 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Oospila        | Oospila venezuelata         | Oospila venezuelata         | (Walker, 1861)               |    1973665 |      nan           |
|            nan | Idalus perlineosa                           | Idalus perlineosa Rothschild, 1917               |  1814683 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Idalus         | Idalus perlineosa           | Idalus perlineosa           | Rothschild, 1917             |    1814683 |      nan           |
|            nan | Dycladia emerita                            | Dycladia emerita                                 |  9111683 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Dycladia       | Dycladia emerita            | Dycladia emerita            | nan                          |    9111683 |      nan           |
|            nan | Melese monima                               | Melese monima Schaus, 1910                       |  1817583 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Melese         | Melese monima               | Melese monima               | Schaus, 1910                 |    1817583 |      nan           |
|            nan | Eubaphe unicolor                            | Eubaphe unicolor (Robinson, 1869)                |  1985952 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eubaphe        | Eubaphe unicolor            | Eubaphe unicolor            | (Robinson, 1869)             |    1985952 |      nan           |
|            nan | Melese laodamia                             | Melese laodamia Druce, 1884                      |  1817589 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Melese         | Melese laodamia             | Melese laodamia             | Druce, 1884                  |    1817589 |      nan           |
|            nan | Nephodia betala                             | Nephodia betala (Druce, 1892)                    |  1981662 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nephodia       | Nephodia betala             | Nephodia betala             | (Druce, 1892)                |    1981662 |      nan           |
|            nan | Sphingidae                                  | Sphingidae                                       |     8868 | EXACT       |           94 | ACCEPTED | FAMILY  | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sphingidae     | nan            | nan                         | Sphingidae                  | nan                          |       8868 |      nan           |
|            nan | Elaphria hyposcota                          | Elaphria hyposcota Hampson, 1909                 |  1780310 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Elaphria       | Elaphria hypophaea          | Elaphria hyposcota          | Hampson, 1909                |    1780310 |        1.78035e+06 |
|            nan | Thysanopyga carfinia                        | Thysanopyga carfinia (Druce, 1893)               |  1958378 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Thysanopyga    | Thysanopyga carfinia        | Thysanopyga carfinia        | (Druce, 1893)                |    1958378 |      nan           |
|            nan | Scena potentia                              | Scena potentia Druce, 1894                       |  1804522 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Scena          | Scena potentia              | Scena potentia              | Druce, 1894                  |    1804522 |      nan           |
|            nan | Aclytia heber                               | Aclytia heber Cramer, 1782                       |  5115046 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Aclytia        | Aclytia heber               | Aclytia heber               | Cramer, 1782                 |    5115046 |      nan           |
|            nan | Feltia subterranea                          | Feltia subterranea Fabricius, 1794               |  7400814 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Feltia         | Feltia subterranea          | Feltia subterranea          | Fabricius, 1794              |    7400814 |      nan           |
|            nan | Gonodes liquida                             | Gonodes liquida Möschler, 1886                   |  1774929 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Gonodes        | Gonodes liquida             | Gonodes liquida             | Möschler, 1886               |    1774929 |      nan           |
|            nan | Eois sp_san_luis_s                          | Eois Hübner, 1818                                |  1989610 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eois           | nan                         | Eois                        | Hübner, 1818                 |    1989610 |      nan           |
|            nan | Neotherina imperilla                        | Neotherina imperilla (Dognin, 1911)              |  1991354 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Neotherina     | Neotherina imperilla        | Neotherina imperilla        | (Dognin, 1911)               |    1991354 |      nan           |
|            nan | Lomographa argentata                        | Lomographa argentata (Schaus, 1911)              |  1986078 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Lomographa     | Lomographa argentata        | Lomographa argentata        | (Schaus, 1911)               |    1986078 |      nan           |
|            nan | Simopteryx torquataria                      | Simopteryx torquataria (Walker, 1860)            |  1976895 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Simopteryx     | Simopteryx torquataria      | Simopteryx torquataria      | (Walker, 1860)               |    1976895 |      nan           |
|            nan | Saturniidae                                 | Animalia                                         |        1 | HIGHERRANK  |           92 | ACCEPTED | KINGDOM | Animalia  | nan           | nan            | nan             | nan            | nan            | nan                         | Animalia                    | nan                          |          1 |      nan           |
|            nan | Synchlora gerularia                         | Synchlora gerularia (Hübner)                     |  1962118 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Synchlora      | Synchlora gerularia         | Synchlora gerularia         | (Hübner)                     |    1962118 |      nan           |
|            nan | Synchlora dependens                         | Synchlora dependens (Warren, 1904)               |  1962082 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Synchlora      | Synchlora dependens         | Synchlora dependens         | (Warren, 1904)               |    1962082 |      nan           |
|            nan | Eois sp_san_luis_b                          | Eois Hübner, 1818                                |  1989610 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eois           | nan                         | Eois                        | Hübner, 1818                 |    1989610 |      nan           |
|            nan | Simena luctifera                            | Simena luctifera Walker, 1856                    |  1965887 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Simena         | Simena luctifera            | Simena luctifera            | Walker, 1856                 |    1965887 |      nan           |
|            nan | Epimecis semicompleta                       | Epimecis semicompleta (Warren, 1905)             |  5149243 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Epimecis       | Epimecis semicompleta       | Epimecis semicompleta       | (Warren, 1905)               |    5149243 |      nan           |
|            nan | Antiblemma pira                             | Antiblemma pira                                  |  1794667 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Antiblemma     | Antiblemma pira             | Antiblemma pira             | nan                          |    1794667 |      nan           |
|            nan | Anisoperas atropunctaria                    | Anisoperas atropunctaria (Walker, 1862)          |  1986752 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Anisoperas     | Anisoperas atropunctaria    | Anisoperas atropunctaria    | (Walker, 1862)               |    1986752 |      nan           |
|            nan | Dyspteris tenuivitta                        | Dyspteris tenuivitta Dognin, 1908                |  1961352 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Dyspteris      | Dyspteris tenuivitta        | Dyspteris tenuivitta        | Dognin, 1908                 |    1961352 |      nan           |
|            nan | Macaria abydata--nundinata                  | Arthropoda                                       |       54 | HIGHERRANK  |           92 | ACCEPTED | PHYLUM  | Animalia  | Arthropoda    | nan            | nan             | nan            | nan            | nan                         | Arthropoda                  | nan                          |         54 |      nan           |
|            nan | Anycles adusta                              | Anycles adusta Felder, 1869                      |  1807283 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Anycles        | Anycles adusta              | Anycles adusta              | Felder, 1869                 |    1807283 |      nan           |
|            nan | Napata lelex                                | Napata lelex                                     | 10277831 | EXACT       |           97 | DOUBTFUL | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Uranophora     | Napata lelex                | Napata lelex                | nan                          |   10277831 |      nan           |
|            nan | Opisthoxia miletia                          | Opisthoxia miletia (Druce, 1892)                 |  1960255 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Opisthoxia     | Opisthoxia miletia          | Opisthoxia miletia          | (Druce, 1892)                |    1960255 |      nan           |
|            nan | Ametris nitocris                            | Ametris nitocris (Cramer)                        |  1959637 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Ametris        | Ametris nitocris            | Ametris nitocris            | (Cramer)                     |    1959637 |      nan           |
|            nan | Hagnagora sp_san_luis                       | Hagnagora Druce, 1885                            |  1978245 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Hagnagora      | nan                         | Hagnagora                   | Druce, 1885                  |    1978245 |      nan           |
|            nan | Periclina sp_san_luis_a                     | Periclina Guenée                                 |  1968537 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Periclina      | nan                         | Periclina                   | Guenée                       |    1968537 |      nan           |
|            nan | Nematocampa c                               | Nematocampa Guenée                               |  1991004 | EXACT       |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nematocampa    | nan                         | Nematocampa                 | Guenée                       |    1991004 |      nan           |
|            nan | Herbita                                     | Herbita Walker, 1860                             |  1976370 | EXACT       |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Herbita        | nan                         | Herbita                     | Walker, 1860                 |    1976370 |      nan           |
|            nan | Macrocneme iole                             | Macrocneme iole Druce, 1884                      |  1805189 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Macrocneme     | Macrocneme chrysitis        | Macrocneme iole             | Druce, 1884                  |    1805189 |        1.8052e+06  |
|            nan | Isogona continua--natatrix                  | Isogona Guenée, 1852                             |  1790931 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Isogona        | nan                         | Isogona                     | Guenée, 1852                 |    1790931 |      nan           |
|            nan | Lichnoptera illudens                        | Lichnoptera illudens Walker, 1856                |  1800100 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Lichnoptera    | Lichnoptera illudens        | Lichnoptera illudens        | Walker, 1856                 |    1800100 |      nan           |
|            nan | Noctuidae delta                             | Noctuidae                                        |     7015 | HIGHERRANK  |           94 | ACCEPTED | FAMILY  | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | nan            | nan                         | Noctuidae                   | nan                          |       7015 |      nan           |
|            nan | Nepheloleuca illiturata                     | Nepheloleuca illiturata (Guenée, 1858)           |  1989583 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nepheloleuca   | Nepheloleuca politia        | Nepheloleuca illiturata     | (Guenée, 1858)               |    1989583 |        1.14625e+07 |
|            nan | Nemoria pescadora                           | Nemoria pescadora (Beutelspacher-Baigts, 1984)   |  1981308 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nemoria        | Nemoria pescadora           | Nemoria pescadora           | (Beutelspacher-Baigts, 1984) |    1981308 |      nan           |
|            nan | Semaeopus viridiplaga                       | Semaeopus viridiplaga (Walker, 1861)             |  1956136 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Semaeopus      | Semaeopus viridiplaga       | Semaeopus viridiplaga       | (Walker, 1861)               |    1956136 |      nan           |
|            nan | Nola sp_san_luis_a                          | Nola Leach, 1815                                 |  1801758 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Nolidae        | Nola           | nan                         | Nola                        | Leach, 1815                  |    1801758 |      nan           |
|            nan | Oxydia trychiata                            | Oxydia trychiata Guenée                          |  1976746 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Oxydia         | Oxydia trychiata            | Oxydia trychiata            | Guenée                       |    1976746 |      nan           |
|            nan | Melese                                      | Melese Walker, 1854                              |  1817551 | EXACT       |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Melese         | nan                         | Melese                      | Walker, 1854                 |    1817551 |      nan           |
|            nan | Eucereon tigrata                            | Eucereon tigrata Herrich-Schäffer, 1855          |  1810172 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | Eucereon tigrata            | Eucereon tigrata            | Herrich-Schäffer, 1855       |    1810172 |      nan           |
|            nan | Lophocampa citrina                          | Lophocampa citrina Sepp, 1852                    |  1817763 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Lophocampa     | Lophocampa citrina          | Lophocampa citrina          | Sepp, 1852                   |    1817763 |      nan           |
|            nan | Hydata                                      | Hydata Walker, 1863                              |  1987240 | EXACT       |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Hydata         | nan                         | Hydata                      | Walker, 1863                 |    1987240 |      nan           |
|            nan | Amastus aconia                              | Amastus aconia Herrich-Schäffer, 1853            |  5117348 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Amastus        | Amastus aconia              | Amastus aconia              | Herrich-Schäffer, 1853       |    5117348 |      nan           |
|            nan | Antiblemma concinnula                       | Antiblemma concinnula Walker, 1865               |  1794642 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Antiblemma     | Antiblemma concinnula       | Antiblemma concinnula       | Walker, 1865                 |    1794642 |      nan           |
|            nan | Spragueia lepus                             | Spragueia Grote, 1875                            |  1758765 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Spragueia      | nan                         | Spragueia                   | Grote, 1875                  |    1758765 |      nan           |
|            nan | Hyponerita tipolis                          | Hyponerita tipolis Druce, 1896                   |  1808620 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hyponerita     | Hyponerita tipolis          | Hyponerita tipolis          | Druce, 1896                  |    1808620 |      nan           |
|            nan | Episcepsis thetis                           | Episcepsis thetis                                | 12186497 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Episcepsis     | Episcepsis thetis           | Episcepsis thetis           | nan                          |   12186497 |      nan           |
|            nan | Hypercompe icasia                           | Hypercompe icasia Cramer, 1777                   |  1811121 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hypercompe     | Hypercompe icasia           | Hypercompe icasia           | Cramer, 1777                 |    1811121 |      nan           |
|            nan | Hampsonodes melagona                        | Hampsonodes melagona Hampson, 1910               |  1791319 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Hampsonodes    | Hampsonodes melagona        | Hampsonodes melagona        | Hampson, 1910                |    1791319 |      nan           |
|            nan | Ramphia albizona                            | Ramphia albizona Latreille, 1817                 |  1767499 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Ramphia        | Ramphia albizona            | Ramphia albizona            | Latreille, 1817              |    1767499 |      nan           |
|            nan | Nemoria eugeniae                            | Nemoria eugeniae Pitkin, 1993                    |  1981212 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nemoria        | Nemoria eugeniae            | Nemoria eugeniae            | Pitkin, 1993                 |    1981212 |      nan           |
|            nan | Procalypta victorina                        | Procalypta victorina Druce, 1884                 |  1812490 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Procalypta     | Procalypta subcyanea        | Procalypta victorina        | Druce, 1884                  |    1812490 |        1.81249e+06 |
|            nan | Itomia opistographa--lignaris               | Itomia Hübner, 1823                              |  4688242 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Itomia         | nan                         | Itomia                      | Hübner, 1823                 |    4688242 |      nan           |
|            nan | Thysanoprymna superba                       | Thysanoprymna superba Schaus, 1889               |  1815290 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Thysanoprymna  | Thysanoprymna superba       | Thysanoprymna superba       | Schaus, 1889                 |    1815290 |      nan           |
|            nan | Macaria tenuiscripta--trigonata             | Arthropoda                                       |       54 | HIGHERRANK  |           92 | ACCEPTED | PHYLUM  | Animalia  | Arthropoda    | nan            | nan             | nan            | nan            | nan                         | Arthropoda                  | nan                          |         54 |      nan           |
|            nan | Dichorda obliquata                          | Dichorda obliquata Warren, 1904                  |  1980252 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Dichorda       | Dichorda obliquata          | Dichorda obliquata          | Warren, 1904                 |    1980252 |      nan           |
|            nan | Xenosoma flavicedes                         | Xenosoma flavisedes Dognin, 1891                 |  5116373 | FUZZY       |           94 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Xenosoma       | Xenosoma flavisedes         | Xenosoma flavisedes         | Dognin, 1891                 |    5116373 |      nan           |
|            nan | Herbita amicaria                            | Herbita amicaria (Schaus, 1912)                  |  1976410 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Herbita        | Herbita amicaria            | Herbita amicaria            | (Schaus, 1912)               |    1976410 |      nan           |
|            nan | Opharus bimaculata                          | Opharus bimaculata Dewitz, 1877                  |  5117646 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Opharus        | Opharus bimaculata          | Opharus bimaculata          | Dewitz, 1877                 |    5117646 |      nan           |
|            nan | Hyalurga sora                               | Hyalurga sora Boisduval, 1870                    |  1804196 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hyalurga       | Hyalurga sora               | Hyalurga sora               | Boisduval, 1870              |    1804196 |      nan           |
|            nan | Gelechiidae Dichomeris                      | Gelechiidae                                      |     3553 | EXACT       |           94 | ACCEPTED | FAMILY  | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Gelechiidae    | nan            | nan                         | Gelechiidae                 | nan                          |       3553 |      nan           |
|            nan | Melanolophia sp_san_luis                    | Melanolophia Hulst, 1896                         |  1962704 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Melanolophia   | nan                         | Melanolophia                | Hulst, 1896                  |    1962704 |      nan           |
|            nan | Cliniodes opalalis                          | Cliniodes opalalis Guenée, 1854                  |  1891276 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Cliniodes      | Cliniodes opalalis          | Cliniodes opalalis          | Guenée, 1854                 |    1891276 |      nan           |
|            nan | Eois sp_san_luis_f                          | Eois Hübner, 1818                                |  1989610 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eois           | nan                         | Eois                        | Hübner, 1818                 |    1989610 |      nan           |
|            nan | Hygrochroma nondina                         | Hygrochroma nondina Druce, 1892                  |  5144328 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Hygrochroma    | Hygrochroma nondina         | Hygrochroma nondina         | Druce, 1892                  |    5144328 |      nan           |
|            nan | Leucanopsis longa                           | Leucanopsis longa Grote, 1880                    |  1810025 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Leucanopsis    | Leucanopsis longa           | Leucanopsis longa           | Grote, 1880                  |    1810025 |      nan           |
|            nan | Crinodes schausi                            | Crinodes schausi Rothschild, 1917                |  1824186 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Notodontidae   | Crinodes       | Crinodes schausi            | Crinodes schausi            | Rothschild, 1917             |    1824186 |      nan           |
|            nan | Oospila athena                              | Oospila athena (Druce, 1892)                     |  1973640 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Oospila        | Oospila athena              | Oospila athena              | (Druce, 1892)                |    1973640 |      nan           |
|            nan | Symphlebia herbosa                          | Symphlebia herbosa Schaus, 1910                  |  1805277 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Symphlebia     | Symphlebia herbosa          | Symphlebia herbosa          | Schaus, 1910                 |    1805277 |      nan           |
|            nan | Hymenomima umbelularia                      | Hymenomima umbelularia (Hübner)                  |  1975855 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Hymenomima     | Hymenomima umbelularia      | Hymenomima umbelularia      | (Hübner)                     |    1975855 |      nan           |
|            nan | Phrudocentra opaca                          | Phrudocentra opaca (Butler, 1881)                |  1985016 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Phrudocentra   | Phrudocentra opaca          | Phrudocentra opaca          | (Butler, 1881)               |    1985016 |      nan           |
|            nan | Xylophanes                                  | Xylophanes Hübner                                |  1862076 | EXACT       |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sphingidae     | Xylophanes     | nan                         | Xylophanes                  | Hübner                       |    1862076 |      nan           |
|            nan | Amaxia apyga                                | Amaxia apyga Hampson, 1901                       |  1814045 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Amaxia         | Amaxia apyga                | Amaxia apyga                | Hampson, 1901                |    1814045 |      nan           |
|            nan | Leucotmemis nexa                            | Leucotmemis nexa Herrich-Schäffer, 1854          |  1817086 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Leucotmemis    | Leucotmemis nexa            | Leucotmemis nexa            | Herrich-Schäffer, 1854       |    1817086 |      nan           |
|            nan | Hemiceras vecina                            | Hemiceras vecina Schaus, 1901                    |  5118706 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Notodontidae   | Hemiceras      | Hemiceras vecina            | Hemiceras vecina            | Schaus, 1901                 |    5118706 |      nan           |
|            nan | Eucereon aroa                               | Eucereon aroa Schaus, 1894                       |  1810220 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | Eucereon aroa               | Eucereon aroa               | Schaus, 1894                 |    1810220 |      nan           |
|            nan | Phalaenophana pyramusalis                   | Phalaenophana pyramusalis (Walker, 1859)         |  9342938 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Phaeolita      | Phaeolita pyramusalis       | Phalaenophana pyramusalis   | (Walker, 1859)               |    9342938 |        1.79083e+06 |
|            nan | Natada fusca                                | Natada fusca (Druce, 1887)                       |  1753658 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Limacodidae    | Natada         | Natada fusca                | Natada fusca                | (Druce, 1887)                |    1753658 |      nan           |
|            nan | Dysschema mariamne                          | Dysschema mariamne Geyer, 1838                   |  5117422 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Dysschema      | Dysschema mariamne          | Dysschema mariamne          | Geyer, 1838                  |    5117422 |      nan           |
|            nan | Herminodes sp_san_luis_a                    | Herminodes Guenée, 1852                          |  4696204 | HIGHERRANK  |           94 | SYNONYM  | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Scolecocampa   | nan                         | Herminodes                  | Guenée, 1852                 |    4696204 |        1.76518e+06 |
|            nan | Paragonia cruraria                          | Paragonia cruraria (Herrich-Schäffer)            |  5144363 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Paragonia      | Paragonia cruraria          | Paragonia cruraria          | (Herrich-Schäffer)           |    5144363 |      nan           |
|            nan | Macrodes                                    | Macrodes Guenée, 1854                            |  1777716 | EXACT       |           92 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Macrodes       | nan                         | Macrodes                    | Guenée, 1854                 |    1777716 |      nan           |
|            nan | Afrida sp_san_luis_a                        | Afrida Möschler, 1886                            |  1802159 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Nolidae        | Afrida         | nan                         | Afrida                      | Möschler, 1886               |    1802159 |      nan           |
|            nan | Microphysetica hermeasalis                  | Microphysetica hermeasalis                       | 10439369 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Microphysetica | Microphysetica hermeasalis  | Microphysetica hermeasalis  | nan                          |   10439369 |      nan           |
|            nan | Talara mona                                 | Talara mona Dyar, 1914                           |  1814769 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Talara         | Talara mona                 | Talara mona                 | Dyar, 1914                   |    1814769 |      nan           |
|            nan | Xenosoma nigromarginatum                    | Xenosoma nigromarginatum Druce, 1886             |  5116374 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Xenosoma       | Xenosoma nigromarginatum    | Xenosoma nigromarginatum    | Druce, 1886                  |    5116374 |      nan           |
|            nan | Zale peruncta                               | Zale peruncta Guenée, 1852                       |  5112997 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Zale           | Zale peruncta               | Zale peruncta               | Guenée, 1852                 |    5112997 |      nan           |
|            nan | Gonodonta paraequalis                       | Gonodonta paraequalis Todd, 1959                 |  1770041 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Gonodonta      | Gonodonta paraequalis       | Gonodonta paraequalis       | Todd, 1959                   |    1770041 |      nan           |
|            nan | Perigea leucanoides                         | Perigea leucanioides Hampson, 1908               |  1790742 | FUZZY       |           95 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Perigea        | Perigea leucanioides        | Perigea leucanioides        | Hampson, 1908                |    1790742 |      nan           |
|            nan | Dysschema lycaste                           | Dysschema lycaste Klug, 1836                     |  5117506 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Dysschema      | Dysschema lycaste           | Dysschema lycaste           | Klug, 1836                   |    5117506 |      nan           |
|            nan | Hypena lollia                               | Hypena lollia Druce, 1890                        |  8256857 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hypena         | Hypena lollia               | Hypena lollia               | Druce, 1890                  |    8256857 |      nan           |
|            nan | Eacles ormondei                             | Eacles ormondei Schaus, 1889                     |  1865108 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Saturniidae    | Eacles         | Eacles ormondei             | Eacles ormondei             | Schaus, 1889                 |    1865108 |      nan           |
|            nan | Antiblemma sp_san_luis_a                    | Antiblemma Hübner, 1823                          |  1794630 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Antiblemma     | nan                         | Antiblemma                  | Hübner, 1823                 |    1794630 |      nan           |
|            nan | Eucereon relegata                           | Eucereon relegatum Schaus, 1911                  |  1810152 | FUZZY       |           96 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | Eucereon relegatum          | Eucereon relegatum          | Schaus, 1911                 |    1810152 |      nan           |
|            nan | Hemiceras modesta                           | Hemiceras modesta Butler, 1879                   |  5118590 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Notodontidae   | Hemiceras      | Hemiceras modesta           | Hemiceras modesta           | Butler, 1879                 |    5118590 |      nan           |
|            nan | Oecophoridae                                | Oecophoridae                                     |     9689 | EXACT       |           94 | ACCEPTED | FAMILY  | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Oecophoridae   | nan            | nan                         | Oecophoridae                | nan                          |       9689 |      nan           |
|            nan | Cosmosoma advena                            | Cosmosoma advena Druce, 1884                     |  5116277 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Cosmosoma      | Cosmosoma advena            | Cosmosoma advena            | Druce, 1884                  |    5116277 |      nan           |
|            nan | Phyllodonta latrata                         | Phyllodonta latrata (Guenée)                     |  1963959 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Phyllodonta    | Phyllodonta latrata         | Phyllodonta latrata         | (Guenée)                     |    1963959 |      nan           |
|            nan | Anisoperas lurida                           | Anisoperas Warren, 1895                          |  1986746 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Anisoperas     | nan                         | Anisoperas                  | Warren, 1895                 |    1986746 |      nan           |
|            nan | Ammalo helops                               | Ammalo helops Cramer, 1775                       |  1814182 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Ammalo         | Ammalo helops               | Ammalo helops               | Cramer, 1775                 |    1814182 |      nan           |
|            nan | Herminodes atrosignata                      | Herminodes atrosignata (Walker, 1858)            |  8028211 | EXACT       |           97 | DOUBTFUL | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Scolecocampa   | Herminodes atrosignata      | Herminodes atrosignata      | (Walker, 1858)               |    8028211 |      nan           |
|            nan | Eusarca minucia                             | Eusarca minucia (Druce, 1892)                    | 10466176 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eusarca        | Eusarca minucia             | Eusarca minucia             | (Druce, 1892)                |   10466176 |      nan           |
|            nan | Orthofidonia sp_san_luis_a                  | Orthofidonia Packard, 1876                       |  1980504 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Orthofidonia   | nan                         | Orthofidonia                | Packard, 1876                |    1980504 |      nan           |
|            nan | Pelochyta affinis                           | Pelochyta affinis Rothschild, 1909               |  1806777 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Pelochyta      | Pelochyta affinis           | Pelochyta affinis           | Rothschild, 1909             |    1806777 |      nan           |
|            nan | Nepheloleuca politia                        | Nepheloleuca politia (Cramer, 1777)              |  1989582 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nepheloleuca   | Nepheloleuca politia        | Nepheloleuca politia        | (Cramer, 1777)               |    1989582 |      nan           |
|            nan | Diacme mopsalis                             | Diacme mopsalis                                  | 11506397 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Diacme         | Diacme mopsalis             | Diacme mopsalis             | nan                          |   11506397 |      nan           |
|            nan | Oecophoridae Ethmia                         | Oecophoridae                                     |     9689 | EXACT       |           94 | ACCEPTED | FAMILY  | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Oecophoridae   | nan            | nan                         | Oecophoridae                | nan                          |       9689 |      nan           |
|            nan | Epitausa dilina                             | Epitausa dilina Herrich-Schäffer, 1858           |  1772235 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Epitausa       | Epitausa dilina             | Epitausa dilina             | Herrich-Schäffer, 1858       |    1772235 |      nan           |
|            nan | Hygrochroma olivinaria                      | Hygrochroma olivinaria Herrich-Schäffer, 1855    |  5144327 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Hygrochroma    | Hygrochroma olivinaria      | Hygrochroma olivinaria      | Herrich-Schäffer, 1855       |    5144327 |      nan           |
|            nan | Pyrinia selecta                             | Pyrinia selecta Schaus, 1913                     |  1988993 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Pyrinia        | Pyrinia selecta             | Pyrinia selecta             | Schaus, 1913                 |    1988993 |      nan           |
|            nan | Opharus rudis                               | Opharus rudis Schaus, 1911                       |  5117618 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Opharus        | Opharus rudis               | Opharus rudis               | Schaus, 1911                 |    5117618 |      nan           |
|            nan | Tineidae Nemapogon                          | Tineidae                                         |     9412 | EXACT       |           94 | ACCEPTED | FAMILY  | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Tineidae       | nan            | nan                         | Tineidae                    | nan                          |       9412 |      nan           |
|            nan | Blastobasidae                               | Blastobasidae                                    |     4865 | EXACT       |           94 | ACCEPTED | FAMILY  | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Blastobasidae  | nan            | nan                         | Blastobasidae               | nan                          |       4865 |      nan           |
|            nan | Lepidokirbyia vittipes                      | Lepidokirbyia vittipes Walker, 1855              |  1818377 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Lepidokirbyia  | Lepidokirbyia vittipes      | Lepidokirbyia vittipes      | Walker, 1855                 |    1818377 |      nan           |
|            nan | Gardinia anopla                             | Gardinia anopla Hering, 1925                     |  5117880 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Gardinia       | Gardinia anopla             | Gardinia anopla             | Hering, 1925                 |    5117880 |      nan           |
|            nan | Epicrisias eschara                          | Epicrisias eschara Dyar, 1912                    |  1805528 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Epicrisias     | Epicrisias eschara          | Epicrisias eschara          | Dyar, 1912                   |    1805528 |      nan           |
|            nan | Anomis gentilis                             | Anomis gentilis Schaus, 1912                     |  1785093 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Anomis         | Anomis gentilis             | Anomis gentilis             | Schaus, 1912                 |    1785093 |      nan           |
|            nan | Depressariidae Ethmia                       | Depressariidae                                   |  4705755 | EXACT       |           94 | ACCEPTED | FAMILY  | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Depressariidae | nan            | nan                         | Depressariidae              | nan                          |    4705755 |      nan           |
|            nan | Diphthera festiva                           | Diphthera festiva Fabricius, 1775                |  5108951 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Diphthera      | Diphthera festiva           | Diphthera festiva           | Fabricius, 1775              |    5108951 |      nan           |
|            nan | Tosale oviplagalis                          | Tosale oviplagalis Walker, 1866                  |  1872514 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Pyralidae      | Tosale         | Tosale oviplagalis          | Tosale oviplagalis          | Walker, 1866                 |    1872514 |      nan           |
|            nan | Eucereon aurantiaca                         | Eucereon aurantiaca Draudt, 1915                 |  1810306 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | Eucereon pseudarchias       | Eucereon aurantiaca         | Draudt, 1915                 |    1810306 |        1.81032e+06 |
|            nan | Elysius conspersus                          | Elysius conspersus Walker, 1855                  |  1811312 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Elysius        | Elysius conspersus          | Elysius conspersus          | Walker, 1855                 |    1811312 |      nan           |
|            nan | Bagisara laverna                            | Bagisara laverna Druce, 1889                     |  9567770 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Bagisara       | Bagisara laverna            | Bagisara laverna            | Druce, 1889                  |    9567770 |      nan           |
|            nan | Semiothisa divergentata                     | Semiothisa divergentata (Snellen, 1874)          |  1958807 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Semiothisa     | Semiothisa divergentata     | Semiothisa divergentata     | (Snellen, 1874)              |    1958807 |      nan           |
|            nan | Darceta                                     | Darceta Herrich-Schäffer, 1856                   |  1772115 | EXACT       |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Darceta        | nan                         | Darceta                     | Herrich-Schäffer, 1856       |    1772115 |      nan           |
|            nan | Symphlebia pyrgion                          | Symphlebia pyrgion Druce, 1897                   |  1805284 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Symphlebia     | Symphlebia pyrgion          | Symphlebia pyrgion          | Druce, 1897                  |    1805284 |      nan           |
|            nan | Hemiceras nigricosta                        | Hemiceras nigricosta Schaus, 1901                |  5118718 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Notodontidae   | Hemiceras      | Hemiceras nigricosta        | Hemiceras nigricosta        | Schaus, 1901                 |    5118718 |      nan           |
|            nan | Amaxia lepida                               | Amaxia lepida Schaus, 1912                       |  1814052 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Amaxia         | Amaxia lepida               | Amaxia lepida               | Schaus, 1912                 |    1814052 |      nan           |
|            nan | Chetone angulosa                            | Chetone angulosa Walker, 1854                    |  1806477 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Chetone        | Chetone angulosa            | Chetone angulosa            | Walker, 1854                 |    1806477 |      nan           |
|            nan | Prochoerodes pilosa                         | Prochoerodes pilosa Warren, 1897                 |  1963844 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Prochoerodes   | Prochoerodes pilosa         | Prochoerodes pilosa         | Warren, 1897                 |    1963844 |      nan           |
|            nan | Nemoria astraea                             | Nemoria astraea (Druce, 1892)                    |  1981252 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nemoria        | Nemoria astraea             | Nemoria astraea             | (Druce, 1892)                |    1981252 |      nan           |
|            nan | Hypercompe laeta                            | Hypercompe laeta Walker, 1855                    |  1811106 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hypercompe     | Hypercompe laeta            | Hypercompe laeta            | Walker, 1855                 |    1811106 |      nan           |
|            nan | Leuciris fimbriaria                         | Leuciris fimbriaria (Stoll, 1781)                |  1978464 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Leuciris       | Leuciris fimbriaria         | Leuciris fimbriaria         | (Stoll, 1781)                |    1978464 |      nan           |
|            nan | Crambidae Scoparia                          | Crambidae                                        |     8841 | EXACT       |           94 | ACCEPTED | FAMILY  | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | nan            | nan                         | Crambidae                   | nan                          |       8841 |      nan           |
|            nan | Hypena andraca                              | Hypena andraca Druce, 1890                       | 10354717 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hypena         | Hypena andraca              | Hypena andraca              | Druce, 1890                  |   10354717 |      nan           |
|            nan | Rivula leucosticta                          | Rivula leucosticta Swinhoe, 1895                 |  1769863 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Rivula         | Rivula leucosticta          | Rivula leucosticta          | Swinhoe, 1895                |    1769863 |      nan           |
|            nan | Macaria delia                               | Macaria delia (Schaus, 1912)                     | 10616747 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Macaria        | Macaria delia               | Macaria delia               | (Schaus, 1912)               |   10616747 |      nan           |
|            nan | Himeromima aulis                            | Himeromima aulis (Druce, 1892)                   |  1985485 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Himeromima     | Himeromima aulis            | Himeromima aulis            | (Druce, 1892)                |    1985485 |      nan           |
|            nan | Sphacelodes vulneraria                      | Sphacelodes vulneraria (Hübner, 1823)            |  1990594 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Sphacelodes    | Sphacelodes vulneraria      | Sphacelodes vulneraria      | (Hübner, 1823)               |    1990594 |      nan           |
|            nan | Leucula meganira                            | Leucula meganira Druce, 1892                     |  1990651 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Leucula        | Leucula meganira            | Leucula meganira            | Druce, 1892                  |    1990651 |      nan           |
|            nan | Eumarozia beckeri                           | Eumarozia beckeri Clarke, 1973                   |  1743460 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Tortricidae    | Eumarozia      | Eumarozia beckeri           | Eumarozia beckeri           | Clarke, 1973                 |    1743460 |      nan           |
|            nan | Glyphodes extorris                          | Glyphodes extorris Dognin, 1905                  |  5127576 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Glyphodes      | Glyphodes confinis          | Glyphodes extorris          | Dognin, 1905                 |    5127576 |        5.12756e+06 |
|            nan | Nemoria aturia                              | Nemoria aturia (Druce, 1892)                     |  1981238 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nemoria        | Nemoria aturia              | Nemoria aturia              | (Druce, 1892)                |    1981238 |      nan           |
|            nan | Sicya aurunca--medangula                    | Animalia                                         |        1 | HIGHERRANK  |           92 | ACCEPTED | KINGDOM | Animalia  | nan           | nan            | nan             | nan            | nan            | nan                         | Animalia                    | nan                          |          1 |      nan           |
|            nan | Hypena mactatalis                           | Hypena mactatalis Walker, 1859                   |  9319452 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hypena         | Hypena mactatalis           | Hypena mactatalis           | Walker, 1859                 |    9319452 |      nan           |
|            nan | Hypena parialis                             | Hypena perialis Schaus, 1904                     |  7740732 | FUZZY       |           93 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hypena         | Hypena perialis             | Hypena perialis             | Schaus, 1904                 |    7740732 |      nan           |
|            nan | Eois sp_san_luis_e                          | Eois Hübner, 1818                                |  1989610 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eois           | nan                         | Eois                        | Hübner, 1818                 |    1989610 |      nan           |
|            nan | Nemoria sp_san_luis_a                       | Nemoria Hübner, 1818                             |  1981142 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nemoria        | nan                         | Nemoria                     | Hübner, 1818                 |    1981142 |      nan           |
|            nan | Eusarca nemora                              | Eusarca nemora (Druce, 1892)                     | 10288719 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eusarca        | Eusarca nemora              | Eusarca nemora              | (Druce, 1892)                |   10288719 |      nan           |
|            nan | Prochoerodes striata                        | Prochoerodes striata (Stoll)                     |  9947436 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Prochoerodes   | Prochoerodes striata        | Prochoerodes striata        | (Stoll)                      |    9947436 |      nan           |
|            nan | Idalus crinis                               | Idalus crinis Druce, 1884                        |  1814711 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Idalus         | Idalus crinis               | Idalus crinis               | Druce, 1884                  |    1814711 |      nan           |
|            nan | Brabirodes cerevia                          | Brabirodes cerevia Druce, 1893                   |  1968653 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Brabirodes     | Brabirodes cerevia          | Brabirodes cerevia          | Druce, 1893                  |    1968653 |      nan           |
|            nan | Opisthoxia molpadia                         | Opisthoxia molpadia (Druce, 1892)                |  1960315 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Opisthoxia     | Opisthoxia molpadia         | Opisthoxia molpadia         | (Druce, 1892)                |    1960315 |      nan           |
|            nan | Eucereon centrale                           | Eucereon centrale Draudt, 1915                   |  1810309 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | Eucereon formosum           | Eucereon centrale           | Draudt, 1915                 |    1810309 |        1.81032e+06 |
|            nan | Hypena gaudialis                            | Hypena gaudialis Schaus, 1912                    | 10444039 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hypena         | Hypena gaudialis            | Hypena gaudialis            | Schaus, 1912                 |   10444039 |      nan           |
|            nan | Hemiceras pernubila                         | Hemiceras pernubila Dyar, 1908                   |  5118779 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Notodontidae   | Hemiceras      | Hemiceras pernubila         | Hemiceras pernubila         | Dyar, 1908                   |    5118779 |      nan           |
|            nan | Anomis                                      | Anomis Hübner                                    |  9188678 | EXACT       |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Anomis         | nan                         | Anomis                      | Hübner                       |    9188678 |      nan           |
|            nan | Prolimacodes montanus--triangulifera        | Prolimacodes Schaus, 1896                        |  1752820 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Limacodidae    | Prolimacodes   | nan                         | Prolimacodes                | Schaus, 1896                 |    1752820 |      nan           |
|            nan | Ptychamalia sp_san_luis_a                   | Ptychamalia Prout, 1932                          |  1963534 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Ptychamalia    | nan                         | Ptychamalia                 | Prout, 1932                  |    1963534 |      nan           |
|            nan | Pantherodes conglomerata                    | Pantherodes conglomerata (Warren, 1894)          |  1978957 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Pantherodes    | Pantherodes conglomerata    | Pantherodes conglomerata    | (Warren, 1894)               |    1978957 |      nan           |
|            nan | Tosale aucta                                | Tosale aucta Hampson, 1897                       |  1872515 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Pyralidae      | Tosale         | Tosale aucta                | Tosale aucta                | Hampson, 1897                |    1872515 |      nan           |
|            nan | Gonodonta holosericea                       | Gonodonta holosericea Guenée, 1852               |  1770032 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Gonodonta      | Gonodonta holosericea       | Gonodonta holosericea       | Guenée, 1852                 |    1770032 |      nan           |
|            nan | Halysidota orientalis                       | Halysidota orientalis Rothschild, 1909           |  1814330 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Halysidota     | Halysidota orientalis       | Halysidota orientalis       | Rothschild, 1909             |    1814330 |      nan           |
|            nan | Synnomos firmamentaria                      | Synnomos firmamentaria Guenée, 1857              |  1963633 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Synnomos       | Synnomos firmamentaria      | Synnomos firmamentaria      | Guenée, 1857                 |    1963633 |      nan           |
|            nan | Amaxia osmophora--pardalis                  | Amaxia Walker, 1855                              |  1814013 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Amaxia         | nan                         | Amaxia                      | Walker, 1855                 |    1814013 |      nan           |
|            nan | Trygodes merta                              | Trygodes merta Druce, 1892                       |  1976361 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Trygodes       | Trygodes solaniferata       | Trygodes merta              | Druce, 1892                  |    1976361 |        1.97637e+06 |
|            nan | Letis herilia                               | Letis herilia Stoll, 1780                        |  1788345 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Letis          | Letis herilia               | Letis herilia               | Stoll, 1780                  |    1788345 |      nan           |
|            nan | Eucereon tripunctatum                       | Eucereon tripunctatum Druce, 1884                |  1810357 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | Eucereon tripunctatum       | Eucereon tripunctatum       | Druce, 1884                  |    1810357 |      nan           |
|            nan | Cosmosoma centralis                         | Cosmosoma centralis Walker, 1854                 |  5116153 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Cosmosoma      | Cosmosoma centralis         | Cosmosoma centralis         | Walker, 1854                 |    5116153 |      nan           |
|            nan | Artace cribrarius                           | Artace cribrarius (Ljungh, 1825)                 |  1732901 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Lasiocampidae  | Artace         | Artace cribrarius           | Artace cribrarius           | (Ljungh, 1825)               |    1732901 |      nan           |
|            nan | Phyllodonta                                 | Phyllodonta Warren, 1894                         |  1963946 | EXACT       |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Phyllodonta    | nan                         | Phyllodonta                 | Warren, 1894                 |    1963946 |      nan           |
|            nan | Antiblemma harmodia                         | Antiblemma harmodia Schaus, 1901                 |  1794636 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Antiblemma     | Antiblemma harmodia         | Antiblemma harmodia         | Schaus, 1901                 |    1794636 |      nan           |
|            nan | Epitausa atriplaga                          | Epitausa atriplaga Walker, 1858                  |  1772219 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Epitausa       | Epitausa atriplaga          | Epitausa atriplaga          | Walker, 1858                 |    1772219 |      nan           |
|            nan | Glyphodes rubrocinctalis                    | Glyphodes rubrocinctalis (Guenée, 1854)          |  9499507 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Trithyris      | Prenesta rubrocinctalis     | Glyphodes rubrocinctalis    | (Guenée, 1854)               |    9499507 |        1.08613e+07 |
|            nan | Gizma undilinealis                          | Gizama undilinealis Schaus, 1916                 |  1765532 | FUZZY       |           85 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Gizama         | Gizama undilinealis         | Gizama undilinealis         | Schaus, 1916                 |    1765532 |      nan           |
|            nan | Sericoptera mahometaria                     | Sericoptera mahometaria (Herrich-Schäffer, 1853) |  1966088 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Sericoptera    | Sericoptera mahometaria     | Sericoptera mahometaria     | (Herrich-Schäffer, 1853)     |    1966088 |      nan           |
|            nan | Hemiceras punctata                          | Hemiceras punctata Dognin, 1889                  |  5118656 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Notodontidae   | Hemiceras      | Hemiceras punctata          | Hemiceras punctata          | Dognin, 1889                 |    5118656 |      nan           |
|            nan | Phrudocentra trimaculata                    | Phrudocentra trimaculata (Warren, 1901)          |  1985006 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Phrudocentra   | Phrudocentra trimaculata    | Phrudocentra trimaculata    | (Warren, 1901)               |    1985006 |      nan           |
|            nan | Altinote ozomene                            | Altinote ozomene                                 | 10487080 | EXACT       |           97 | DOUBTFUL | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Nymphalidae    | Acraea         | Altinote ozomene            | Altinote ozomene            | nan                          |   10487080 |      nan           |
|            nan | Phyle cartago                               | Phyle cartago Rindge, 1990                       |  1964270 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Phyle          | Phyle cartago               | Phyle cartago               | Rindge, 1990                 |    1964270 |      nan           |
|            nan | Eusarca crameraria--green                   | Eusarca Hübner                                   |  7514692 | HIGHERRANK  |           92 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eusarca        | nan                         | Eusarca                     | Hübner                       |    7514692 |      nan           |
|            nan | Melese amastris                             | Melese amastris Druce, 1884                      |  1817562 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Melese         | Melese amastris             | Melese amastris             | Druce, 1884                  |    1817562 |      nan           |
|            nan | Phyllodonta indeterminata                   | Phyllodonta indeterminata Schaus, 1901           |  1963980 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Phyllodonta    | Phyllodonta indeterminata   | Phyllodonta indeterminata   | Schaus, 1901                 |    1963980 |      nan           |
|            nan | Scaptius vinasia                            | Scaptius vinasia Schaus, 1910                    |  1805891 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Scaptius       | Scaptius vinasia            | Scaptius vinasia            | Schaus, 1910                 |    1805891 |      nan           |
|            nan | Microxydia sp_san_luis_a                    | Microxydia Warren, 1895                          |  1954241 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Microxydia     | nan                         | Microxydia                  | Warren, 1895                 |    1954241 |      nan           |
|            nan | Crinodes besckei                            | Crinodes besckei Hübner, 1824                    |  1824179 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Notodontidae   | Crinodes       | Crinodes besckei            | Crinodes besckei            | Hübner, 1824                 |    1824179 |      nan           |
|            nan | Argyria lacteella                           | Argyria lacteella Fabricius, 1794                |  5126721 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Argyria        | Argyria lacteella           | Argyria lacteella           | Fabricius, 1794              |    5126721 |      nan           |
|            nan | Eucereon sp_san_luis_c                      | Eucereon Hübner, 1819                            |  1810140 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | nan                         | Eucereon                    | Hübner, 1819                 |    1810140 |      nan           |
|            nan | Racasta spatiaria                           | Racasta spatiaria (Guenée)                       |  1988833 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Racasta        | Racasta spatiaria           | Racasta spatiaria           | (Guenée)                     |    1988833 |      nan           |
|            nan | Cosmosoma pudica                            | Cosmosoma pudica Druce, 1894                     |  5116247 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Cosmosoma      | Cosmosoma pudica            | Cosmosoma pudica            | Druce, 1894                  |    5116247 |      nan           |
|            nan | Macaria regulata                            | Macaria regulata (Fabricius, 1775)               | 10127068 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Macaria        | Macaria regulata            | Macaria regulata            | (Fabricius, 1775)            |   10127068 |      nan           |
|            nan | Phrudocentra janeira                        | Phrudocentra janeira (Schaus, 1897)              |  1985018 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Phrudocentra   | Phrudocentra janeira        | Phrudocentra janeira        | (Schaus, 1897)               |    1985018 |      nan           |
|            nan | Pyralidae Acrobasis                         | Pyralidae                                        |     5336 | EXACT       |           94 | ACCEPTED | FAMILY  | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Pyralidae      | nan            | nan                         | Pyralidae                   | nan                          |       5336 |      nan           |
|            nan | Podalia contigua                            | Podalia contigua (Walker, 1866)                  |  5105187 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Megalopygidae  | Podalia        | Podalia contigua            | Podalia contigua            | (Walker, 1866)               |    5105187 |      nan           |
|            nan | Therinia transversaria                      | Therinia transversaria Druce, 1887               | 10103570 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Saturniidae    | Therinia       | Therinia transversaria      | Therinia transversaria      | Druce, 1887                  |   10103570 |      nan           |
|            nan | Oxydia translinquens                        | Oxydia translinquens (Walker, 1860)              |  1976653 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Oxydia         | Oxydia translinquens        | Oxydia translinquens        | (Walker, 1860)               |    1976653 |      nan           |
|            nan | Spragueia apicalis                          | Spragueia apicalis (Herrich-Schäffer, 1868)      |  7911151 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Spragueia      | Spragueia apicalis          | Spragueia apicalis          | (Herrich-Schäffer, 1868)     |    7911151 |      nan           |
|            nan | Hypercompe albescens                        | Hypercompe albescens Hampson, 1901               |  1811100 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hypercompe     | Hypercompe albescens        | Hypercompe albescens        | Hampson, 1901                |    1811100 |      nan           |
|            nan | Rhescyntis hippodamia                       | Rhescyntis hippodamia Cramer, 1777               |  1865837 | EXACT       |           98 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Saturniidae    | Rhescyntis     | Rhescyntis hippodamia       | Rhescyntis hippodamia       | Cramer, 1777                 |    1865837 |      nan           |
|            nan | Sericoptera reductata                       | Sericoptera reducta Warren, 1909                 |  1966085 | FUZZY       |           92 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Sericoptera    | Sericoptera area            | Sericoptera reducta         | Warren, 1909                 |    1966085 |        1.96607e+06 |
|            nan | Argyrotome prospectata                      | Argyrotome prospectata (Snellen, 1874)           |  1967346 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Argyrotome     | Argyrotome prospectata      | Argyrotome prospectata      | (Snellen, 1874)              |    1967346 |      nan           |
|            nan | Crinodes ritsemae                           | Crinodes ritsemae Druce, 1898                    |  1824191 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Notodontidae   | Crinodes       | Crinodes besckei            | Crinodes ritsemae           | Druce, 1898                  |    1824191 |        1.82418e+06 |
|            nan | Polla hemeraria                             | Polla hemeraria Dyar, 1910                       |  5144043 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Polla          | Polla hemeraria             | Polla hemeraria             | Dyar, 1910                   |    5144043 |      nan           |
|            nan | Cosmosoma stibostictum                      | Cosmosoma stibosticta Butler, 1876               |  5116208 | FUZZY       |           95 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Cosmosoma      | Cosmosoma stibosticta       | Cosmosoma stibosticta       | Butler, 1876                 |    5116208 |      nan           |
|            nan | Thysanoprymna haemorrhoides                 | Thysanoprymna haemorrhoides Schaus, 1905         |  1815297 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Thysanoprymna  | Thysanoprymna haemorrhoides | Thysanoprymna haemorrhoides | Schaus, 1905                 |    1815297 |      nan           |
|            nan | Macrodes cynara                             | Macrodes cynara Cramer, 1775                     |  1777723 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Macrodes       | Macrodes cynara             | Macrodes cynara             | Cramer, 1775                 |    1777723 |      nan           |
|            nan | Hypena livia                                | Hypena livia Druce, 1890                         | 11540684 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hypena         | Hypena livia                | Hypena livia                | Druce, 1890                  |   11540684 |      nan           |
|            nan | Ormetica maura--triangularis                | Ormetica Clemens, 1861                           |  1803615 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Ormetica       | nan                         | Ormetica                    | Clemens, 1861                |    1803615 |      nan           |
|            nan | Dysschema magdala                           | Dysschema magdala Boisduval, 1870                |  5117551 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Dysschema      | Dysschema magdala           | Dysschema magdala           | Boisduval, 1870              |    5117551 |      nan           |
|            nan | Ormetica taeniata--temperata                | Ormetica Clemens, 1861                           |  1803615 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Ormetica       | nan                         | Ormetica                    | Clemens, 1861                |    1803615 |      nan           |
|            nan | Trauaxa lua                                 | Trauaxa lua Druce, 1890                          |  1788533 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Trauaxa        | Trauaxa lua                 | Trauaxa lua                 | Druce, 1890                  |    1788533 |      nan           |
|            nan | Meganola                                    | Meganola Dyar, 1898                              |  4405493 | EXACT       |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Nolidae        | Meganola       | nan                         | Meganola                    | Dyar, 1898                   |    4405493 |      nan           |
|            nan | Syntomeida melanthus                        | Syntomeida melanthus Cramer                      |  1809127 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Syntomeida     | Syntomeida melanthus        | Syntomeida melanthus        | Cramer                       |    1809127 |      nan           |
|            nan | Phalaenophana pyramusalis Dark-banded Owlet | Phalaenophana pyramusalis (Walker, 1859)         |  9342938 | EXACT       |           96 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Phaeolita      | Phaeolita pyramusalis       | Phalaenophana pyramusalis   | (Walker, 1859)               |    9342938 |        1.79083e+06 |
|            nan | Lepidodes gallopavo                         | Lepidodes gallopavo Druce                        |  5110889 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Lepidodes      | Lepidodes limbulata         | Lepidodes gallopavo         | Druce                        |    5110889 |        5.11089e+06 |
|            nan | Pantographa expansalis                      | Pantographa Lederer, 1863                        |  1884683 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Pantographa    | nan                         | Pantographa                 | Lederer, 1863                |    1884683 |      nan           |
|            nan | Sematura luna                               | Sematura lunus (Linnaeus, 1758)                  |  9527128 | FUZZY       |           96 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sematuridae    | Nothus         | Nothus lunus                | Sematura lunus              | (Linnaeus, 1758)             |    9527128 |        5.14328e+06 |
|            nan | Schrankia macula                            | Schrankia macula Druce, 1891                     |  9522383 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Schrankia      | Schrankia macula            | Schrankia macula            | Druce, 1891                  |    9522383 |      nan           |
|            nan | Polypoetes villia                           | Polypoetes villia Druce, 1897                    |  1824722 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Notodontidae   | Polypoetes     | Polypoetes villia           | Polypoetes villia           | Druce, 1897                  |    1824722 |      nan           |
|            nan | Urania fulgens                              | Urania fulgens Walker, 1854                      |  5143232 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Uraniidae      | Urania         | Urania fulgens              | Urania fulgens              | Walker, 1854                 |    5143232 |      nan           |
|            nan | Opisthoxia uncinata                         | Opisthoxia uncinata (Schaus, 1912)               |  1960250 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Opisthoxia     | Opisthoxia uncinata         | Opisthoxia uncinata         | (Schaus, 1912)               |    1960250 |      nan           |
|            nan | Castrica phalaenoides                       | Castrica phalaenoides Drury, 1773                |  1809033 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Castrica       | Castrica phalaenoides       | Castrica phalaenoides       | Drury, 1773                  |    1809033 |      nan           |
|            nan | Hyalorista sp_san_luis_a                    | Hyalorista Warren, 1892                          |  1891030 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Hyalorista     | nan                         | Hyalorista                  | Warren, 1892                 |    1891030 |      nan           |
|            nan | Pseudophaloe troetschi                      | Pseudophaloe troetschi Druce, 1884               |  1809200 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Pseudophaloe   | Pseudophaloe troetschi      | Pseudophaloe troetschi      | Druce, 1884                  |    1809200 |      nan           |
|            nan | Hypena hemonalis                            | Hypena hemonalis Walker, 1859                    |  9436584 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hypena         | Hypena hemonalis            | Hypena hemonalis            | Walker, 1859                 |    9436584 |      nan           |
|            nan | Melese asana                                | Melese asana Druce, 1884                         |  1817601 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Melese         | Melese asana                | Melese asana                | Druce, 1884                  |    1817601 |      nan           |
|            nan | Eucereon aeolum                             | Eucereon aeolum Hampson, 1898                    |  1810348 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | Eucereon aeolum             | Eucereon aeolum             | Hampson, 1898                |    1810348 |      nan           |
|            nan | Nematocampa straminea                       | Nematocampa straminea (Warren, 1900)             |  1991021 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nematocampa    | Nematocampa straminea       | Nematocampa straminea       | (Warren, 1900)               |    1991021 |      nan           |
|            nan | Isochromodes granula                        | Isochromodes granula (Dognin, 1896)              |  1975658 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Isochromodes   | Isochromodes granula        | Isochromodes granula        | (Dognin, 1896)               |    1975658 |      nan           |
|            nan | Eucereon sp_san_luis_a                      | Eucereon Hübner, 1819                            |  1810140 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | nan                         | Eucereon                    | Hübner, 1819                 |    1810140 |      nan           |
|            nan | micro                                       | Micro González-Sponga, 1987                      |  9574074 | EXACT       |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Arachnida      | Opiliones       | Zalmoxidae     | Micro          | nan                         | Micro                       | González-Sponga, 1987        |    9574074 |      nan           |
|            nan | Morpho peleides                             | Morpho peleides Kollar, 1850                     |  5133433 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Nymphalidae    | Morpho         | Morpho peleides             | Morpho peleides             | Kollar, 1850                 |    5133433 |      nan           |
|            nan | Tornos erectarius                           | Tornos erectarius Grossbeck, 1909                |  1975391 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Tornos         | Tornos erectarius           | Tornos erectarius           | Grossbeck, 1909              |    1975391 |      nan           |
|            nan | Bagisara patula                             | Bagisara patula Druce, 1898                      |  1767587 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Bagisara       | Bagisara patula             | Bagisara patula             | Druce, 1898                  |    1767587 |      nan           |
|            nan | Dysschema leucophaea                        | Dysschema leucophaea Walker, 1854                |  5117462 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Dysschema      | Dysschema leucophaea        | Dysschema leucophaea        | Walker, 1854                 |    5117462 |      nan           |
|            nan | Cyphoedma transvolutata                     | Cyphoedma transvolutata (Walker, 1860)           |  1986882 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Cyphoedma      | Cyphoedma transvolutata     | Cyphoedma transvolutata     | (Walker, 1860)               |    1986882 |      nan           |
|            nan | Viviennea ardesiaca                         | Viviennea ardesiaca Rothschild, 1909             |  1807998 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Viviennea      | Viviennea ardesiaca         | Viviennea ardesiaca         | Rothschild, 1909             |    1807998 |      nan           |
|            nan | Perigea hippia--micrippia                   | Perigea Guenée, 1852                             |  1790726 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Perigea        | nan                         | Perigea                     | Guenée, 1852                 |    1790726 |      nan           |
|            nan | Gonodonta pyrgo                             | Gonodonta pyrgo Cramer                           |  1770044 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Gonodonta      | Gonodonta pyrgo             | Gonodonta pyrgo             | Cramer                       |    1770044 |      nan           |
|            nan | Cathydata batina                            | Cathydata batina Druce, 1892                     |  1964698 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Cathydata      | Cathydata batina            | Cathydata batina            | Druce, 1892                  |    1964698 |      nan           |
|            nan | Lophocampa modesta                          | Lophocampa modesta Kirby, 1892                   |  1817774 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Lophocampa     | Lophocampa modesta          | Lophocampa modesta          | Kirby, 1892                  |    1817774 |      nan           |
|            nan | Eulepidotis                                 | Eulepidotis Hübner, 1823                         |  1763151 | EXACT       |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eulepidotis    | nan                         | Eulepidotis                 | Hübner, 1823                 |    1763151 |      nan           |
|            nan | Bertholdia albipuncta                       | Bertholdia albipuncta Schaus, 1896               |  5115488 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Bertholdia     | Bertholdia albipuncta       | Bertholdia albipuncta       | Schaus, 1896                 |    5115488 |      nan           |
|            nan | Isanthrene monticola                        | Isanthrene monticola Schaus, 1911                |  1811196 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Isanthrene     | Isanthrene monticola        | Isanthrene monticola        | Schaus, 1911                 |    1811196 |      nan           |
|            nan | Carales arizonensis--astur                  | Carales Walker, 1855                             |  1809292 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Carales        | nan                         | Carales                     | Walker, 1855                 |    1809292 |      nan           |
|            nan | Eucereon patrona                            | Eucereon patrona Schaus, 1896                    |  1810274 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | Eucereon patrona            | Eucereon patrona            | Schaus, 1896                 |    1810274 |      nan           |
|            nan | Opisthoxia phrynearia                       | Opisthoxia phrynearia (Schaus, 1912)             |  1960279 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Opisthoxia     | Opisthoxia phrynearia       | Opisthoxia phrynearia       | (Schaus, 1912)               |    1960279 |      nan           |
|            nan | Eusarca melenda                             | Eusarca melenda (Druce, 1892)                    | 10278975 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eusarca        | Eusarca melenda             | Eusarca melenda             | (Druce, 1892)                |   10278975 |      nan           |
|            nan | Semiothisa valmonaria                       | Semiothisa valmonaria Schaus, 1901               |  1958935 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Semiothisa     | Semiothisa valmonaria       | Semiothisa valmonaria       | Schaus, 1901                 |    1958935 |      nan           |
|            nan | Herminodes subclara                         | Herminodes Guenée, 1852                          |  4696204 | HIGHERRANK  |           94 | SYNONYM  | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Scolecocampa   | nan                         | Herminodes                  | Guenée, 1852                 |    4696204 |        1.76518e+06 |
|            nan | Ascalapha odorata                           | Ascalapha odorata (Linnaeus, 1758)               |  1775796 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Ascalapha      | Ascalapha odorata           | Ascalapha odorata           | (Linnaeus, 1758)             |    1775796 |      nan           |
|            nan | Renia discoloralis                          | Renia discoloralis Guenée, 1854                  |  5112060 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Renia          | Renia discoloralis          | Renia discoloralis          | Guenée, 1854                 |    5112060 |      nan           |
|            nan | Spodoptera dolichos                         | Spodoptera dolichos Fabricius, 1794              |  5109881 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Spodoptera     | Spodoptera dolichos         | Spodoptera dolichos         | Fabricius, 1794              |    5109881 |      nan           |
|            nan | Hemeroblemma opigena                        | Hemeroblemma opigena Drury, 1773                 |  1786219 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hemeroblemma   | Hemeroblemma opigena        | Hemeroblemma opigena        | Drury, 1773                  |    1786219 |      nan           |
|            nan | Argyrotome alba                             | Argyrotome alba (Druce, 1892)                    |  1967353 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Argyrotome     | Argyrotome alba             | Argyrotome alba             | (Druce, 1892)                |    1967353 |      nan           |
|            nan | Eois isographata                            | Eois isographata Walker                          |  1989782 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eois           | Eois isographata            | Eois isographata            | Walker                       |    1989782 |      nan           |
|            nan | Cosmosoma impudica                          | Cosmosoma impudica Schaus, 1911                  |  5116151 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Cosmosoma      | Cosmosoma impudica          | Cosmosoma impudica          | Schaus, 1911                 |    5116151 |      nan           |
|            nan | Hypercompe caudata                          | Hypercompe caudata Walker, 1855                  |  1811019 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hypercompe     | Hypercompe caudata          | Hypercompe caudata          | Walker, 1855                 |    1811019 |      nan           |
|            nan | Symphlebia ipsea                            | Symphlebia ipsea Druce, 1884                     |  1805261 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Symphlebia     | Symphlebia ipsea            | Symphlebia ipsea            | Druce, 1884                  |    1805261 |      nan           |
|            nan | Neonerita dorsipuncta                       | Neonerita dorsipuncta Hampson, 1901              |  1805478 | EXACT       |           98 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Neonerita      | Neonerita dorsipuncta       | Neonerita dorsipuncta       | Hampson, 1901                |    1805478 |      nan           |
|            nan | Hypocrita reedia                            | Hypocrita reedia Schaus, 1910                    |  1805395 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hypocrita      | Hypocrita reedia            | Hypocrita reedia            | Schaus, 1910                 |    1805395 |      nan           |
|            nan | Pseudasellodes fenestraria                  | Pseudasellodes fenestraria (Guenée)              |  1957051 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Pseudasellodes | Pseudasellodes fenestraria  | Pseudasellodes fenestraria  | (Guenée)                     |    1957051 |      nan           |
|            nan | Renia vinasalis                             | Renia vinasalis Schaus, 1912                     |  5112092 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Renia          | Renia vinasalis             | Renia vinasalis             | Schaus, 1912                 |    5112092 |      nan           |
|            nan | Celiptera levina                            | Celiptera levina Stoll, 1781                     |  1792827 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Celiptera      | Celiptera levina            | Celiptera levina            | Stoll, 1781                  |    1792827 |      nan           |
|            nan | Phrygionis polita                           | Phrygionis polita (Cramer)                       |  1968313 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Phrygionis     | Phrygionis polita           | Phrygionis polita           | (Cramer)                     |    1968313 |      nan           |
|            nan | Bernathonomus piperita                      | Bernathonomus piperita Herrich-Schäffer, 1855    |  1817532 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Bernathonomus  | Bernathonomus piperita      | Bernathonomus piperita      | Herrich-Schäffer, 1855       |    1817532 |      nan           |
|            nan | Herminodes partina                          | Herminodes Guenée, 1852                          |  4696204 | HIGHERRANK  |           94 | SYNONYM  | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Scolecocampa   | nan                         | Herminodes                  | Guenée, 1852                 |    4696204 |        1.76518e+06 |
|            nan | Bagisara repanda                            | Bagisara repanda Fabricius, 1793                 |  1767571 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Bagisara       | Bagisara repanda            | Bagisara repanda            | Fabricius, 1793              |    1767571 |      nan           |
|            nan | Mesothen temperata                          | Mesothen temperata Schaus, 1911                  |  1812543 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Mesothen       | Mesothen temperata          | Mesothen temperata          | Schaus, 1911                 |    1812543 |      nan           |
|            nan | Macaria pernicata                           | Macaria pernicata Guenée                         | 10325227 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Macaria        | Macaria pernicata           | Macaria pernicata           | Guenée                       |   10325227 |      nan           |
|            nan | Perasia garnoti                             | Perasia garnoti Guenée, 1852                     |  1787464 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Perasia        | Perasia garnoti             | Perasia garnoti             | Guenée, 1852                 |    1787464 |      nan           |
|            nan | Sennia auritalis                            | Sennia auritalis                                 | 10251952 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Pyralidae      | Sennia         | Sennia auritalis            | Sennia auritalis            | nan                          |   10251952 |      nan           |
|            nan | Notarctia proxima                           | Notarctia proxima Guérin-Méneville, 1844         |  1815414 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Notarctia      | Notarctia proxima           | Notarctia proxima           | Guérin-Méneville, 1844       |    1815414 |      nan           |
|            nan | Ormetica sp_san_luis_a                      | Ormetica Clemens, 1861                           |  1803615 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Ormetica       | nan                         | Ormetica                    | Clemens, 1861                |    1803615 |      nan           |
|            nan | Massala hieroglyphica                       | Massala hieroglyphica Walker, 1867               |  1796606 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Massala        | Massala hieroglyphica       | Massala hieroglyphica       | Walker, 1867                 |    1796606 |      nan           |
|            nan | Nesara casada                               | Nesara casada (Schaus, 1911)                     |  1733038 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Lasiocampidae  | Nesara         | Nesara casada               | Nesara casada               | (Schaus, 1911)               |    1733038 |      nan           |
|            nan | Eucereon decora                             | Eucereon Hübner, 1819                            |  1810140 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | nan                         | Eucereon                    | Hübner, 1819                 |    1810140 |      nan           |
|            nan | Elaphria deltoides                          | Elaphria deltoides Möschler, 1880                |  1780440 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Elaphria       | Elaphria deltoides          | Elaphria deltoides          | Möschler, 1880               |    1780440 |      nan           |
|            nan | Micranthops alceste                         | Micranthops alceste Druce, 1890                  |  1793681 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Micranthops    | Micranthops alceste         | Micranthops alceste         | Druce, 1890                  |    1793681 |      nan           |
|            nan | Lesmone formularis                          | Lesmone formularis Geyer, 1837                   |  8331687 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Lesmone        | Lesmone formularis          | Lesmone formularis          | Geyer, 1837                  |    8331687 |      nan           |
|            nan | Stenoma luctifica                           | Stenoma luctifica Zeller, 1877                   |  1836730 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Depressariidae | Stenoma        | Stenoma luctifica           | Stenoma luctifica           | Zeller, 1877                 |    1836730 |      nan           |
|            nan | Mimosema consociata                         | Mimosema consociata (Schaus, 1911)               |  1960218 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Mimosema       | Mimosema consociata         | Mimosema consociata         | (Schaus, 1911)               |    1960218 |      nan           |
|            nan | Scopula compensata                          | Scopula compensata (Walker, 1861)                |  5145466 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Scopula        | Scopula compensata          | Scopula compensata          | (Walker, 1861)               |    5145466 |      nan           |
|            nan | Eucereon                                    | Eucereon Hübner, 1819                            |  1810140 | EXACT       |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | nan                         | Eucereon                    | Hübner, 1819                 |    1810140 |      nan           |
|            nan | Oospila albicoma                            | Oospila albicoma Felder & Rogenhofer, 1875       |  1973687 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Oospila        | Oospila albicoma            | Oospila albicoma            | Felder & Rogenhofer, 1875    |    1973687 |      nan           |
|            nan | Glenopteris oculifera                       | Glenopteris oculifera Hübner, 1821               |  5110251 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Glenopteris    | Glenopteris oculifera       | Glenopteris oculifera       | Hübner, 1821                 |    5110251 |      nan           |
|            nan | Gerra sevorsa                               | Gerra sevorsa Grote, 1882                        |  1761143 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Gerra          | Gerra sevorsa               | Gerra sevorsa               | Grote, 1882                  |    1761143 |      nan           |
|            nan | Xylophanes ceratomioides                    | Xylophanes ceratomioides Grote & Robinson, 1866  |  1862087 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sphingidae     | Xylophanes     | Xylophanes ceratomioides    | Xylophanes ceratomioides    | Grote & Robinson, 1866       |    1862087 |      nan           |
|            nan | Cosmosoma impar                             | Cosmosoma impar Walker, 1854                     |  5116195 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Cosmosoma      | Cosmosoma impar             | Cosmosoma impar             | Walker, 1854                 |    5116195 |      nan           |
|            nan | Perasia helvina                             | Perasia helvina Guenée, 1852                     |  1787453 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Perasia        | Perasia helvina             | Perasia helvina             | Guenée, 1852                 |    1787453 |      nan           |
|            nan | Gonodonta sicheas                           | Gonodonta sicheas Cramer                         |  1770069 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Gonodonta      | Gonodonta sicheas           | Gonodonta sicheas           | Cramer                       |    1770069 |      nan           |
|            nan | Eois sp_san_luis_k                          | Eois Hübner, 1818                                |  1989610 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eois           | nan                         | Eois                        | Hübner, 1818                 |    1989610 |      nan           |
|            nan | Quentalia surynorta                         | Quentalia surynorta Schaus, 1900                 |  1868225 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Bombycidae     | Quentalia      | Quentalia surynorta         | Quentalia surynorta         | Schaus, 1900                 |    1868225 |      nan           |
|            nan | Euglyphis libnites                          | Euglyphis libnites (Druce, 1897)                 |  5102203 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Lasiocampidae  | Euglyphis      | Euglyphis libnites          | Euglyphis libnites          | (Druce, 1897)                |    5102203 |      nan           |
|            nan | Tricentrogyna vinacea                       | Tricentrogyna vinacea (Butler, 1878)             |  1963239 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Tricentrogyna  | Tricentrogyna vinacea       | Tricentrogyna vinacea       | (Butler, 1878)               |    1963239 |      nan           |
|            nan | Nyceryx ericea                              | Nyceryx ericea Druce, 1888                       |  1863142 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sphingidae     | Nyceryx        | Nyceryx ericea              | Nyceryx ericea              | Druce, 1888                  |    1863142 |      nan           |
|            nan | Xylophanes anubus                           | Xylophanes anubus Cramer, 1777                   |  1862078 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sphingidae     | Xylophanes     | Xylophanes anubus           | Xylophanes anubus           | Cramer, 1777                 |    1862078 |      nan           |
|            nan | Meganola sp_san_luis_a                      | Meganola Dyar, 1898                              |  4405493 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Nolidae        | Meganola       | nan                         | Meganola                    | Dyar, 1898                   |    4405493 |      nan           |
|            nan | Spodoptera androgea                         | Spodoptera androgea Stoll                        |  5109923 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Spodoptera     | Spodoptera androgea         | Spodoptera androgea         | Stoll                        |    5109923 |      nan           |
|            nan | Eois xanthoperata                           | Eois xanthoperata (Warren, 1897)                 |  1989833 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eois           | Eois xanthoperata           | Eois xanthoperata           | (Warren, 1897)               |    1989833 |      nan           |
|            nan | Eois sp san luis b                          | Eois Hübner, 1818                                |  1989610 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eois           | nan                         | Eois                        | Hübner, 1818                 |    1989610 |      nan           |
|            nan | Xylophanes turbata                          | Xylophanes turbata Edwards, 1887                 |  1862218 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sphingidae     | Xylophanes     | Xylophanes turbata          | Xylophanes turbata          | Edwards, 1887                |    1862218 |      nan           |
|            nan | Phrygionis platinata                        | Phrygionis platinata (Guenée)                    |  1968321 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Phrygionis     | Phrygionis platinata        | Phrygionis platinata        | (Guenée)                     |    1968321 |      nan           |
|            nan | Bromeliaceae                                | Bromeliaceae                                     |     3740 | EXACT       |           96 | ACCEPTED | FAMILY  | Plantae   | Tracheophyta  | Liliopsida     | Poales          | Bromeliaceae   | nan            | nan                         | Bromeliaceae                | nan                          |       3740 |      nan           |
|            nan | Epimecis subroraria                         | Epimecis subroraria (Walker, 1860)               |  5149257 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Epimecis       | Epimecis subroraria         | Epimecis subroraria         | (Walker, 1860)               |    5149257 |      nan           |
|            nan | Anticarsia gemmatalis                       | Anticarsia gemmatalis Hübner, 1818               |  1777942 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Anticarsia     | Anticarsia gemmatalis       | Anticarsia gemmatalis       | Hübner, 1818                 |    1777942 |      nan           |
|            nan | Gonodonta sinaldus                          | Gonodonta sinaldus Guenée, 1852                  |  1770038 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Gonodonta      | Gonodonta sinaldus          | Gonodonta sinaldus          | Guenée, 1852                 |    1770038 |      nan           |
|            nan | Cosmosoma cingulatum                        | Cosmosoma cingulatum Butler, 1876                |  5116305 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Cosmosoma      | Cosmosoma teuthras          | Cosmosoma cingulatum        | Butler, 1876                 |    5116305 |        5.11616e+06 |
|            nan | Ammalo paranomon                            | Ammalo paranomon Dyar, 1912                      |  1814180 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Ammalo         | Ammalo paranomon            | Ammalo paranomon            | Dyar, 1912                   |    1814180 |      nan           |
|            nan | Phrygionis privignaria                      | Phrygionis privignaria (Guenée)                  |  1968324 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Phrygionis     | Phrygionis privignaria      | Phrygionis privignaria      | (Guenée)                     |    1968324 |      nan           |
|            nan | Melinodes detersaria                        | Melinodes detersaria Herrich-Schäffer            |  9979538 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Melinoides     | Melinoides detersaria       | Melinodes detersaria        | Herrich-Schäffer             |    9979538 |        1.98548e+06 |
|            nan | Hyperthaema orbicularis                     | Hyperthaema orbicularis Maassen, 1890            |  1817839 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Hyperthaema    | Hyperthaema orbicularis     | Hyperthaema orbicularis     | Maassen, 1890                |    1817839 |      nan           |
|            nan | Condica mimica                              | Condica mimica Hampson, 1908                     |  7617061 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Condica        | Condica mimica              | Condica mimica              | Hampson, 1908                |    7617061 |      nan           |
|            nan | Acrosemia vulpecularia                      | Acrosemia vulpecularia Herrich-Schäffer, 1855    |  1960090 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Acrosemia      | Acrosemia vulpecularia      | Acrosemia vulpecularia      | Herrich-Schäffer, 1855       |    1960090 |      nan           |
|            nan | Eulepidotis guttata--ouocco                 | Eulepidotis Hübner, 1823                         |  1763151 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eulepidotis    | nan                         | Eulepidotis                 | Hübner, 1823                 |    1763151 |      nan           |
|            nan | Hemiceras rufescens                         | Hemiceras rufescens Walker, 1965                 |  5118760 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Notodontidae   | Hemiceras      | Hemiceras rufescens         | Hemiceras rufescens         | Walker, 1965                 |    5118760 |      nan           |
|            nan | Mesocia dumilla                             | Mesoscia dumilla Dyar, 1913                      |  1751739 | FUZZY       |           85 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Megalopygidae  | Mesoscia       | Mesoscia dumilla            | Mesoscia dumilla            | Dyar, 1913                   |    1751739 |      nan           |
|            nan | Coronidia orithea                           | Coronidia orithea Cramer, 1780                   |  1954146 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sematuridae    | Coronidia      | Coronidia orithea           | Coronidia orithea           | Cramer, 1780                 |    1954146 |      nan           |
|            nan | Episimus transferranus                      | Episimus transferrana (Walker, 1863)             |  1741397 | FUZZY       |           96 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Tortricidae    | Episimus       | Episimus transferrana       | Episimus transferrana       | (Walker, 1863)               |    1741397 |      nan           |
|            nan | Orsa erythrospila                           | Orsa erythrospila Walker, 1865                   |  5113949 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Orsa           | Orsa erythrospila           | Orsa erythrospila           | Walker, 1865                 |    5113949 |      nan           |
|            nan | Rothschildia orizaba                        | Rothschildia orizaba Westwood, 1854              |  5806244 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Saturniidae    | Rothschildia   | Rothschildia orizaba        | Rothschildia orizaba        | Westwood, 1854               |    5806244 |      nan           |
|            nan | Mursa imitatrix                             | Mursa Walker, 1859                               |  1789812 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Mursa          | nan                         | Mursa                       | Walker, 1859                 |    1789812 |      nan           |
|            nan | Calymniodes acamas--conchylis--pyrostorta   | Calymniodes Dognin, 1907                         |  1792759 | HIGHERRANK  |           92 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Calymniodes    | nan                         | Calymniodes                 | Dognin, 1907                 |    1792759 |      nan           |
|            nan | archaeognatha                               | Archaeognatha                                    |     1187 | EXACT       |           95 | ACCEPTED | ORDER   | Animalia  | Arthropoda    | Insecta        | Archaeognatha   | nan            | nan            | nan                         | Archaeognatha               | nan                          |       1187 |      nan           |
|            nan | Glyphodes sibillalis                        | Glyphodes sibillalis Walker, 1859                |  5127631 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Glyphodes      | Glyphodes sibillalis        | Glyphodes sibillalis        | Walker, 1859                 |    5127631 |      nan           |
|            nan | Anomis rubida                               | Anomis rubida Schaus, 1911                       |  1784930 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Anomis         | Anomis rubida               | Anomis rubida               | Schaus, 1911                 |    1784930 |      nan           |
|            nan | Bleptina caradrinalis                       | Bleptina caradrinalis Guenée, 1854               |  1793538 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Bleptina       | Bleptina caradrinalis       | Bleptina caradrinalis       | Guenée, 1854                 |    1793538 |      nan           |
|            nan | Cosmosoma festivum                          | Cosmosoma festivum Walker, 1854                  |  5116274 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Cosmosoma      | Cosmosoma festivum          | Cosmosoma festivum          | Walker, 1854                 |    5116274 |      nan           |
|            nan | Diaphania nitidalis                         | Diaphania nitidalis (Stoll, 1781)                |  7598372 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Cryptographis  | Cryptographis nitidalis     | Diaphania nitidalis         | (Stoll, 1781)                |    7598372 |        1.88612e+06 |
|            nan | Xylophanes libya                            | Xylophanes libya Druce, 1878                     |  1862187 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sphingidae     | Xylophanes     | Xylophanes libya            | Xylophanes libya            | Druce, 1878                  |    1862187 |      nan           |
|            nan | Morphomima fulvivacta                       | Morphomima fulvivacta                            | 11910725 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Uraniidae      | Morphomima     | Morphomima fulvivacta       | Morphomima fulvivacta       | nan                          |   11910725 |      nan           |
|            nan | Aellopos titan                              | Aellopos titan Cramer, 1777                      |  5124450 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sphingidae     | Aellopos       | Aellopos titan              | Aellopos titan              | Cramer, 1777                 |    5124450 |      nan           |
|            nan | Cliniodes nomadalis                         | Cliniodes nomadalis Dyar, 1912                   |  1891286 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Cliniodes      | Cliniodes cyllarusalis      | Cliniodes nomadalis         | Dyar, 1912                   |    1891286 |        1.89128e+06 |
|            nan | pupa chrysalis                              | Pupa chrysalis Férussac, 1832                    | 11331817 | EXACT       |           97 | SYNONYM  | SPECIES | Animalia  | Mollusca      | Gastropoda     | Stylommatophora | Cerionidae     | Cerion         | Cerion mumia                | Pupa chrysalis              | Férussac, 1832               |   11331817 |        1.13773e+07 |
|            nan | Lophocampa annulosa                         | Lophocampa annulosa Walker, 1855                 |  1817734 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Lophocampa     | Lophocampa annulosa         | Lophocampa annulosa         | Walker, 1855                 |    1817734 |      nan           |
|            nan | Isanthrene felderi                          | Isanthrene felderi Druce, 1883                   |  1811202 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Isanthrene     | Isanthrene felderi          | Isanthrene felderi          | Druce, 1883                  |    1811202 |      nan           |
|            nan | Cimicodes albicosta                         | Cimicodes albicosta Dognin, 1914                 |  1957131 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Cimicodes      | Cimicodes albicosta         | Cimicodes albicosta         | Dognin, 1914                 |    1957131 |      nan           |
|            nan | Lampruna rosea                              | Lampruna rosea Schaus, 1894                      |  1816605 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Lampruna       | Lampruna rosea              | Lampruna rosea              | Schaus, 1894                 |    1816605 |      nan           |
|            nan | Xylophanes cthulhu maybe                    | Xylophanes cthulhu Haxaire & Vaglia, 2008        |  7731757 | HIGHERRANK  |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sphingidae     | Xylophanes     | Xylophanes cthulhu          | Xylophanes cthulhu          | Haxaire & Vaglia, 2008       |    7731757 |      nan           |
|            nan | Enyo lugubris                               | Enyo lugubris Linnaeus, 1771                     |  5124250 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sphingidae     | Enyo           | Enyo lugubris               | Enyo lugubris               | Linnaeus, 1771               |    5124250 |      nan           |
|            nan | fungus                                      | Fungus Adanson ex O.Kuntze, 1898                 |  2522460 | EXACT       |           93 | SYNONYM  | GENUS   | Fungi     | Basidiomycota | Agaricomycetes | Agaricales      | Agaricaceae    | Agaricus       | nan                         | Fungus                      | Adanson ex O.Kuntze, 1898    |    2522460 |        2.51865e+06 |
|            nan | Nyceryx stuarti                             | Nyceryx stuarti Rothschild, 1894                 |  1863146 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sphingidae     | Nyceryx        | Nyceryx stuarti             | Nyceryx stuarti             | Rothschild, 1894             |    1863146 |      nan           |
|            nan | Crambidae delta                             | Crambidae                                        |     8841 | HIGHERRANK  |           94 | ACCEPTED | FAMILY  | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | nan            | nan                         | Crambidae                   | nan                          |       8841 |      nan           |
|            nan | Sosxetra grata                              | Sosxetra grata Walker, 1862                      |  5110390 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Sosxetra       | Sosxetra grata              | Sosxetra grata              | Walker, 1862                 |    5110390 |      nan           |
|            nan | Eacles imperialis                           | Eacles imperialis Drury, 1773                    |  1865107 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Saturniidae    | Eacles         | Eacles imperialis           | Eacles imperialis           | Drury, 1773                  |    1865107 |      nan           |
|            nan | Spodoptera                                  | Spodoptera Guenée, 1852                          |  1765207 | EXACT       |           92 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Spodoptera     | nan                         | Spodoptera                  | Guenée, 1852                 |    1765207 |      nan           |
|            nan | Herminocala sabata                          | Herminocala Hampson, 1913                        |  9401599 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Herminocala    | nan                         | Herminocala                 | Hampson, 1913                |    9401599 |      nan           |
|            nan | Rhodochlora rothschildi                     | Rhodochlora rothschildi Warren, 1901             |  1985631 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Rhodochlora    | Rhodochlora rothschildi     | Rhodochlora rothschildi     | Warren, 1901                 |    1985631 |      nan           |
|            nan | Prolimacodes badia                          | Prolimacodes badia (Hübner, 1835)                |  1752825 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Limacodidae    | Prolimacodes   | Prolimacodes badia          | Prolimacodes badia          | (Hübner, 1835)               |    1752825 |      nan           |
|            nan | cicada                                      | Cicada Linnaeus, 1758                            |  1682542 | EXACT       |           93 | DOUBTFUL | GENUS   | Animalia  | Arthropoda    | Insecta        | Hemiptera       | Cicadidae      | Cicada         | nan                         | Cicada                      | Linnaeus, 1758               |    1682542 |      nan           |
|            nan | Hemiceras singuloides                       | Hemiceras singuloides Dyar, 1908                 |  5118648 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Notodontidae   | Hemiceras      | Hemiceras violascens        | Hemiceras singuloides       | Dyar, 1908                   |    5118648 |        5.11866e+06 |
|            nan | Macrosoma cascaria                          | Macrosoma cascaria Schaus, 1901                  |  5150043 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Hedylidae      | Macrosoma      | Macrosoma cascaria          | Macrosoma cascaria          | Schaus, 1901                 |    5150043 |      nan           |
|            nan | Eulepte concordalis                         | Eulepte concordalis Hübner, 1826                 |  1885178 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Eulepte        | Eulepte concordalis         | Eulepte concordalis         | Hübner, 1826                 |    1885178 |      nan           |
|            nan | Lineodes sp_san_luis_a                      | Lineodes Guenée, 1854                            |  1892404 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Lineodes       | nan                         | Lineodes                    | Guenée, 1854                 |    1892404 |      nan           |
|            nan | Synclera jarbusalis                         | Synclera jarbusalis Walker, 1859                 |  1884779 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Synclera       | Synclera traducalis         | Synclera jarbusalis         | Walker, 1859                 |    1884779 |        1.88477e+06 |
|            nan | Argyrosticta aurifundens                    | Argyrosticta aurifundens Walker, 1858            |  1768982 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Argyrosticta   | Argyrosticta aurifundens    | Argyrosticta aurifundens    | Walker, 1858                 |    1768982 |      nan           |
|            nan | Ischnurges eudamidasalis                    | Ischnurges Lederer, 1863                         |  1887584 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Ischnurges     | nan                         | Ischnurges                  | Lederer, 1863                |    1887584 |      nan           |
|            nan | Tortyra slossonia                           | Tortyra slossonia Fernald, 1900                  |  1860443 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Choreutidae    | Tortyra        | Tortyra slossonia           | Tortyra slossonia           | Fernald, 1900                |    1860443 |      nan           |
|            nan | Thoscora acca                               | Thoscora acca (Schaus, 1892)                     |  1751521 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Megalopygidae  | Thoscora       | Thoscora acca               | Thoscora acca               | (Schaus, 1892)               |    1751521 |      nan           |
|            nan | Epitausa flagrans                           | Epitausa flagrans Walker, 1869                   |  1772204 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Epitausa       | Epitausa flagrans           | Epitausa flagrans           | Walker, 1869                 |    1772204 |      nan           |
|            nan | Syngamia florella                           | Syngamia florella Cramer, 1781                   |  1885950 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Syngamia       | Syngamia florella           | Syngamia florella           | Cramer, 1781                 |    1885950 |      nan           |
|            nan | Syngamilyta apicolor                        | Syngamilyta apicolor                             | 10060843 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Syngamilyta    | Syngamilyta apicolor        | Syngamilyta apicolor        | nan                          |   10060843 |      nan           |
|            nan | Sarsina purpurascens                        | Sarsina purpurascens Walker, 1855                |  1819310 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Sarsina        | Sarsina purpurascens        | Sarsina purpurascens        | Walker, 1855                 |    1819310 |      nan           |
|            nan | Pseudogerespa usipetes                      | Pseudogerespa usipetes Druce, 1898               |  1794238 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Pseudogerespa  | Pseudogerespa usipetes      | Pseudogerespa usipetes      | Druce, 1898                  |    1794238 |      nan           |
|            nan | Euglyphis gutturalis                        | Euglyphis gutturalis                             | 10019673 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Lasiocampidae  | Euglyphis      | Euglyphis gutturalis        | Euglyphis gutturalis        | nan                          |   10019673 |      nan           |
|            nan | Hemiceras metastigma                        | Hemiceras metastigma Walker, 1857                |  5118614 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Notodontidae   | Hemiceras      | Hemiceras metastigma        | Hemiceras metastigma        | Walker, 1857                 |    5118614 |      nan           |
|            nan | Eucereon xanthura                           | Eucereon Hübner, 1819                            |  1810140 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | nan                         | Eucereon                    | Hübner, 1819                 |    1810140 |      nan           |
|            nan | Euglyphis larunda--obliterata               | Euglyphis Hübner, 1820                           |  7899191 | HIGHERRANK  |           92 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Lasiocampidae  | Euglyphis      | nan                         | Euglyphis                   | Hübner, 1820                 |    7899191 |      nan           |
|            nan | Empelathra amplificans                      | Empelathra amplificans Walker, 1858              |  1768679 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Empelathra     | Empelathra amplificans      | Empelathra amplificans      | Walker, 1858                 |    1768679 |      nan           |
|            nan | Eulepidotis juncida                         | Eulepidotis juncida Guenée, 1852                 |  1763163 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eulepidotis    | Eulepidotis juncida         | Eulepidotis juncida         | Guenée, 1852                 |    1763163 |      nan           |
|            nan | Sarsina festiva                             | Sarsina festiva Schaus, 1912                     |  1819305 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Sarsina        | Sarsina festiva             | Sarsina festiva             | Schaus, 1912                 |    1819305 |      nan           |
|            nan | Rothschildia erycina                        | Rothschildia erycina Shaw                        |  5124595 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Saturniidae    | Rothschildia   | Rothschildia erycina        | Rothschildia erycina        | Shaw                         |    5124595 |      nan           |
|            nan | Cyanopepla scintillans                      | Cyanopepla scintillans Butler, 1872              |  1805932 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Cyanopepla     | Cyanopepla scintillans      | Cyanopepla scintillans      | Butler, 1872                 |    1805932 |      nan           |
|            nan | Eucereon flavicaput                         | Eucereon Hübner, 1819                            |  1810140 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eucereon       | nan                         | Eucereon                    | Hübner, 1819                 |    1810140 |      nan           |
|            nan | Antiblemma lothos                           | Antiblemma lothos Cramer, 1777                   |  1794739 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Antiblemma     | Antiblemma lothos           | Antiblemma lothos           | Cramer, 1777                 |    1794739 |      nan           |
|            nan | Bagisara tristicta                          | Bagisara tristicta Hampson, 1898                 |  9511345 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Bagisara       | Bagisara tristicta          | Bagisara tristicta          | Hampson, 1898                |    9511345 |      nan           |
|            nan | Cerconota impresella                        | Cerconota impressella Walker, 1864               |  1836073 | FUZZY       |           96 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Depressariidae | Cerconota      | Cerconota impressella       | Cerconota impressella       | Walker, 1864                 |    1836073 |      nan           |
|            nan | Bertholdia specularis                       | Bertholdia specularis Herrich-Schäffer, 1853     |  5115498 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Bertholdia     | Bertholdia specularis       | Bertholdia specularis       | Herrich-Schäffer, 1853       |    5115498 |      nan           |
|            nan | Argyrotome perspecta                        | Argyrotome Warren, 1894                          |  1967337 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Argyrotome     | nan                         | Argyrotome                  | Warren, 1894                 |    1967337 |      nan           |
|            nan | Parasa wellesca                             | Parasa wellesca Dyar, 1900                       |  5105447 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Limacodidae    | Parasa         | Parasa wellesca             | Parasa wellesca             | Dyar, 1900                   |    5105447 |      nan           |
|            nan | Macrotes commatica                          | Macrotes commatica Prout, 1916                   |  1959982 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Macrotes       | Macrotes commatica          | Macrotes commatica          | Prout, 1916                  |    1959982 |      nan           |
|            nan | Xylophanes titana                           | Xylophanes titana Druce, 1878                    |  1862094 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sphingidae     | Xylophanes     | Xylophanes titana           | Xylophanes titana           | Druce, 1878                  |    1862094 |      nan           |
|            nan | Amastus episcotosia                         | Amastus episcotosia Dognin, 1901                 |  5117262 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Amastus        | Amastus episcotosia         | Amastus episcotosia         | Dognin, 1901                 |    5117262 |      nan           |
|            nan | Nebulosa erymas                             | Nebulosa erymas                                  | 10422460 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Notodontidae   | Nebulosa       | Nebulosa erymas             | Nebulosa erymas             | nan                          |   10422460 |      nan           |
|            nan | Pupa chrysalis                              | Pupa chrysalis Férussac, 1832                    | 11331817 | EXACT       |           98 | SYNONYM  | SPECIES | Animalia  | Mollusca      | Gastropoda     | Stylommatophora | Cerionidae     | Cerion         | Cerion mumia                | Pupa chrysalis              | Férussac, 1832               |   11331817 |        1.13773e+07 |
|            nan | Udea rubigalis                              | Udea rubigalis Guenée, 1854                      |  1887122 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Udea           | Udea rubigalis              | Udea rubigalis              | Guenée, 1854                 |    1887122 |      nan           |
|            nan | Gonodonta                                   | Gonodonta Hübner, 1818                           |  1770011 | EXACT       |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Gonodonta      | nan                         | Gonodonta                   | Hübner, 1818                 |    1770011 |      nan           |
|            nan | Bradina aaronalis                           | Bradina aaronalis Schaus, 1924                   |  1885834 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Bradina        | Bradina aaronalis           | Bradina aaronalis           | Schaus, 1924                 |    1885834 |      nan           |
|            nan | Orthoptera                                  | Orthoptera                                       |     1458 | EXACT       |           94 | ACCEPTED | ORDER   | Animalia  | Arthropoda    | Insecta        | Orthoptera      | nan            | nan            | nan                         | Orthoptera                  | nan                          |       1458 |      nan           |
|            nan | Antaeotricha stigmatias                     | Antaeotricha Zeller, 1854                        |  1838338 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Depressariidae | Antaeotricha   | nan                         | Antaeotricha                | Zeller, 1854                 |    1838338 |      nan           |
|            nan | Xylophanes cthulhu                          | Xylophanes cthulhu Haxaire & Vaglia, 2008        |  7731757 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sphingidae     | Xylophanes     | Xylophanes cthulhu          | Xylophanes cthulhu          | Haxaire & Vaglia, 2008       |    7731757 |      nan           |
|            nan | Diaphania praxialis                         | Diaphania Hübner, 1818                           |  3258127 | HIGHERRANK  |           92 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | Diaphania      | nan                         | Diaphania                   | Hübner, 1818                 |    3258127 |      nan           |
|            nan | Carathis byblis--septentrionalis            | Carathis Grote, 1865                             |  1806001 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Carathis       | nan                         | Carathis                    | Grote, 1865                  |    1806001 |      nan           |
|            nan | Parasa viridogrisea                         | Parasa viridogrisea (Dyar, 1898)                 |  5105518 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Limacodidae    | Parasa         | Parasa viridogrisea         | Parasa viridogrisea         | (Dyar, 1898)                 |    5105518 |      nan           |
|            nan | Xylophanes porcus                           | Xylophanes porcus Hübner                         |  1862088 | EXACT       |           98 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sphingidae     | Xylophanes     | Xylophanes porcus           | Xylophanes porcus           | Hübner                       |    1862088 |      nan           |
|            nan | Rhodochlora brunneipalpis                   | Rhodochlora brunneipalpis Warren, 1894           |  1985647 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Rhodochlora    | Rhodochlora brunneipalpis   | Rhodochlora brunneipalpis   | Warren, 1894                 |    1985647 |      nan           |
|            nan | Idalus borealis                             | Idalus borealis                                  |  1814704 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Idalus         | Idalus borealis             | Idalus borealis             | nan                          |    1814704 |      nan           |
|            nan | Gonodonta pulverea                          | Gonodonta pulverea Schaus, 1911                  |  1770056 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Gonodonta      | Gonodonta pulverea          | Gonodonta pulverea          | Schaus, 1911                 |    1770056 |      nan           |
|            nan | Patalene falcularia                         | Patalene falcularia (Sepp, 1854)                 |  1978658 | EXACT       |           97 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Patalene       | Patalene falcularia         | Patalene falcularia         | (Sepp, 1854)                 |    1978658 |      nan           |
|            nan | Fulgoridae                                  | Fulgoridae                                       |     8585 | EXACT       |           94 | ACCEPTED | FAMILY  | Animalia  | Arthropoda    | Insecta        | Hemiptera       | Fulgoridae     | nan            | nan                         | Fulgoridae                  | nan                          |       8585 |      nan           |
|            nan | Eois sp_san_luis_c                          | Eois Hübner, 1818                                |  1989610 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eois           | nan                         | Eois                        | Hübner, 1818                 |    1989610 |      nan           |
|            nan | Eois brunneicosta                           | Eois brunneicosta (Dognin, 1916)                 |  1989850 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eois           | Eois brunneicosta           | Eois brunneicosta           | (Dognin, 1916)               |    1989850 |      nan           |
|            nan | Eois sp_san_luis_j                          | Eois Hübner, 1818                                |  1989610 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eois           | nan                         | Eois                        | Hübner, 1818                 |    1989610 |      nan           |
|            nan | Letis scops                                 | Letis scops Guenée, 1852                         |  1788381 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Letis          | Letis scops                 | Letis scops                 | Guenée, 1852                 |    1788381 |      nan           |
|            nan | Perigonia ilus                              | Perigonia ilus Boisduval, 1870                   |  1861568 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Sphingidae     | Perigonia      | Perigonia ilus              | Perigonia ilus              | Boisduval, 1870              |    1861568 |      nan           |
|            nan | Automeris belti                             | Automeris belti Druce, 1886                      |  1866003 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Saturniidae    | Automeris      | Automeris belti             | Automeris belti             | Druce, 1886                  |    1866003 |      nan           |
|            nan | Acharia hyperoche                           | Acharia hyperoche                                |  9941073 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Limacodidae    | Acharia        | Acharia hyperoche           | Acharia hyperoche           | nan                          |    9941073 |      nan           |
|            nan | Notodontidae                                | Notodontidae                                     |     7016 | EXACT       |           94 | ACCEPTED | FAMILY  | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Notodontidae   | nan            | nan                         | Notodontidae                | nan                          |       7016 |      nan           |
|            nan | Nemoria astraeoides                         | Nemoria Hübner, 1818                             |  1981142 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Nemoria        | nan                         | Nemoria                     | Hübner, 1818                 |    1981142 |      nan           |
|            nan | Dalcerides mesoa                            | Dalcerides mesoa                                 | 10520508 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Dalceridae     | Dalcerides     | Dalcerides mesoa            | Dalcerides mesoa            | nan                          |   10520508 |      nan           |
|            nan | Bryolymnia bicon                            | Bryolymnia bicon Druce, 1889                     |  1764142 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Bryolymnia     | Bryolymnia bicon            | Bryolymnia bicon            | Druce, 1889                  |    1764142 |      nan           |
|            nan | Herminocala sp_san_luis_a                   | Herminocala Hampson, 1913                        |  9401599 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Herminocala    | nan                         | Herminocala                 | Hampson, 1913                |    9401599 |      nan           |
|            nan | Letis                                       | Letis Hübner, 1821                               |  1788326 | EXACT       |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Letis          | nan                         | Letis                       | Hübner, 1821                 |    1788326 |      nan           |
|            nan | Crambidia                                   | Crambidia Packard, 1864                          |  1805536 | EXACT       |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Crambidia      | nan                         | Crambidia                   | Packard, 1864                |    1805536 |      nan           |
|            nan | Macaria                                     | Arthropoda                                       |       54 | HIGHERRANK  |           92 | ACCEPTED | PHYLUM  | Animalia  | Arthropoda    | nan            | nan             | nan            | nan            | nan                         | Arthropoda                  | nan                          |         54 |      nan           |
|            nan | Tosale                                      | Tosale Walker, 1863                              |  1872509 | EXACT       |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Pyralidae      | Tosale         | nan                         | Tosale                      | Walker, 1863                 |    1872509 |      nan           |
|            nan | Pseudodirphia menander                      | Pseudodirphia menander Druce, 1886               | 10248521 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Saturniidae    | Pseudodirphia  | Pseudodirphia menander      | Pseudodirphia menander      | Druce, 1886                  |   10248521 |      nan           |
|            nan | Dalcerides gugelmanni                       | Dalcerides gugelmanni                            | 10501366 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Dalceridae     | Dalcerides     | Dalcerides gugelmanni       | Dalcerides gugelmanni       | nan                          |   10501366 |      nan           |
|            nan | Eois sp_san_luis_i                          | Eois Hübner, 1818                                |  1989610 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eois           | nan                         | Eois                        | Hübner, 1818                 |    1989610 |      nan           |
|            nan | Ctenucha sp_san_luis_a                      | Ctenucha Kirby, 1837                             |  1814799 | HIGHERRANK  |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Ctenucha       | nan                         | Ctenucha                    | Kirby, 1837                  |    1814799 |      nan           |
|            nan | Macrosoma rubedinaria                       | Macrosoma rubedinaria Walker, 1862               |  5150027 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Hedylidae      | Macrosoma      | Macrosoma rubedinaria       | Macrosoma rubedinaria       | Walker, 1862                 |    5150027 |      nan           |
|            nan | Crambidae                                   | Crambidae                                        |     8841 | EXACT       |           94 | ACCEPTED | FAMILY  | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Crambidae      | nan            | nan                         | Crambidae                   | nan                          |       8841 |      nan           |
|            nan | Tolype caieta                               | Tolype caieta Druce, 1897                        |  5102377 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Lasiocampidae  | Tolype         | Tolype caieta               | Tolype caieta               | Druce, 1897                  |    5102377 |      nan           |
|            nan | Saurita fumosa                              | Saurita fumosa Schaus, 1912                      |  1809500 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Saurita        | Saurita fumosa              | Saurita fumosa              | Schaus, 1912                 |    1809500 |      nan           |
|            nan | Josiomorpha penetrata                       | Josiomorpha penetrata Walker, 1865               |  1818424 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Josiomorpha    | Josiomorpha penetrata       | Josiomorpha penetrata       | Walker, 1865                 |    1818424 |      nan           |
|            nan | Eulepidotis hemura                          | Eulepidotis hemura                               | 10256607 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Eulepidotis    | Eulepidotis hemura          | Eulepidotis hemura          | nan                          |   10256607 |      nan           |
|            nan | Glena mosparia                              | Glena mopsaria (Schaus, 1913)                    |  1965683 | FUZZY       |           95 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Glena          | Glena mopsaria              | Glena mopsaria              | (Schaus, 1913)               |    1965683 |      nan           |
|            nan | Seirocastnia tribuna                        | Seirocastnia tribuna Hübner, 1825                |  1790400 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Noctuidae      | Seirocastnia   | Seirocastnia tribuna        | Seirocastnia tribuna        | Hübner, 1825                 |    1790400 |      nan           |
|            nan | Eusarca flexiilis                           | Eusarca flexilis (Schaus, 1912)                  | 11721130 | FUZZY       |           96 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eusarca        | Eusarca flexilis            | Eusarca flexilis            | (Schaus, 1912)               |   11721130 |      nan           |
|            nan | Macaria carpo                               | Macaria carpo (Druce, 1893)                      |  9338558 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Macaria        | Macaria carpo               | Macaria carpo               | (Druce, 1893)                |    9338558 |      nan           |
|            nan | Eusarca crameraria--brown                   | Eusarca Hübner                                   |  7514692 | HIGHERRANK  |           92 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eusarca        | nan                         | Eusarca                     | Hübner                       |    7514692 |      nan           |
|            nan | Elysius discoplaga                          | Elysius discoplaga Walker, 1856                  | 11667005 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Elysius        | Elysius discoplaga          | Elysius discoplaga          | Walker, 1856                 |   11667005 |      nan           |
|            nan | Morpheis strigifer                          | Morpheis Hübner, 1820                            |  1730685 | HIGHERRANK  |           92 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Cossidae       | Morpheis       | nan                         | Morpheis                    | Hübner, 1820                 |    1730685 |      nan           |
|            nan | Ptychoglene xylophila                       | Ptychoglene xylophila Druce, 1885                |  1803126 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Ptychoglene    | Ptychoglene xylophila       | Ptychoglene xylophila       | Druce, 1885                  |    1803126 |      nan           |
|            nan | Pachydota iodea                             | Pachydota iodea Herrich-Schäffer, 1855           |  1805995 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Erebidae       | Pachydota      | Pachydota iodea             | Pachydota iodea             | Herrich-Schäffer, 1855       |    1805995 |      nan           |
|            nan | Lymantriidae                                | Lymantriidae                                     |     5472 | EXACT       |           94 | ACCEPTED | FAMILY  | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Lymantriidae   | nan            | nan                         | Lymantriidae                | nan                          |       5472 |      nan           |
|            nan | Mapeta xanthomelas                          | Mapeta xanthomelas Walker, 1863                  |  1870308 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Pyralidae      | Mapeta         | Mapeta xanthomelas          | Mapeta xanthomelas          | Walker, 1863                 |    1870308 |      nan           |
|            nan | Oxydia sociata                              | Oxydia sociata (Warren, 1895)                    |  1976664 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Oxydia         | Oxydia sociata              | Oxydia sociata              | (Warren, 1895)               |    1976664 |      nan           |
|            nan | Glena moparia                               | Glena mopsaria (Schaus, 1913)                    |  1965683 | FUZZY       |           95 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Glena          | Glena mopsaria              | Glena mopsaria              | (Schaus, 1913)               |    1965683 |      nan           |
|            nan | Trygodes musivaria                          | Trygodes musivaria (Herrich-Schäffer, 1855)      |  1976366 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Trygodes       | Trygodes musivaria          | Trygodes musivaria          | (Herrich-Schäffer, 1855)     |    1976366 |      nan           |
|            nan | Eois                                        | Eois Hübner, 1818                                |  1989610 | EXACT       |           94 | ACCEPTED | GENUS   | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | Eois           | nan                         | Eois                        | Hübner, 1818                 |    1989610 |      nan           |
|            nan | Geometridae                                 | Geometridae                                      |     6950 | EXACT       |           94 | ACCEPTED | FAMILY  | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Geometridae    | nan            | nan                         | Geometridae                 | nan                          |       6950 |      nan           |
|            nan | Phassus n-signatus                          | Phassus n-signatus Weymer, 1907                  |  5119316 | EXACT       |           99 | ACCEPTED | SPECIES | Animalia  | Arthropoda    | Insecta        | Lepidoptera     | Hepialidae     | Phassus        | Phassus n-signatus          | Phassus n-signatus          | Weymer, 1907                 |    5119316 |      nan           |

In [7]:
# I then generated a list of species and their corresponding local image filenames that were an EXACT matchType at the SPECIES rank:

In [ ]:
import pandas as pd

# Input files
gbif_taxa = "unique_values_output_gbif_unmatched_excluded_moth_removed.csv"
dl_moths_all = "combined_output_moth_removed.csv"

# Output files
# good_matching_gbif_taxa = "good_matching_gbif_taxa.csv"
# dl_moths_matching_gbif = "dl_moths_matching_gbif.csv"
good_matching_gbif_taxa = "good_matching_gbif_taxa_not_just_species.csv"
dl_moths_matching_gbif = "dl_moths_matching_gbif_not_just_spiecies.csv"

filterBySpecies = False

# Step 1: Filter 1.csv for matchType == "EXACT" and rank == "SPECIES"
try:
    # Read 1.csv
    df1 = pd.read_csv(gbif_taxa)

    # Filter rows where matchType is "EXACT" and rank is "SPECIES"

    if(filterBySpecies):
        filtered_df1 = df1[(df1["matchType"] == "EXACT") & (df1["rank"] == "SPECIES")]
    else:
        # 
        filtered_df1 = df1[(df1["matchType"] == "EXACT") ]

    # Get unique verbatimScientificName values
    unique_scientific_names = filtered_df1["verbatimScientificName"].drop_duplicates()

    # Save to a.csv
    unique_scientific_names.to_csv(good_matching_gbif_taxa, index=False, header=["verbatimScientificName"])

    print(f"Filtered scientific names written to {good_matching_gbif_taxa}")

    # Step 2: Match scientificName values from 2.csv with values in a.csv
    # Read 2.csv
    df2 = pd.read_csv(dl_moths_all)

    # Filter rows where scientificName is in the list from a.csv
    matching_df2 = df2[df2["scientificName"].isin(unique_scientific_names)]

    # Select filename and scientificName columns
    result_df2 = matching_df2[["Filename", "scientificName"]]

    # Save to b.csv
    result_df2.to_csv(dl_moths_matching_gbif, index=False)

    print(f"Matching entries written to {dl_moths_matching_gbif}")

except Exception as e:
    print(f"An error occurred: {e}")
